In [126]:
import gdown

gurl = 'https://drive.google.com/uc?id=1-kBJM45JuxTtel58mMC7nmmaFJIPgMUC'
gdown.download(gurl, 'tube_type.csv.xz')

Downloading...
From: https://drive.google.com/uc?id=1-kBJM45JuxTtel58mMC7nmmaFJIPgMUC
To: /home/painstudy/dl_ETT/tube_type.csv.xz
100%|██████████| 290k/290k [00:00<00:00, 1.07MB/s]


'tube_type.csv.xz'

In [16]:
df

,opid,sex,age,weight,height,airway_tube_size,airway_tube_type,airway_lma,old_tube_size
542702,190226144,True,8,38.70,138.9,6.5,plain,NaN,6.0
191113,100920106,True,3,15.00,97.6,5.0,plain,NaN,5.0
347799,140715139,True,1,7.75,72.3,4.5,plain,NaN,4.0
73080,70319098,True,10,59.20,165.5,6.0,plain,NaN,6.5
215319,110513102,False,5,21.00,113.9,6.0,plain,NaN,5.0
...,...,...,...,...,...,...,...,...,...
232735,111019127,True,2,11.05,87.7,4.5,plain,NaN,4.5
244097,120130155,True,9,43.30,142.8,6.0,plain,NaN,6.0
142725,90513112,False,9,27.15,132.0,6.0,plain,NaN,6.0
495283,180105178,True,4,14.80,96.6,4.5,plain,NaN,5.0


In [20]:
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61951 entries, 542702 to 151649
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   opid              61951 non-null  int64  
 1   sex               61951 non-null  bool   
 2   age               61951 non-null  int64  
 3   weight            61921 non-null  float64
 4   height            61396 non-null  float64
 5   airway_tube_size  61951 non-null  float64
 6   airway_tube_type  61951 non-null  object 
 7   airway_lma        32 non-null     float64
 8   old_tube_size     61951 non-null  float64
dtypes: bool(1), float64(5), int64(2), object(1)
memory usage: 4.3+ MB


In [21]:
df0 = pd.read_csv('tube.csv')
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695913 entries, 0 to 695912
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   opid              695913 non-null  int64  
 1   sex               695913 non-null  object 
 2   age               695913 non-null  float64
 3   weight            692133 non-null  float64
 4   height            691875 non-null  float64
 5   airway_tube_size  428650 non-null  float64
 6   airway_tube_type  492572 non-null  object 
 7   airway_lma        44047 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 42.5+ MB


In [22]:
size_lis = df['airway_tube_size'].values
np.min(size_lis), np.max(size_lis), np.mean(size_lis)

(2.0, 8.5, 4.817557424416071)

# Dataset loading

In [1]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils import shuffle
import pandas as pd
import xgboost as xgb
import numpy as np
import random, os, datetime, pickle
import scipy
import matplotlib.pyplot as plt
import numpy as np
import csv

df = pd.read_csv('tube.csv')
df.loc[df['weight'] <= 0, 'weight'] = None
df.loc[df['height'] <= 0, 'height'] = None
df['age'] = df['age'].astype(int)
df = df.loc[df['age'] <= 10]
df = df.loc[df['airway_tube_type'] == 'plain']
# [nan 'plain' 'RAE(oral)' 'reinforced' 'LMA' 'T-tube' 'CobraPLA', 'double lumen tube' 'RAE(nasal)' 'laser' 'combitube' 'univent']
df = df.loc[~df['airway_tube_size'].isnull()]
df.dropna(subset=['age', 'sex'], inplace=True)
df['sex'] = (df['sex'] == 'M')

INPUT_VARS = ['age','sex','weight','height']
TARGET_VAR = 'airway_tube_size'

OLD_VAR = 'old_tube_size'
df[OLD_VAR] = np.round((df['age'] / 4 + 4) * 2) / 2

random.seed(0)
df = shuffle(df)
y = df[[TARGET_VAR]].values.flatten().astype(float)
y_old = df[[OLD_VAR]].values.flatten().astype(float)
x = df.loc[:, INPUT_VARS].values.astype(float)

# training set의 뒤쪽 20%를 test set 으로 사용
nsamp = len(y)
ntest = int(nsamp * 0.2)
ntrain = nsamp - ntest
x_test = x[-ntest:, :]
y_test = y[-ntest:]
y_test_old = y_old[-ntest:]
x_train = x[:ntrain, :]
y_train = y[:ntrain]

In [5]:
# encoding : 2 -> 0, 2.5 -> 1, ... 8.5 -> 14 로 tube size x 2 - 4
df['label'] = df[TARGET_VAR].apply(lambda x: x*2-4)

for i in range(14):
    df[f'{i}'] = df['label'].apply(lambda x: 1 if x==i else 0)
    
y_label = df[[f'{i}' for i in range(14)]].values
yl_test = y_label[-ntest:]
yl_train = y_label[:ntrain]

# Modeling

## Age-based formula (Cole)

In [33]:
print(f'acc: {np.mean(y_test_old==y_test):.3f}')
print(f'acc(+-0.5mm): {np.mean((y_test_old >= y_test-0.5) & (y_test_old <= y_test+0.5)):.3f}')

acc: 0.353
acc(+-0.5mm): 0.847


## XGBR (randomsearchCV)

In [2]:
param_dict = {
                #'learning_rate': [ 0.05, 0.07, 0.1], #[0.01, 0.03, 0.05],
                'max_depth': [4, 5, 7],#[3,4,5],
                'n_estimators': [100, 200, 300],
                #'n_estimators': [100],#[25, 50, 75, 100],
                #'n_estimators': [50],
                'subsample': [0.5, 0.8], #[0.5, 0.8, 1],
                'colsample_bytree': [0.5, 0.8], #[0.8, 1],
                #'gamma': [0.9], #[0.3, 0.5, 0.7, 0.9],
                #'scale_pos_weight': [5, 10], #[1,10,30,100]
            }
nfold = 5
gs = RandomizedSearchCV(n_iter=100,
                        estimator=xgb.sklearn.XGBRegressor(),
                        n_jobs=-1,
                        verbose=10,
                        param_distributions=param_dict, cv=nfold)
gs.fit(x_train, y_train)
model = gs.best_estimator_.get_booster()

print()
print("========= found hyperparameter =========")
print(gs.best_params_)
print(gs.best_score_)
print("========================================")

y_pred = gs.predict(x_test).flatten()
y_pred = np.round(y_pred * 2) / 2

print('--------------')
print('old model = age/4+4')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test, y_test_old):.3f}')
print(f'mean_squared_errors: {mean_squared_error(y_test, y_test_old):.3f}')
print(f'r2_score: {r2_score(y_test, y_test_old):.3f}')
# accuracy
acc1 = np.mean(y_test_old==y_test)
acc3 = np.mean((y_test_old >= y_test-0.5) & (y_test_old <= y_test+0.5))
print(f'acc: {acc1:.3f}')
print(f'acc(+-0.5mm): {acc3:.3f}')


xvals = []
yvals = []
cvals = []
cvals_old = []
for x in np.arange(3, 8, 0.5):
    for y in np.arange(3, 8, 0.5):
        xvals.append(x)
        yvals.append(y)
        cvals.append(sum((y_pred == x) & (y_test == y)))
        cvals_old.append(sum((y_test_old == x) & (y_test == y)))
xvals = np.array(xvals)
yvals = np.array(yvals)
cvals = np.array(cvals) / 2
cvals_old = np.array(cvals_old) / 2

plt.figure(figsize=(8, 8))
plt.xlim(2, 8)
plt.ylim(2, 8)
plt.scatter(xvals, yvals, c='red', alpha=0.5, s=cvals_old, label='4+AGE/4')
plt.xlabel('Prediction')
plt.ylabel('Ground Truth')
lgnd = plt.legend()
lgnd.legendHandles[0]._sizes = [30]
plt.plot([2,8], [2,8], 'k-', alpha=0.1)
plt.savefig('old.png')
plt.close()

plt.figure(figsize=(8, 8))
plt.xlim(2, 8)
plt.ylim(2, 8)
plt.scatter(xvals, yvals, c='blue', alpha=0.5, s=cvals, label='GBRT')
plt.xlabel('Prediction')
plt.ylabel('Ground Truth')
lgnd = plt.legend()
lgnd.legendHandles[0]._sizes = [30]
plt.plot([2,8], [2,8], 'k-', alpha=0.1)
plt.savefig('new.png')
plt.close()

print('--------------')
print('new model')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test, y_pred):.3f}')
print(f'mean_squared_errors: {mean_squared_error(y_test, y_pred):.3f}')
print(f'r2_score: {r2_score(y_test, y_pred):.3f}')
# accuracy
acc1 = np.mean(y_pred==y_test)
acc3 = np.mean((y_pred >= y_test-0.5) & (y_pred <= y_test+0.5))
print(f'acc: {acc1:.3f}')
print(f'acc(+-0.5mm): {acc3:.3f}')


# save model
odir_f = f'acc1-{acc1:.3f}_acc3-{acc3:.3f}_XGBR_{nfold}fold'
odir = f'result/{odir_f}'
if not os.path.exists(odir):
    os.mkdir(odir)
model.save_model(f'{odir}/model.model')

# 모델에 대한 정보 txt로 저장
pickle.dump(param_dict, open(f'{odir}/param_dict', 'wb'))
f = open(f'{odir}/result.txt', 'w')
f.write(f'regression model')
f.write(f'explained_variance_score: {explained_variance_score(y_test, y_pred):.3f}')
f.write(f'mean_squared_errors: {mean_squared_error(y_test, y_pred):.3f}')
f.write(f'r2_score: {r2_score(y_test, y_pred):.3f}')
f.close()

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/home/painstudy/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 36 is smaller than n_iter=100. Running 36 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(



========= found hyperparameter =========
{'subsample': 0.8, 'n_estimators': 100, 'max_depth': 5, 'colsample_bytree': 0.8}
0.8187579741290584
--------------
old model = age/4+4
--------------
explained_variance_score: 0.670
mean_squared_errors: 0.337
r2_score: 0.669
--------------
new model
--------------
explained_variance_score: 0.798
mean_squared_errors: 0.206
r2_score: 0.798
acc: 0.452
acc(+-0.5mm): 0.924
[CV 4/5; 2/36] START colsample_bytree=0.5, max_depth=4, n_estimators=100, subsample=0.8
[CV 4/5; 2/36] END colsample_bytree=0.5, max_depth=4, n_estimators=100, subsample=0.8;, score=0.820 total time=   2.7s
[CV 1/5; 10/36] START colsample_bytree=0.5, max_depth=5, n_estimators=200, subsample=0.8
[CV 1/5; 10/36] END colsample_bytree=0.5, max_depth=5, n_estimators=200, subsample=0.8;, score=0.810 total time=   5.0s
[CV 2/5; 18/36] START colsample_bytree=0.5, max_depth=7, n_estimators=300, subsample=0.8
[CV 2/5; 18/36] END colsample_bytree=0.5, max_depth=7, n_estimators=300, subsample

In [26]:
print(f'acc: {np.mean(y_pred==y_test):.3f}')
print(f'acc(+-0.5mm): {np.mean((y_pred >= y_test-0.5) & (y_pred <= y_test+0.5)):.3f}')

acc: 0.460
acc(+-0.5mm): 0.928


### Classification

In [15]:
param_dict = {
                #'learning_rate': [ 0.05, 0.07, 0.1], #[0.01, 0.03, 0.05],
                'max_depth': [4, 5, 7],#[3,4,5],
                'n_estimators': [100, 200, 300],
                #'n_estimators': [100],#[25, 50, 75, 100],
                #'n_estimators': [50],
                'subsample': [0.5, 0.8], #[0.5, 0.8, 1],
                'colsample_bytree': [0.5, 0.8], #[0.8, 1],
                #'gamma': [0.9], #[0.3, 0.5, 0.7, 0.9],
                #'scale_pos_weight': [5, 10], #[1,10,30,100]
            }
nfold = 5
gs = RandomizedSearchCV(n_iter=100,
                        estimator=xgb.sklearn.XGBClassifier(),
                        n_jobs=-1,
                        verbose=10,
                        param_distributions=param_dict, cv=nfold)
gs.fit(x_train, yl_train)
model = gs.best_estimator_.get_booster()

print()
print("========= found hyperparameter =========")
print(gs.best_params_)
print(gs.best_score_)
print("========================================")

y_pred = gs.predict(x_test).argmax(1)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/home/painstudy/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 36 is smaller than n_iter=100. Running 36 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(



========= found hyperparameter =========
{'subsample': 0.8, 'n_estimators': 300, 'max_depth': 7, 'colsample_bytree': 0.8}
0.29149939216950177


In [30]:
yl_test_ = yl_test.argmax(1)
print(f'acc: {np.mean(y_pred==yl_test_):.3f}')
print(f'acc(+-0.5mm): {np.mean((y_pred >= yl_test_-1) & (y_pred <= yl_test_+1)):.3f}')

acc: 0.291
acc(+-0.5mm): 0.538


In [ ]:
print('--------------')
print('new model')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test, y_pred):.3f}')
print(f'mean_squared_errors: {mean_squared_error(y_test, y_pred):.3f}')
print(f'r2_score: {r2_score(y_test, y_pred):.3f}')
# accuracy
acc1 = np.mean(y_pred==y_test)
acc3 = np.mean((y_pred >= y_test-0.5) & (y_pred <= y_test+0.5))
print(f'acc: {acc1:.3f}')
print(f'acc(+-0.5mm): {acc3:.3f}')


# save model
odir_f = f'acc1-{acc1:.3f}_acc3-{acc3:.3f}_XGBR_{nfold}fold'
odir = f'result/{odir_f}'
if not os.path.exists(odir):
    os.mkdir(odir)
model.save_model(f'{odir}/model.model')

# 모델에 대한 정보 txt로 저장
pickle.dump(param_dict, open(f'{odir}/param_dict', 'wb'))
f = open(f'{odir}/result.txt', 'w')
f.write(f'classification model')
f.write(f'explained_variance_score: {explained_variance_score(y_test, y_pred):.3f}')
f.write(f'mean_squared_errors: {mean_squared_error(y_test, y_pred):.3f}')
f.write(f'r2_score: {r2_score(y_test, y_pred):.3f}')
f.close()

## Gradient Boosting

In [32]:
from sklearn.ensemble import GradientBoostingRegressor

print('#####GradientBoostingRegressor#####')
gbr = RandomizedSearchCV(n_iter=100,
                        estimator=GradientBoostingRegressor(),
                        n_jobs=-1,
                        verbose=5,
                        param_distributions={
                            'learning_rate': [ 0.05, 0.07, 0.1], #[0.01, 0.03, 0.05],
                            'max_depth': [4, 5, 7],#[3,4,5],
                            'n_estimators': [100, 200, 300],
                            #'n_estimators': [100],#[25, 50, 75, 100],
                            #'n_estimators': [50],
                            'subsample': [0.5, 0.8], #[0.5, 0.8, 1],
                            #'gamma': [0.9], #[0.3, 0.5, 0.7, 0.9],
                            #'scale_pos_weight': [5, 10], #[1,10,30,100],
                        }, cv=5)
gbr.fit(x_train, y_train)
model = gbr.best_estimator_.get_booster()

print()
print("========= found hyperparameter =========")
print(gbr.best_params_)
print(gbr.best_score_)
print("========================================")

y_pred = gbr.predict(x_test).flatten()
y_pred = np.round(y_pred * 2) / 2

print('--------------')
print('old model = age/4+4')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test, y_test_old):.3f}')
print(f'mean_squared_errors: {mean_squared_error(y_test, y_test_old):.3f}')
print(f'r2_score: {r2_score(y_test, y_test_old):.3f}')


print('--------------')
print('new model')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test, y_pred):.3f}')
print(f'mean_squared_errors: {mean_squared_error(y_test, y_pred):.3f}')
print(f'r2_score: {r2_score(y_test, y_pred):.3f}')
print(f'acc: {np.mean(y_pred==y_test):.3f}')
print(f'acc(+-0.5mm): {np.mean((y_pred >= y_test-0.5) & (y_pred <= y_test+0.5)):.3f}')

#####GradientBoostingRegressor#####
Fitting 5 folds for each of 18 candidates, totalling 90 fits


/home/painstudy/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 18 is smaller than n_iter=100. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/home/painstudy/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
90 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/home/painstudy/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/painstudy/.local/lib/python3.8/s

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

[CV 2/5] END max_depth=4, n_estimators=200, subsample=0.5;, score=nan total time=   0.0s
[CV 3/5] END max_depth=7, n_estimators=200, subsample=0.8;, score=nan total time=   0.0s
[CV 2/5] END max_depth=4, n_estimators=100, subsample=0.5;, score=nan total time=   0.0s
[CV 5/5] END max_depth=5, n_estimators=200, subsample=0.5;, score=nan total time=   0.0s
[CV 4/5] END max_depth=5, n_estimators=200, subsample=0.8;, score=nan total time=   0.0s
[CV 2/5] END max_depth=5, n_estimators=300, subsample=0.8;, score=nan total time=   0.0s
[CV 1/5] END max_depth=7, n_estimators=100, subsample=0.8;, score=nan total time=   0.0s
[CV 4/5] END max_depth=7, n_estimators=200, subsample=0.8;, score=nan total time=   0.0s
[CV 3/5] END max_depth=4, n_estimators=100, subsample=0.8;, score=nan total time=   0.0s
[CV 3/5] END max_depth=5, n_estimators=300, subsample=0.8;, score=nan total time=   0.0s
[CV 2/5] END max_depth=7, n_estimators=100, subsample=0.8;, score=nan total time=   0.0s
[CV 5/5] END max_dept

In [31]:
GradientBoostingRegressor().get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

## RandomForest

## sklearn.model_selection

### manual randomsearch

In [ ]:
COL_DROP = SIGNAL_KEYS + ['data', 'label', 'filename', 'lung_resistance']
df_ = df.dropna()
X = np.array(df_.drop(COL_DROP, axis=1))
y = np.array(df_['label'])
groups = np.array(df_['person_id'])

gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=SEED)
for (train_index, test_index) in gss.split(X, y, groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    groups_train, groups_test = groups[train_index], groups[test_index]
    
print(set(groups_train) & set(groups_test))
X_shuffled, y_shuffled, groups_shuffled = shuffle(X_train, y_train, groups_train, random_state=SEED)

group_kf = GroupKFold(n_splits=5)

N_ESTIMATORS = [50, 100]
MAX_DEPTH = [6, 7, 8, 9, 10, 11]
MIN_SAMPLES_LEAF = [5, 6]
MIN_SAMPLES_SPLIT = [7, 8]
HYPERPARAMS = list(itertools.product(N_ESTIMATORS, MAX_DEPTH, MIN_SAMPLES_LEAF, MIN_SAMPLES_SPLIT))

BEST_MODEL, BEST_F1 = None, 0.0

for i, (train_index, val_index) in enumerate(group_kf.split(X_shuffled, groups=groups_shuffled)):
    print(f'\n===== Fold #{i + 1} =====\n')
    train_set = set(groups_shuffled[train_index])
    val_set = set(groups_shuffled[val_index])
    print(train_set & val_set)
    
    X_train, X_val = X_shuffled[train_index], X_shuffled[val_index]
    y_train, y_val = y_shuffled[train_index], y_shuffled[val_index]
    
    print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
    
    for (n_estimators, max_depth, min_samples_leaf, min_samples_split) in HYPERPARAMS:

        model = RandomForestClassifier(n_estimators=n_estimators,
                                       max_depth=max_depth,
                                       min_samples_leaf=min_samples_leaf,
                                       min_samples_split=min_samples_split)
        model.fit(X_train, y_train)

        y_train_prob = model.predict_proba(X_train)[:, 1]
        y_train_pred = model.predict(X_train)
        
        y_val_prob = model.predict_proba(X_val)[:, 1]
        y_val_pred = model.predict(X_val)

        train_auc = roc_auc_score(y_train, y_train_prob)
        train_f1  = f1_score(y_train, y_train_pred, average="macro")
        
        val_auc   = roc_auc_score(y_val, y_val_prob)
        val_f1    = f1_score(y_val, y_val_pred, average="macro")
        
        print(f'[{n_estimators}, {max_depth:02d}, {min_samples_leaf}, {min_samples_split}]  ', end='')
        print(f'TRAIN AUC {train_auc:.4f}  Macro-F1 {train_f1:.4f}\t', end='')
        print(f'VAL AUC {val_auc:.4f}  Macro-F1 : {val_f1:.4f}\t', end='')

        if val_f1 > BEST_F1:
            BEST_MODEL, BEST_F1 = model, val_f1
            print('*', end='')
        print()

In [ ]:
### TEST ###
y_pred_prob = BEST_MODEL.predict_proba(X_test)[:, 1]
y_pred      = BEST_MODEL.predict(X_test)

test_auc = roc_auc_score(y_test, y_pred_prob)
test_f1  = f1_score(y_test, y_pred, average='macro')

print(f'TEST AUC {test_auc:.4f}\tTEST F1 {test_f1:.4f}')

## DNN

In [33]:
from keras import losses, metrics
from keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization, Dropout, Activation, Input
from keras.layers import GlobalAveragePooling1D, Flatten, SeparableConv1D, concatenate, Add
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve
import tensorflow as tf
import itertools as it
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats, interp
import os, sys, pickle, shutil
import pandas as pd
import numpy as np
import random, datetime, time
from sklearn.model_selection import KFold


# tensorflow 사용 시 seed 고정
def seed_everything(seed: int = 98):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
SEED = 98
os.environ["CUDA_VISIBLE_DEVICES"]= "3"

### Input

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61384 entries, 296909 to 527936
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   opid              61384 non-null  int64  
 1   sex               61384 non-null  bool   
 2   age               61384 non-null  int64  
 3   weight            61384 non-null  float64
 4   height            61384 non-null  float64
 5   airway_tube_size  61384 non-null  float64
 6   airway_tube_type  61384 non-null  object 
 7   airway_lma        32 non-null     float64
 8   old_tube_size     61384 non-null  float64
dtypes: bool(1), float64(5), int64(2), object(1)
memory usage: 4.3+ MB


In [113]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils import shuffle
import pandas as pd
import xgboost as xgb
import numpy as np
import random, os, datetime, pickle
import scipy
import matplotlib.pyplot as plt
import numpy as np
import csv

df = pd.read_csv('tube.csv')
df.loc[df['weight'] <= 0, 'weight'] = None
df.loc[df['height'] <= 0, 'height'] = None
df['age'] = df['age'].astype(int)
df = df.loc[df['age'] <= 10]
df = df.loc[df['airway_tube_type'] == 'plain']
# [nan 'plain' 'RAE(oral)' 'reinforced' 'LMA' 'T-tube' 'CobraPLA', 'double lumen tube' 'RAE(nasal)' 'laser' 'combitube' 'univent']
df = df.loc[~df['airway_tube_size'].isnull()]
df.dropna(subset=['age', 'sex', 'weight', 'height'], inplace=True)
df['sex'] = (df['sex'] == 'M')


INPUT_VARS = ['age','sex','weight','height']
TARGET_VAR = 'airway_tube_size'

OLD_VAR = 'old_tube_size'
df[OLD_VAR] = np.round((df['age'] / 4 + 4) * 2) / 2

random.seed(0)
df = shuffle(df)
df.reset_index(drop=True, inplace=True)
y = df[[TARGET_VAR]].values.flatten().astype(float)
y_old = df[[OLD_VAR]].values.flatten().astype(float)


# training set의 뒤쪽 20%를 test set 으로 사용
nsamp = len(y)
ntest = int(nsamp * 0.2)
ntrain = nsamp - ntest
sc = StandardScaler()
x_train = sc.fit_transform(df.loc[:ntrain-1,INPUT_VARS])
x_test = sc.transform(df.loc[ntrain:,INPUT_VARS])
#x = df.loc[:, INPUT_VARS].values.astype(float)

y_test = y[-ntest:]
y_test_old = y_old[-ntest:]
y_train = y[:ntrain]

In [111]:
x_train.shape, x_test.shape

((49108, 4), (61384, 4))

In [91]:
# save model
#odir_f = f'acc1-{acc1:.3f}_acc3-{acc3:.3f}_XGBR_{nfold}fold'
odir_f = f'DNN_class'
odir = f'result/{odir_f}'
if not os.path.exists(odir):
    os.mkdir(odir)

### regression

In [92]:
dense_node = 256
dropout = 0.3
batch_size = 64
learning_rate = 0.001

# model
inp = Input(shape=(x_train.shape[1],))
out = inp

out = Dense(dense_node, activation='tanh')(out)
out = Dropout(dropout)(out)
out = Dense(1, activation=None)(out)

model = Model(inputs=[inp], outputs=[out])
#model.save_weights(f'{odir}/initial_weights.hdf5')

In [93]:
weightcache = f"{odir}/weights.hdf5"
model.compile(loss='mse', optimizer=Adam(lr=learning_rate), metrics=["mean_absolute_error"])
hist = model.fit(x_train, y_train, validation_split=0.2, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                        callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                    EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

model.load_weights(weightcache)
y_pred = model.predict(x_test).flatten()

/home/painstudy/anaconda3/envs/keras/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
605/614 [============================>.] - ETA: 0s - loss: 2.4490 - mean_absolute_error: 0.9652

ValueError: Data cardinality is ambiguous:
  x sizes: 9822
  y sizes: 9821
Make sure all arrays contain the same number of samples.

In [81]:
y_pred = model.predict(x_train).flatten()
y_pred = np.round(y_pred * 2 ) / 2

print(f'explained_variance_score: {explained_variance_score(y_train, y_pred):.3f}')
print(f'mean_squared_errors: {mean_squared_error(y_train, y_pred):.3f}')
print(f'r2_score: {r2_score(y_train, y_pred):.3f}')
print(f'acc: {np.mean(y_pred==y_train):.3f}')
print(f'acc(+-0.5mm): {np.mean((y_pred >= y_train-0.5) & (y_pred <= y_train+0.5)):.3f}')

explained_variance_score: 0.763
mean_squared_errors: 0.239
r2_score: 0.763
acc: 0.413
acc(+-0.5mm): 0.902


In [80]:
y_pred = model.predict(x_test).flatten()
y_pred = np.round(y_pred * 2) / 2

print(f'explained_variance_score: {explained_variance_score(y_test, y_pred):.3f}')
print(f'mean_squared_errors: {mean_squared_error(y_test, y_pred):.3f}')
print(f'r2_score: {r2_score(y_test, y_pred):.3f}')
print(f'acc: {np.mean(y_pred==y_test):.3f}')
print(f'acc(+-0.5mm): {np.mean((y_pred >= y_test-0.5) & (y_pred <= y_test+0.5)):.3f}')

explained_variance_score: 0.764
mean_squared_errors: 0.242
r2_score: 0.764
acc: 0.407
acc(+-0.5mm): 0.899


### Classification

In [88]:
# encoding : 2 -> 0, 2.5 -> 1, ... 8.5 -> 14 로 tube size x 2 - 4
df['label'] = df[TARGET_VAR].apply(lambda x: x*2-4)

for i in range(14):
    df[f'{i}'] = df['label'].apply(lambda x: 1 if x==i else 0)
    
y_label = df[[f'{i}' for i in range(14)]].values
yl_test = y_label[-ntest:]
yl_train = y_label[:ntrain]

In [102]:
yl_train.shape, y_train.shape, x_train.shape

((49108, 14), (49108,), (49108, 4))

In [97]:
dense_node = 256
dropout = 0.3
batch_size = 64
learning_rate = 0.001

# model
inp = Input(shape=(x_train.shape[1],))
out = inp

out = Dense(dense_node, activation='tanh')(out)
out = Dropout(dropout)(out)
out = Dense(14, activation='softmax')(out)

model = Model(inputs=[inp], outputs=[out])
#model.save_weights(f'{odir}/initial_weights.hdf5')

In [103]:
fold = 0
weightcache = f"{odir}/weights_{fold}.hdf5"
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate), metrics=[tf.keras.metrics.AUC()])
hist = model.fit(x_train, yl_train, validation_split=0.2, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                        callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                    EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

model.load_weights(weightcache)
y_pred = model.predict(x_test)

/home/painstudy/anaconda3/envs/keras/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
602/614 [============================>.] - ETA: 0s - loss: 2.1932 - auc_3: 0.7691
Epoch 00001: val_loss improved from inf to 2.06036, saving model to result/DNN_class/weights_0.hdf5
614/614 [==============================] - 3s 5ms/step - loss: 2.1908 - auc_3: 0.7691 - val_loss: 2.0604 - val_auc_3: 0.7792
Epoch 2/100
613/614 [============================>.] - ETA: 0s - loss: 2.0593 - auc_3: 0.7812
Epoch 00002: val_loss did not improve from 2.06036
614/614 [==============================] - 3s 4ms/step - loss: 2.0593 - auc_3: 0.7812 - val_loss: 2.0617 - val_auc_3: 0.7796
Epoch 3/100
602/614 [============================>.] - ETA: 0s - loss: 2.0575 - auc_3: 0.7819
Epoch 00003: val_loss improved from 2.06036 to 2.06023, saving model to result/DNN_class/weights_0.hdf5
614/614 [==============================] - 3s 4ms/step - loss: 2.0572 - auc_3: 0.7821 - val_loss: 2.0602 - val_auc_3: 0.7797
Epoch 4/100
610/614 [============================>.] - ETA: 0s - loss: 2.0559 - auc_3: 0

NameError: name 'X_test' is not defined

In [114]:
y_pred = model.predict(x_test).argmax(1)
yl_test_ = yl_test.argmax(1)
print(f'acc: {np.mean(y_pred==yl_test_):.3f}')
print(f'acc(+-0.5mm): {np.mean((y_pred >= yl_test_-1) & (y_pred <= yl_test_+1)):.3f}')

acc: 0.178
acc(+-0.5mm): 0.427


# Appendix

## 소아 성장곡선에서 백분위 추출 (성장도표해설집, 질병관리청)

In [ ]:
# GC_Ht_M (남아 신장)
df = {"Month":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,],
"L":[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0915,-1.0011,-0.9126,-0.8262,-0.7418,-0.6592,-0.5827,-0.5077,-0.4343,-0.3625,-0.2921,-0.2232,-0.1597,-0.0975,-0.0364,0.0234,0.0822,0.1398,0.1897,0.2387,0.2868,0.3341,0.3805,0.4261,0.4242,0.4223,0.4205,0.4187,0.4169,0.4151,0.3787,0.3429,0.3074,0.2724,0.2378,0.2036,0.1783,0.1533,0.1287,0.1044,0.0805,0.0569,0.0563,0.0557,0.0552,0.0546,0.0541,0.0536,0.0492,0.0448,0.0405,0.0363,0.0321,0.027999999999999997,0.0397,0.0514,0.0628,0.0741,0.0853,0.0964,0.1205,0.1443,0.1679,0.1912,0.2142,0.237,0.2339,0.2308,0.2278,0.2248,0.2218,0.2188,0.1885,0.1585,0.1288,0.0994,0.0704,0.0416,0.021,0.0008,-0.0191,-0.0386,-0.0579,-0.0768,-0.0752,-0.0736,-0.07200000000000001,-0.0705,-0.0689,-0.0674,-0.0489,-0.0308,-0.0131,0.0041,0.0209,0.0373,0.0886,0.1383,0.1864,0.2331,0.2784,0.3223,0.4064,0.4882,0.5676,0.6448,0.7199,0.7929,0.8928,0.991,1.0877,1.1829,1.2764,1.3684,1.4580000000000002,1.5475,1.6369999999999998,1.7263,1.8155,1.9045,2.0111,2.1194,2.2294,2.3409999999999997,2.4544,2.5694,2.6754,2.7844,2.8966,3.0121,3.1309,3.2531,3.3119,3.3725,3.4351,3.4998,3.5666,3.6357,3.6315,3.6262,3.6198,3.6121,3.6031,3.5927,3.5208,3.4457,3.3672,3.2853,3.1997,3.1104,2.9809,2.8483,2.7127,2.574,2.4321,2.2872,2.154,2.0196,1.8841,1.7474,1.6096,1.4707,1.3966,1.3226,1.2488,1.1751,1.1016,1.0282,0.9751,0.9218,0.8685,0.815,0.7614,0.7077,0.6595,0.611,0.5623,0.5134,0.4642,0.4148,0.3638,0.3126,0.2611,0.2094,0.1573,0.105,0.0449,-0.0155,-0.0764,-0.1377,-0.1994,-0.2615,],
"M":[49.8842,54.7244,58.4249,61.4292,63.886,65.9026,67.6236,69.1645,70.5994,71.9687,73.2812,74.5388,75.7488,76.9186,78.0497,79.1458,80.2113,81.2487,82.2587,83.2418,84.1996,85.1348,86.0477,86.941,87.1161,87.97200000000001,88.8065,89.6197,90.412,91.1828,91.9327,92.6631,93.3753,94.0711,94.7532,95.4236,96.4961,97.0464,97.5965,98.1463,98.6959,99.2452,99.79299999999999,100.3406,100.8881,101.4353,101.9824,102.5294,103.0749,103.6204,104.1657,104.7109,105.256,105.8009,106.344,106.8869,107.4298,107.9726,108.5153,109.0579,109.5896,110.1212,110.6529,111.1846,111.7162,112.2479,112.7735,113.29899999999999,113.8245,114.3501,114.8756,115.40100000000001,115.9183,116.4356,116.9528,117.4701,117.9874,118.5047,119.0136,119.5225,120.0314,120.5404,121.0493,121.5582,122.0537,122.5492,123.0447,123.5402,124.0357,124.5313,125.0114,125.4914,125.9715,126.4515,126.9316,127.4116,127.8793,128.3469,128.8145,129.2821,129.7497,130.2172,130.6754,131.1336,131.5919,132.0501,132.5084,132.9666,133.4136,133.8606,134.3076,134.7545,135.2014,135.6482,136.1026,136.5569,137.0113,137.4656,137.9199,138.3742,138.8473,139.3205,139.7936,140.2667,140.7398,141.213,141.7059,142.1987,142.6916,143.1844,143.6773,144.1701,144.701,145.2319,145.7627,146.2935,146.8242,147.3549,147.9321,148.5094,149.0866,149.6638,150.2409,150.81799999999998,151.4223,152.0268,152.6312,153.2358,153.8404,154.445,155.0459,155.6468,156.2477,156.8487,157.4497,158.0507,158.6245,159.1984,159.7724,160.3465,160.9207,161.4949,162.0038,162.5128,163.0218,163.5308,164.0398,164.5488,164.965,165.3813,165.7975,166.2138,166.6301,167.0464,167.3647,167.6831,168.0013,168.3195,168.6377,168.9558,169.1812,169.4066,169.6319,169.8571,170.0824,170.3076,170.4684,170.6294,170.7906,170.952,171.1137,171.2756,171.3949,171.5145,171.6344,171.7546,171.8752,171.99599999999998,172.0897,172.1836,172.2775,172.3716,172.4658,172.56,172.6404,172.7207,172.8009,172.8812,172.9614,173.0416,173.1222,173.2027,173.2832,173.3636,173.44400000000002,173.5244,173.6037,173.683,173.7622,173.8413,173.9204,173.9995,174.0764,174.1532,174.23,174.3067,174.3833,174.4598,],
"S":[0.038,0.0356,0.0342,0.0333,0.0326,0.032,0.0317,0.0314,0.0312,0.0312,0.0312,0.0313,0.0314,0.0315,0.0317,0.032,0.0322,0.0325,0.0328,0.0331,0.0334,0.0338,0.0341,0.0345,0.0351,0.0354,0.0358,0.0361,0.0364,0.0367,0.037000000000000005,0.0373,0.0376,0.0379,0.0381,0.0384,0.0403,0.0403,0.0402,0.0402,0.0402,0.0402,0.0401,0.0401,0.0401,0.0401,0.0401,0.04,0.04,0.04,0.04,0.0399,0.0399,0.0399,0.0399,0.0399,0.0398,0.0398,0.0398,0.0398,0.0398,0.0399,0.0399,0.0399,0.04,0.04,0.04,0.0401,0.0401,0.0402,0.0402,0.0403,0.0403,0.0404,0.0404,0.0405,0.0405,0.0406,0.0406,0.0406,0.0406,0.0406,0.0406,0.0406,0.0406,0.0406,0.0406,0.0406,0.0406,0.0406,0.0406,0.0405,0.0405,0.0405,0.0405,0.0405,0.0405,0.0404,0.0404,0.0404,0.0404,0.0404,0.0404,0.0404,0.0404,0.0404,0.0404,0.0404,0.0405,0.0406,0.0406,0.0407,0.0408,0.0409,0.040999999999999995,0.0411,0.0412,0.0414,0.0415,0.0416,0.0417,0.0419,0.042,0.0422,0.0423,0.0424,0.0426,0.0428,0.043,0.0432,0.0434,0.0436,0.0438,0.0441,0.0443,0.0445,0.0448,0.045,0.0453,0.0455,0.0457,0.0459,0.0462,0.0464,0.0465,0.0466,0.0467,0.0468,0.0469,0.047,0.0469,0.0469,0.0468,0.0467,0.0466,0.0465,0.0463,0.046,0.0457,0.0454,0.0452,0.0449,0.0445,0.044000000000000004,0.0436,0.0431,0.0427,0.0423,0.0417,0.0412,0.0407,0.0402,0.0397,0.0392,0.0388,0.0384,0.0379,0.0375,0.037000000000000005,0.0366,0.0363,0.036000000000000004,0.0357,0.0353,0.035,0.0347,0.0345,0.0342,0.034,0.0338,0.0335,0.0333,0.0332,0.033,0.0328,0.0327,0.0325,0.0324,0.0323,0.0323,0.0322,0.0322,0.0321,0.0321,0.0321,0.0321,0.0321,0.0321,0.0321,0.0321,0.0321,0.0321,0.0321,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.0319,0.0319,0.0319,],
"1p":[45.5,50.2,53.8,56.7,59.0,61.0,62.6,64.1,65.5,66.8,68.0,69.1,70.2,71.3,72.3,73.3,74.2,75.1,76.0,76.8,77.7,78.4,79.2,80.0,80.0,80.7,81.4,82.1,82.8,83.4,84.0,84.6,85.2,85.8,86.4,86.9,88.3,88.7,89.2,89.7,90.2,90.6,91.1,91.6,92.1,92.5,93.0,93.5,94.0,94.5,94.9,95.4,95.9,96.4,96.8,97.3,97.8,98.3,98.8,99.2,99.7,100.2,100.7,101.1,101.6,102.1,102.6,103.1,103.5,104.0,104.5,105.0,105.5,105.9,106.4,106.9,107.3,107.8,108.3,108.7,109.2,109.6,110.1,110.6,111.0,111.5,111.9,112.4,112.8,113.3,113.7,114.2,114.6,115.0,115.5,115.9,116.3,116.7,117.2,117.6,118.0,118.4,118.8,119.3,119.7,120.1,120.5,120.9,121.3,121.7,122.1,122.5,122.9,123.3,123.7,124.1,124.5,124.9,125.3,125.7,126.0,126.4,126.8,127.2,127.6,128.0,128.4,128.7,129.1,129.5,129.9,130.2,130.6,131.0,131.4,131.7,132.1,132.5,132.8,133.2,133.6,134.0,134.4,134.7,135.1,135.6,136.0,136.4,136.8,137.2,137.7,138.1,138.6,139.1,139.5,140.0,140.5,141.0,141.5,142.0,142.5,143.0,143.5,144.1,144.6,145.1,145.6,146.2,146.7,147.3,147.8,148.4,149.0,149.5,150.0,150.5,151.1,151.6,152.1,152.6,153.1,153.5,153.9,154.3,154.8,155.2,155.5,155.9,156.2,156.6,156.9,157.3,157.5,157.8,158.0,158.3,158.5,158.8,158.9,159.1,159.2,159.4,159.5,159.7,159.8,159.9,160.0,160.1,160.2,160.3,160.4,160.5,160.6,160.7,160.8,160.9,161.0,161.1,161.2,161.3,161.4,161.5,161.6,161.7,161.8,161.9,162.0,162.1,],
"3p":[46.3,51.1,54.7,57.6,60.0,61.9,63.6,65.1,66.5,67.7,69.0,70.2,71.3,72.4,73.4,74.4,75.4,76.3,77.2,78.1,78.9,79.7,80.5,81.3,81.4,82.1,82.8,83.5,84.2,84.9,85.5,86.2,86.8,87.4,88.0,88.5,89.7,90.2,90.7,91.2,91.7,92.2,92.7,93.2,93.7,94.2,94.7,95.2,95.6,96.1,96.6,97.1,97.6,98.1,98.6,99.1,99.6,100.1,100.6,101.1,101.6,102.0,102.5,103.0,103.5,104.0,104.5,105.0,105.5,105.9,106.4,106.9,107.4,107.9,108.3,108.8,109.3,109.8,110.3,110.7,111.2,111.7,112.1,112.6,113.1,113.5,114.0,114.4,114.9,115.4,115.8,116.3,116.7,117.1,117.6,118.0,118.5,118.9,119.3,119.8,120.2,120.6,121.0,121.5,121.9,122.3,122.7,123.2,123.6,124.0,124.4,124.8,125.2,125.6,126.0,126.4,126.8,127.2,127.6,128.0,128.4,128.8,129.2,129.6,130.0,130.4,130.8,131.2,131.6,132.0,132.4,132.8,133.2,133.6,134.0,134.4,134.8,135.2,135.7,136.1,136.5,136.9,137.4,137.8,138.2,138.7,139.2,139.6,140.1,140.6,141.1,141.6,142.1,142.6,143.1,143.6,144.2,144.7,145.2,145.8,146.3,146.8,147.4,147.9,148.5,149.0,149.5,150.1,150.6,151.2,151.7,152.2,152.8,153.3,153.8,154.2,154.7,155.2,155.6,156.1,156.5,156.8,157.2,157.6,158.0,158.3,158.6,158.9,159.2,159.5,159.8,160.1,160.3,160.5,160.7,160.9,161.2,161.4,161.5,161.6,161.8,161.9,162.0,162.1,162.2,162.3,162.4,162.5,162.6,162.7,162.8,162.9,163.0,163.1,163.2,163.3,163.3,163.4,163.5,163.6,163.7,163.8,163.9,164.0,164.1,164.2,164.3,164.4,],
"5p":[46.8,51.5,55.1,58.1,60.5,62.4,64.1,65.6,67.0,68.3,69.5,70.7,71.8,72.9,74.0,75.0,76.0,76.9,77.8,78.7,79.6,80.4,81.2,82.0,82.1,82.8,83.6,84.3,85.0,85.7,86.3,87.0,87.6,88.2,88.8,89.4,90.5,91.0,91.5,92.0,92.5,93.0,93.5,94.0,94.5,95.0,95.5,96.0,96.5,97.0,97.5,98.0,98.6,99.1,99.6,100.1,100.6,101.1,101.6,102.1,102.5,103.0,103.5,104.0,104.5,105.0,105.5,106.0,106.5,107.0,107.5,108.0,108.4,108.9,109.4,109.9,110.4,110.8,111.3,111.8,112.3,112.7,113.2,113.7,114.2,114.6,115.1,115.6,116.0,116.5,116.9,117.4,117.8,118.3,118.7,119.2,119.6,120.0,120.5,120.9,121.4,121.8,122.2,122.6,123.1,123.5,123.9,124.4,124.8,125.2,125.6,126.0,126.4,126.8,127.2,127.6,128.0,128.4,128.8,129.2,129.7,130.1,130.5,130.9,131.3,131.7,132.1,132.5,132.9,133.4,133.8,134.2,134.6,135.0,135.5,135.9,136.3,136.7,137.2,137.6,138.1,138.5,139.0,139.4,139.9,140.4,140.9,141.3,141.8,142.3,142.9,143.4,143.9,144.4,145.0,145.5,146.1,146.6,147.1,147.7,148.2,148.8,149.3,149.9,150.4,151.0,151.5,152.1,152.6,153.1,153.6,154.2,154.7,155.2,155.6,156.1,156.5,157.0,157.4,157.9,158.2,158.6,158.9,159.2,159.6,159.9,160.2,160.5,160.7,161.0,161.3,161.5,161.7,161.9,162.1,162.3,162.5,162.7,162.8,163.0,163.1,163.2,163.3,163.4,163.5,163.6,163.7,163.8,163.9,164.0,164.1,164.2,164.3,164.3,164.4,164.5,164.6,164.7,164.8,164.9,165.0,165.1,165.1,165.2,165.3,165.4,165.5,165.6,],
"10p":[47.5,52.2,55.9,58.8,61.2,63.2,64.9,66.4,67.8,69.1,70.4,71.6,72.7,73.8,74.9,75.9,76.9,77.9,78.8,79.7,80.6,81.5,82.3,83.1,83.2,84.0,84.7,85.5,86.2,86.9,87.6,88.2,88.9,89.5,90.1,90.7,91.8,92.3,92.8,93.3,93.8,94.3,94.9,95.4,95.9,96.4,96.9,97.4,97.9,98.5,99.0,99.5,100.0,100.5,101.0,101.5,102.0,102.6,103.1,103.6,104.1,104.6,105.1,105.6,106.1,106.6,107.1,107.6,108.1,108.6,109.1,109.6,110.1,110.5,111.0,111.5,112.0,112.5,113.0,113.5,113.9,114.4,114.9,115.4,115.9,116.3,116.8,117.3,117.7,118.2,118.7,119.1,119.6,120.0,120.5,121.0,121.4,121.8,122.3,122.7,123.2,123.6,124.0,124.5,124.9,125.4,125.8,126.2,126.6,127.1,127.5,127.9,128.3,128.7,129.1,129.5,130.0,130.4,130.8,131.2,131.6,132.1,132.5,132.9,133.3,133.7,134.2,134.6,135.0,135.5,135.9,136.3,136.8,137.2,137.7,138.1,138.6,139.0,139.5,140.0,140.5,140.9,141.4,141.9,142.4,143.0,143.5,144.0,144.5,145.0,145.6,146.1,146.7,147.2,147.8,148.4,148.9,149.5,150.0,150.6,151.2,151.7,152.3,152.8,153.4,153.9,154.5,155.0,155.5,156.0,156.5,157.0,157.5,158.0,158.4,158.8,159.3,159.7,160.1,160.5,160.8,161.1,161.4,161.7,162.0,162.3,162.6,162.8,163.0,163.3,163.5,163.8,163.9,164.1,164.3,164.4,164.6,164.8,164.9,165.0,165.1,165.2,165.3,165.5,165.5,165.6,165.7,165.8,165.9,166.0,166.1,166.1,166.2,166.3,166.4,166.5,166.6,166.7,166.7,166.8,166.9,167.0,167.1,167.2,167.3,167.3,167.4,167.5,],
"15p":[47.9,52.7,56.4,59.3,61.7,63.7,65.4,66.9,68.3,69.6,70.9,72.1,73.3,74.4,75.5,76.5,77.5,78.5,79.5,80.4,81.3,82.2,83.0,83.8,83.9,84.7,85.5,86.3,87.0,87.7,88.4,89.1,89.7,90.4,91.0,91.6,92.6,93.2,93.7,94.2,94.7,95.3,95.8,96.3,96.8,97.3,97.9,98.4,98.9,99.4,99.9,100.5,101.0,101.5,102.0,102.5,103.1,103.6,104.1,104.6,105.1,105.6,106.1,106.6,107.1,107.7,108.2,108.7,109.2,109.7,110.2,110.7,111.2,111.6,112.1,112.6,113.1,113.6,114.1,114.6,115.1,115.6,116.1,116.5,117.0,117.5,118.0,118.4,118.9,119.4,119.9,120.3,120.8,121.2,121.7,122.2,122.6,123.1,123.5,124.0,124.4,124.9,125.3,125.7,126.2,126.6,127.1,127.5,127.9,128.3,128.8,129.2,129.6,130.0,130.4,130.9,131.3,131.7,132.1,132.5,133.0,133.4,133.8,134.3,134.7,135.1,135.6,136.0,136.5,136.9,137.4,137.8,138.3,138.7,139.2,139.7,140.1,140.6,141.1,141.6,142.1,142.6,143.1,143.6,144.1,144.7,145.2,145.8,146.3,146.8,147.4,148.0,148.6,149.1,149.7,150.3,150.8,151.4,152.0,152.5,153.1,153.7,154.2,154.8,155.3,155.9,156.4,157.0,157.4,157.9,158.4,158.9,159.4,159.9,160.2,160.6,161.0,161.4,161.8,162.2,162.5,162.8,163.1,163.4,163.6,163.9,164.1,164.4,164.6,164.8,165.0,165.2,165.4,165.5,165.7,165.9,166.0,166.2,166.3,166.4,166.5,166.6,166.7,166.8,166.9,167.0,167.1,167.1,167.2,167.3,167.4,167.5,167.6,167.7,167.7,167.8,167.9,168.0,168.1,168.2,168.2,168.3,168.4,168.5,168.6,168.6,168.7,168.8,],
"25p":[48.6,53.4,57.1,60.1,62.5,64.5,66.2,67.7,69.1,70.5,71.7,73.0,74.1,75.3,76.4,77.4,78.5,79.5,80.4,81.4,82.3,83.2,84.1,84.9,85.1,85.9,86.7,87.4,88.2,88.9,89.6,90.3,91.0,91.7,92.3,93.0,93.9,94.5,95.0,95.5,96.1,96.6,97.1,97.7,98.2,98.7,99.3,99.8,100.3,100.9,101.4,101.9,102.5,103.0,103.5,104.0,104.6,105.1,105.6,106.2,106.7,107.2,107.7,108.2,108.7,109.2,109.8,110.3,110.8,111.3,111.8,112.3,112.8,113.3,113.8,114.3,114.8,115.3,115.8,116.3,116.8,117.3,117.8,118.3,118.8,119.2,119.7,120.2,120.7,121.2,121.6,122.1,122.6,123.0,123.5,124.0,124.4,124.9,125.3,125.8,126.3,126.7,127.2,127.6,128.0,128.5,128.9,129.4,129.8,130.2,130.7,131.1,131.5,132.0,132.4,132.8,133.3,133.7,134.1,134.6,135.0,135.4,135.9,136.3,136.8,137.2,137.7,138.2,138.6,139.1,139.5,140.0,140.5,141.0,141.5,141.9,142.4,142.9,143.5,144.0,144.5,145.1,145.6,146.1,146.7,147.2,147.8,148.4,149.0,149.5,150.1,150.7,151.3,151.9,152.4,153.0,153.6,154.2,154.7,155.3,155.9,156.5,157.0,157.6,158.1,158.6,159.2,159.7,160.2,160.6,161.1,161.5,162.0,162.5,162.8,163.2,163.6,163.9,164.3,164.6,164.9,165.2,165.4,165.7,166.0,166.2,166.4,166.6,166.8,167.0,167.2,167.4,167.5,167.7,167.8,167.9,168.1,168.2,168.3,168.4,168.5,168.6,168.7,168.8,168.9,169.0,169.1,169.1,169.2,169.3,169.4,169.5,169.6,169.6,169.7,169.8,169.9,170.0,170.0,170.1,170.2,170.3,170.4,170.4,170.5,170.6,170.7,170.8,],
"50p":[49.9,54.7,58.4,61.4,63.9,65.9,67.6,69.2,70.6,72.0,73.3,74.5,75.7,76.9,78.0,79.1,80.2,81.2,82.3,83.2,84.2,85.1,86.0,86.9,87.1,88.0,88.8,89.6,90.4,91.2,91.9,92.7,93.4,94.1,94.8,95.4,96.5,97.0,97.6,98.1,98.7,99.2,99.8,100.3,100.9,101.4,102.0,102.5,103.1,103.6,104.2,104.7,105.3,105.8,106.3,106.9,107.4,108.0,108.5,109.1,109.6,110.1,110.7,111.2,111.7,112.2,112.8,113.3,113.8,114.4,114.9,115.4,115.9,116.4,117.0,117.5,118.0,118.5,119.0,119.5,120.0,120.5,121.0,121.6,122.1,122.5,123.0,123.5,124.0,124.5,125.0,125.5,126.0,126.5,126.9,127.4,127.9,128.3,128.8,129.3,129.7,130.2,130.7,131.1,131.6,132.1,132.5,133.0,133.4,133.9,134.3,134.8,135.2,135.6,136.1,136.6,137.0,137.5,137.9,138.4,138.8,139.3,139.8,140.3,140.7,141.2,141.7,142.2,142.7,143.2,143.7,144.2,144.7,145.2,145.8,146.3,146.8,147.4,147.9,148.5,149.1,149.7,150.2,150.8,151.4,152.0,152.6,153.2,153.8,154.4,155.0,155.6,156.2,156.8,157.4,158.1,158.6,159.2,159.8,160.3,160.9,161.5,162.0,162.5,163.0,163.5,164.0,164.5,165.0,165.4,165.8,166.2,166.6,167.0,167.4,167.7,168.0,168.3,168.6,169.0,169.2,169.4,169.6,169.9,170.1,170.3,170.5,170.6,170.8,171.0,171.1,171.3,171.4,171.5,171.6,171.8,171.9,172.0,172.1,172.2,172.3,172.4,172.5,172.6,172.6,172.7,172.8,172.9,173.0,173.0,173.1,173.2,173.3,173.4,173.4,173.5,173.6,173.7,173.8,173.8,173.9,174.0,174.1,174.2,174.2,174.3,174.4,174.5,],
"75p":[51.2,56.0,59.8,62.8,65.3,67.3,69.1,70.6,72.1,73.5,74.8,76.1,77.4,78.6,79.7,80.9,82.0,83.0,84.1,85.1,86.1,87.1,88.0,89.0,89.2,90.1,90.9,91.8,92.6,93.4,94.2,95.0,95.7,96.5,97.2,97.9,99.2,99.8,100.3,100.9,101.4,102.0,102.6,103.1,103.7,104.2,104.8,105.3,105.9,106.5,107.0,107.6,108.1,108.7,109.2,109.8,110.3,110.9,111.5,112.0,112.6,113.1,113.7,114.2,114.8,115.3,115.8,116.4,116.9,117.5,118.0,118.6,119.1,119.6,120.2,120.7,121.3,121.8,122.3,122.8,123.4,123.9,124.4,124.9,125.4,126.0,126.5,127.0,127.5,128.0,128.5,129.0,129.5,130.0,130.4,130.9,131.4,131.9,132.4,132.8,133.3,133.8,134.3,134.7,135.2,135.7,136.2,136.6,137.1,137.6,138.0,138.5,139.0,139.4,139.9,140.4,140.9,141.4,141.8,142.3,142.8,143.3,143.8,144.3,144.8,145.3,145.8,146.4,146.9,147.4,147.9,148.5,149.0,149.6,150.2,150.7,151.3,151.9,152.5,153.1,153.7,154.3,154.9,155.6,156.2,156.8,157.4,158.1,158.7,159.3,159.9,160.5,161.1,161.7,162.3,162.9,163.5,164.1,164.6,165.2,165.7,166.3,166.7,167.2,167.7,168.2,168.6,169.1,169.5,169.8,170.2,170.6,171.0,171.3,171.6,171.9,172.2,172.4,172.7,173.0,173.2,173.4,173.6,173.8,174.0,174.2,174.3,174.5,174.6,174.8,174.9,175.1,175.2,175.3,175.4,175.5,175.6,175.7,175.8,175.9,176.0,176.1,176.2,176.3,176.4,176.5,176.5,176.6,176.7,176.8,176.9,177.0,177.0,177.1,177.2,177.3,177.4,177.5,177.5,177.6,177.7,177.8,177.9,177.9,178.0,178.1,178.2,178.3,],
"85p":[51.8,56.7,60.5,63.5,66.0,68.1,69.8,71.4,72.9,74.3,75.6,77.0,78.2,79.4,80.6,81.8,82.9,84.0,85.1,86.1,87.1,88.1,89.1,90.0,90.3,91.2,92.1,93.0,93.8,94.7,95.5,96.2,97.0,97.8,98.5,99.2,100.7,101.3,101.8,102.4,103.0,103.5,104.1,104.6,105.2,105.8,106.3,106.9,107.5,108.0,108.6,109.1,109.7,110.3,110.8,111.4,111.9,112.5,113.1,113.6,114.2,114.7,115.3,115.8,116.4,117.0,117.5,118.1,118.6,119.2,119.7,120.3,120.8,121.4,121.9,122.5,123.0,123.6,124.1,124.7,125.2,125.7,126.2,126.8,127.3,127.8,128.3,128.8,129.4,129.9,130.4,130.9,131.4,131.9,132.4,132.9,133.3,133.8,134.3,134.8,135.3,135.8,136.2,136.7,137.2,137.7,138.1,138.6,139.1,139.6,140.1,140.6,141.0,141.5,142.0,142.5,143.0,143.5,144.0,144.5,145.0,145.5,146.0,146.5,147.1,147.6,148.1,148.7,149.2,149.7,150.3,150.8,151.4,152.0,152.6,153.2,153.8,154.3,155.0,155.6,156.2,156.8,157.5,158.1,158.7,159.4,160.0,160.6,161.3,161.9,162.5,163.1,163.7,164.3,164.9,165.5,166.1,166.6,167.1,167.7,168.2,168.8,169.2,169.6,170.1,170.5,171.0,171.4,171.8,172.1,172.5,172.8,173.2,173.5,173.8,174.0,174.3,174.6,174.8,175.1,175.3,175.5,175.6,175.8,176.0,176.2,176.4,176.5,176.6,176.8,176.9,177.1,177.2,177.3,177.4,177.5,177.6,177.7,177.8,177.9,178.0,178.1,178.2,178.3,178.4,178.5,178.6,178.6,178.7,178.8,178.9,179.0,179.1,179.2,179.3,179.3,179.4,179.5,179.6,179.7,179.8,179.9,179.9,180.0,180.1,180.2,180.3,180.4,],
"90p":[52.3,57.2,61.0,64.0,66.6,68.6,70.4,71.9,73.4,74.8,76.2,77.5,78.8,80.0,81.2,82.4,83.5,84.6,85.7,86.8,87.8,88.8,89.8,90.8,91.0,92.0,92.9,93.8,94.6,95.5,96.3,97.1,97.9,98.6,99.4,100.1,101.8,102.3,102.9,103.5,104.0,104.6,105.1,105.7,106.3,106.8,107.4,108.0,108.5,109.1,109.6,110.2,110.8,111.3,111.9,112.5,113.0,113.6,114.1,114.7,115.3,115.8,116.4,117.0,117.5,118.1,118.7,119.2,119.8,120.3,120.9,121.5,122.0,122.6,123.2,123.7,124.3,124.8,125.4,125.9,126.4,127.0,127.5,128.0,128.6,129.1,129.6,130.1,130.7,131.2,131.7,132.2,132.7,133.2,133.7,134.2,134.7,135.1,135.6,136.1,136.6,137.1,137.6,138.1,138.5,139.0,139.5,140.0,140.5,141.0,141.5,142.0,142.4,142.9,143.4,143.9,144.5,145.0,145.5,146.0,146.5,147.0,147.5,148.1,148.6,149.1,149.7,150.2,150.8,151.3,151.9,152.4,153.0,153.6,154.2,154.8,155.4,156.0,156.7,157.3,157.9,158.6,159.2,159.8,160.5,161.1,161.7,162.4,163.0,163.7,164.2,164.8,165.4,166.0,166.6,167.2,167.8,168.3,168.8,169.3,169.9,170.4,170.8,171.3,171.7,172.1,172.5,173.0,173.3,173.6,174.0,174.3,174.6,174.9,175.2,175.4,175.7,175.9,176.2,176.4,176.6,176.8,177.0,177.2,177.4,177.6,177.7,177.8,178.0,178.1,178.3,178.4,178.5,178.6,178.7,178.8,179.0,179.1,179.2,179.3,179.4,179.5,179.6,179.7,179.7,179.8,179.9,180.0,180.1,180.2,180.3,180.4,180.5,180.6,180.6,180.7,180.8,180.9,181.0,181.1,181.2,181.3,181.3,181.4,181.5,181.6,181.7,181.8,],
"95p":[53.0,57.9,61.7,64.8,67.3,69.4,71.1,72.7,74.2,75.7,77.0,78.4,79.7,80.9,82.1,83.3,84.5,85.6,86.7,87.8,88.8,89.9,90.9,91.9,92.1,93.1,94.0,94.9,95.8,96.7,97.5,98.4,99.2,99.9,100.7,101.4,103.4,103.9,104.5,105.1,105.6,106.2,106.7,107.3,107.9,108.4,109.0,109.6,110.1,110.7,111.3,111.8,112.4,112.9,113.5,114.1,114.6,115.2,115.8,116.3,116.9,117.5,118.1,118.6,119.2,119.8,120.4,120.9,121.5,122.1,122.7,123.3,123.8,124.4,125.0,125.5,126.1,126.7,127.2,127.8,128.3,128.9,129.4,129.9,130.5,131.0,131.5,132.1,132.6,133.1,133.6,134.1,134.6,135.1,135.7,136.2,136.6,137.1,137.6,138.1,138.6,139.1,139.6,140.1,140.6,141.0,141.5,142.0,142.5,143.0,143.6,144.1,144.6,145.1,145.6,146.1,146.6,147.2,147.7,148.2,148.7,149.3,149.8,150.4,150.9,151.4,152.0,152.6,153.2,153.7,154.3,154.9,155.5,156.1,156.7,157.3,157.9,158.5,159.2,159.8,160.5,161.1,161.8,162.4,163.0,163.7,164.3,165.0,165.6,166.2,166.8,167.4,168.0,168.6,169.2,169.8,170.3,170.8,171.3,171.8,172.3,172.8,173.2,173.6,174.0,174.4,174.8,175.2,175.5,175.8,176.1,176.4,176.7,177.0,177.3,177.5,177.7,178.0,178.2,178.4,178.6,178.8,179.0,179.1,179.3,179.5,179.6,179.8,179.9,180.1,180.2,180.3,180.5,180.6,180.7,180.8,180.9,181.0,181.1,181.3,181.4,181.5,181.6,181.7,181.8,181.9,182.0,182.0,182.1,182.2,182.3,182.4,182.5,182.6,182.7,182.8,182.9,183.0,183.1,183.2,183.3,183.4,183.5,183.6,183.7,183.7,183.8,183.9,],
"97p":[53.4,58.4,62.2,65.3,67.8,69.9,71.6,73.2,74.7,76.2,77.6,78.9,80.2,81.5,82.7,83.9,85.1,86.2,87.3,88.4,89.5,90.5,91.6,92.6,92.9,93.8,94.8,95.7,96.6,97.5,98.3,99.2,100.0,100.8,101.5,102.3,104.4,105.0,105.6,106.1,106.7,107.2,107.8,108.4,108.9,109.5,110.1,110.6,111.2,111.7,112.3,112.9,113.4,114.0,114.6,115.1,115.7,116.3,116.8,117.4,118.0,118.6,119.1,119.7,120.3,120.9,121.5,122.1,122.6,123.2,123.8,124.4,125.0,125.6,126.1,126.7,127.3,127.9,128.4,129.0,129.5,130.1,130.6,131.2,131.7,132.3,132.8,133.3,133.9,134.4,134.9,135.4,135.9,136.4,136.9,137.5,137.9,138.4,138.9,139.4,139.9,140.4,140.9,141.4,141.9,142.4,142.9,143.4,143.9,144.4,144.9,145.4,146.0,146.5,147.0,147.5,148.1,148.6,149.1,149.7,150.2,150.8,151.3,151.9,152.4,153.0,153.6,154.1,154.7,155.3,155.9,156.5,157.1,157.7,158.3,159.0,159.6,160.2,160.8,161.5,162.1,162.8,163.4,164.1,164.7,165.4,166.0,166.6,167.3,167.9,168.5,169.1,169.6,170.2,170.8,171.4,171.9,172.4,172.8,173.3,173.8,174.3,174.7,175.1,175.5,175.8,176.2,176.6,176.9,177.2,177.5,177.8,178.0,178.3,178.6,178.8,179.0,179.2,179.5,179.7,179.9,180.0,180.2,180.4,180.6,180.7,180.9,181.0,181.2,181.3,181.5,181.6,181.7,181.8,182.0,182.1,182.2,182.3,182.4,182.5,182.6,182.8,182.9,183.0,183.1,183.2,183.3,183.4,183.5,183.6,183.7,183.8,183.9,184.0,184.1,184.2,184.3,184.4,184.5,184.6,184.7,184.8,184.8,184.9,185.0,185.1,185.2,185.3,],
"99p":[54.3,59.3,63.1,66.2,68.7,70.8,72.6,74.2,75.7,77.2,78.6,80.0,81.3,82.6,83.8,85.0,86.2,87.4,88.5,89.7,90.7,91.8,92.9,93.9,94.2,95.2,96.2,97.1,98.1,99.0,99.9,100.7,101.5,102.4,103.2,103.9,106.5,107.1,107.6,108.2,108.7,109.3,109.8,110.4,111.0,111.5,112.1,112.6,113.2,113.8,114.3,114.9,115.5,116.0,116.6,117.2,117.7,118.3,118.9,119.4,120.0,120.6,121.2,121.8,122.4,123.0,123.6,124.2,124.8,125.4,126.0,126.6,127.2,127.8,128.4,129.0,129.6,130.2,130.8,131.3,131.9,132.4,133.0,133.6,134.1,134.7,135.2,135.8,136.3,136.8,137.4,137.9,138.4,138.9,139.4,139.9,140.4,140.9,141.4,141.9,142.4,142.9,143.4,143.9,144.4,144.9,145.4,145.9,146.5,147.0,147.5,148.1,148.6,149.2,149.7,150.3,150.8,151.4,151.9,152.5,153.1,153.6,154.2,154.8,155.3,155.9,156.5,157.1,157.7,158.3,158.9,159.5,160.2,160.8,161.4,162.1,162.7,163.3,164.0,164.6,165.3,166.0,166.6,167.3,167.9,168.5,169.1,169.8,170.4,171.0,171.6,172.1,172.7,173.3,173.8,174.4,174.9,175.3,175.8,176.2,176.7,177.2,177.5,177.8,178.2,178.5,178.9,179.2,179.5,179.7,180.0,180.2,180.5,180.7,181.0,181.2,181.4,181.6,181.8,182.0,182.2,182.3,182.5,182.7,182.9,183.0,183.2,183.3,183.5,183.6,183.8,183.9,184.1,184.2,184.3,184.5,184.6,184.7,184.8,185.0,185.1,185.2,185.3,185.4,185.5,185.7,185.8,185.9,186.0,186.1,186.2,186.3,186.4,186.5,186.6,186.7,186.8,187.0,187.1,187.2,187.3,187.4,187.5,187.6,187.7,187.8,187.9,188.0,],
"[-3SD]":[44.2,48.9,52.4,55.3,57.6,59.6,61.2,62.7,64.0,65.2,66.4,67.6,68.6,69.6,70.6,71.6,72.5,73.3,74.2,75.0,75.8,76.5,77.2,78.0,78.0,78.6,79.3,79.9,80.5,81.1,81.7,82.3,82.8,83.4,83.9,84.4,86.1,86.6,87.0,87.5,87.9,88.4,88.8,89.3,89.7,90.2,90.6,91.1,91.5,92.0,92.4,92.9,93.3,93.8,94.2,94.7,95.1,95.6,96.0,96.5,96.9,97.4,97.9,98.3,98.8,99.3,99.7,100.2,100.7,101.2,101.6,102.1,102.6,103.0,103.5,104.0,104.4,104.9,105.3,105.8,106.2,106.7,107.1,107.6,108.0,108.5,108.9,109.3,109.8,110.2,110.7,111.1,111.5,111.9,112.3,112.8,113.2,113.6,114.0,114.4,114.8,115.2,115.6,116.0,116.4,116.8,117.2,117.6,118.0,118.4,118.8,119.2,119.6,120.0,120.3,120.7,121.1,121.5,121.8,122.2,122.6,122.9,123.3,123.7,124.0,124.4,124.7,125.1,125.4,125.8,126.1,126.5,126.8,127.1,127.4,127.7,128.0,128.3,128.7,129.0,129.3,129.6,129.9,130.1,130.5,130.8,131.1,131.4,131.7,132.0,132.4,132.8,133.2,133.6,134.0,134.4,134.8,135.2,135.6,136.1,136.5,136.9,137.4,137.8,138.3,138.8,139.3,139.7,140.3,140.9,141.5,142.1,142.7,143.3,143.9,144.5,145.1,145.7,146.3,146.9,147.5,148.0,148.6,149.1,149.6,150.2,150.6,151.1,151.6,152.0,152.5,152.9,153.2,153.6,153.9,154.2,154.6,154.9,155.1,155.2,155.4,155.6,155.8,155.9,156.0,156.2,156.3,156.4,156.5,156.6,156.7,156.9,157.0,157.1,157.2,157.3,157.4,157.5,157.7,157.8,157.9,158.0,158.1,158.2,158.4,158.5,158.6,158.7,],
"[-2SD]":[46.1,50.8,54.4,57.3,59.7,61.7,63.3,64.8,66.2,67.5,68.7,69.9,71.0,72.1,73.1,74.1,75.0,76.0,76.9,77.7,78.6,79.4,80.2,81.0,81.0,81.7,82.5,83.1,83.8,84.5,85.1,85.7,86.4,86.9,87.5,88.1,89.3,89.8,90.3,90.8,91.3,91.8,92.3,92.8,93.2,93.7,94.2,94.7,95.2,95.7,96.2,96.7,97.2,97.6,98.1,98.6,99.1,99.6,100.1,100.6,101.1,101.5,102.0,102.5,103.0,103.5,104.0,104.5,104.9,105.4,105.9,106.4,106.9,107.3,107.8,108.3,108.8,109.2,109.7,110.2,110.7,111.1,111.6,112.1,112.5,113.0,113.4,113.9,114.4,114.8,115.3,115.7,116.1,116.6,117.0,117.5,117.9,118.3,118.7,119.2,119.6,120.0,120.4,120.9,121.3,121.7,122.1,122.6,123.0,123.4,123.8,124.2,124.6,125.0,125.4,125.8,126.2,126.6,127.0,127.4,127.8,128.2,128.6,129.0,129.4,129.8,130.1,130.5,130.9,131.3,131.7,132.1,132.5,132.9,133.3,133.7,134.1,134.5,134.9,135.3,135.7,136.1,136.6,137.0,137.4,137.9,138.3,138.8,139.2,139.7,140.2,140.7,141.2,141.7,142.2,142.7,143.2,143.7,144.2,144.8,145.3,145.8,146.4,146.9,147.4,148.0,148.5,149.1,149.6,150.1,150.7,151.2,151.8,152.3,152.8,153.3,153.7,154.2,154.7,155.2,155.6,156.0,156.3,156.7,157.1,157.5,157.8,158.1,158.4,158.7,159.0,159.3,159.6,159.8,160.0,160.2,160.5,160.7,160.8,160.9,161.1,161.2,161.3,161.5,161.6,161.7,161.8,161.9,161.9,162.0,162.1,162.2,162.3,162.4,162.5,162.6,162.7,162.8,162.9,163.0,163.1,163.2,163.3,163.4,163.5,163.6,163.7,163.8,],
"[-1SD]":[48.0,52.8,56.4,59.4,61.8,63.8,65.5,67.0,68.4,69.7,71.0,72.2,73.4,74.5,75.6,76.6,77.6,78.6,79.6,80.5,81.4,82.3,83.1,83.9,84.1,84.9,85.6,86.4,87.1,87.8,88.5,89.2,89.9,90.5,91.1,91.8,92.8,93.3,93.8,94.3,94.9,95.4,95.9,96.4,97.0,97.5,98.0,98.5,99.0,99.6,100.1,100.6,101.1,101.7,102.2,102.7,103.2,103.7,104.2,104.8,105.3,105.8,106.3,106.8,107.3,107.8,108.3,108.8,109.3,109.8,110.3,110.8,111.3,111.8,112.3,112.8,113.3,113.8,114.3,114.8,115.3,115.7,116.2,116.7,117.2,117.7,118.1,118.6,119.1,119.6,120.0,120.5,121.0,121.4,121.9,122.3,122.8,123.2,123.7,124.1,124.6,125.0,125.5,125.9,126.4,126.8,127.2,127.7,128.1,128.5,128.9,129.4,129.8,130.2,130.6,131.1,131.5,131.9,132.3,132.7,133.2,133.6,134.0,134.5,134.9,135.4,135.8,136.2,136.7,137.1,137.6,138.0,138.5,139.0,139.4,139.9,140.4,140.8,141.3,141.8,142.3,142.8,143.4,143.9,144.4,144.9,145.5,146.0,146.6,147.1,147.7,148.3,148.8,149.4,150.0,150.5,151.1,151.7,152.3,152.8,153.4,154.0,154.5,155.1,155.6,156.1,156.7,157.2,157.7,158.2,158.7,159.2,159.6,160.1,160.5,160.9,161.3,161.7,162.1,162.5,162.7,163.0,163.3,163.6,163.9,164.2,164.4,164.6,164.8,165.0,165.2,165.4,165.6,165.8,165.9,166.1,166.2,166.4,166.5,166.6,166.7,166.8,166.9,167.0,167.1,167.2,167.3,167.4,167.4,167.5,167.6,167.7,167.8,167.9,167.9,168.0,168.1,168.2,168.3,168.3,168.4,168.5,168.6,168.7,168.8,168.8,168.9,169.0,],
"Mean":[49.9,54.7,58.4,61.4,63.9,65.9,67.6,69.2,70.6,72.0,73.3,74.5,75.7,76.9,78.0,79.1,80.2,81.2,82.3,83.2,84.2,85.1,86.0,86.9,87.1,88.0,88.8,89.6,90.4,91.2,91.9,92.7,93.4,94.1,94.8,95.4,96.5,97.0,97.6,98.1,98.7,99.2,99.8,100.3,100.9,101.4,102.0,102.5,103.1,103.6,104.2,104.7,105.3,105.8,106.3,106.9,107.4,108.0,108.5,109.1,109.6,110.1,110.7,111.2,111.7,112.2,112.8,113.3,113.8,114.4,114.9,115.4,115.9,116.4,117.0,117.5,118.0,118.5,119.0,119.5,120.0,120.5,121.0,121.6,122.1,122.5,123.0,123.5,124.0,124.5,125.0,125.5,126.0,126.5,126.9,127.4,127.9,128.3,128.8,129.3,129.7,130.2,130.7,131.1,131.6,132.1,132.5,133.0,133.4,133.9,134.3,134.8,135.2,135.6,136.1,136.6,137.0,137.5,137.9,138.4,138.8,139.3,139.8,140.3,140.7,141.2,141.7,142.2,142.7,143.2,143.7,144.2,144.7,145.2,145.8,146.3,146.8,147.4,147.9,148.5,149.1,149.7,150.2,150.8,151.4,152.0,152.6,153.2,153.8,154.4,155.0,155.6,156.2,156.8,157.4,158.1,158.6,159.2,159.8,160.3,160.9,161.5,162.0,162.5,163.0,163.5,164.0,164.5,165.0,165.4,165.8,166.2,166.6,167.0,167.4,167.7,168.0,168.3,168.6,169.0,169.2,169.4,169.6,169.9,170.1,170.3,170.5,170.6,170.8,171.0,171.1,171.3,171.4,171.5,171.6,171.8,171.9,172.0,172.1,172.2,172.3,172.4,172.5,172.6,172.6,172.7,172.8,172.9,173.0,173.0,173.1,173.2,173.3,173.4,173.4,173.5,173.6,173.7,173.8,173.8,173.9,174.0,174.1,174.2,174.2,174.3,174.4,174.5,],
"[+1SD]":[51.8,56.7,60.4,63.5,66.0,68.0,69.8,71.3,72.8,74.2,75.6,76.9,78.1,79.3,80.5,81.7,82.8,83.9,85.0,86.0,87.0,88.0,89.0,89.9,90.2,91.1,92.0,92.9,93.7,94.5,95.3,96.1,96.9,97.6,98.4,99.1,100.6,101.1,101.7,102.2,102.8,103.4,103.9,104.5,105.1,105.6,106.2,106.7,107.3,107.9,108.4,109.0,109.5,110.1,110.7,111.2,111.8,112.3,112.9,113.4,114.0,114.6,115.1,115.7,116.2,116.8,117.3,117.9,118.5,119.0,119.6,120.1,120.7,121.2,121.8,122.3,122.9,123.4,123.9,124.5,125.0,125.5,126.1,126.6,127.1,127.6,128.1,128.7,129.2,129.7,130.2,130.7,131.2,131.7,132.2,132.7,133.1,133.6,134.1,134.6,135.1,135.6,136.0,136.5,137.0,137.5,137.9,138.4,138.9,139.4,139.9,140.3,140.8,141.3,141.8,142.3,142.8,143.3,143.8,144.3,144.8,145.3,145.8,146.3,146.8,147.3,147.9,148.4,149.0,149.5,150.0,150.6,151.2,151.8,152.3,152.9,153.5,154.1,154.7,155.3,156.0,156.6,157.2,157.8,158.5,159.1,159.7,160.4,161.0,161.6,162.2,162.8,163.5,164.1,164.7,165.3,165.8,166.3,166.9,167.4,168.0,168.5,169.0,169.4,169.9,170.3,170.7,171.2,171.5,171.9,172.2,172.6,172.9,173.3,173.6,173.8,174.1,174.3,174.6,174.9,175.1,175.3,175.4,175.6,175.8,176.0,176.2,176.3,176.4,176.6,176.7,176.9,177.0,177.1,177.2,177.3,177.4,177.5,177.6,177.7,177.8,177.9,178.0,178.1,178.2,178.3,178.4,178.4,178.5,178.6,178.7,178.8,178.9,179.0,179.0,179.1,179.2,179.3,179.4,179.5,179.6,179.6,179.7,179.8,179.9,180.0,180.1,180.1,],
"[+2SD]":[53.7,58.6,62.4,65.5,68.0,70.1,71.9,73.5,75.0,76.5,77.9,79.2,80.5,81.8,83.0,84.2,85.4,86.5,87.7,88.8,89.8,90.9,91.9,92.9,93.2,94.2,95.2,96.1,97.0,97.9,98.7,99.6,100.4,101.2,102.0,102.7,105.0,105.5,106.1,106.7,107.2,107.8,108.3,108.9,109.5,110.0,110.6,111.2,111.7,112.3,112.8,113.4,114.0,114.5,115.1,115.7,116.2,116.8,117.4,117.9,118.5,119.1,119.7,120.3,120.9,121.4,122.0,122.6,123.2,123.8,124.4,125.0,125.6,126.2,126.7,127.3,127.9,128.5,129.1,129.6,130.2,130.7,131.3,131.8,132.4,132.9,133.4,134.0,134.5,135.0,135.6,136.1,136.6,137.1,137.6,138.1,138.6,139.1,139.6,140.1,140.6,141.1,141.6,142.1,142.6,143.1,143.6,144.1,144.6,145.1,145.6,146.1,146.7,147.2,147.7,148.3,148.8,149.3,149.9,150.4,151.0,151.5,152.1,152.6,153.2,153.8,154.3,154.9,155.5,156.1,156.7,157.3,157.9,158.5,159.2,159.8,160.4,161.0,161.7,162.3,163.0,163.6,164.3,164.9,165.6,166.2,166.8,167.5,168.1,168.7,169.3,169.9,170.5,171.0,171.6,172.2,172.7,173.2,173.6,174.1,174.6,175.1,175.5,175.8,176.2,176.6,176.9,177.3,177.6,177.9,178.1,178.4,178.7,179.0,179.2,179.4,179.6,179.9,180.1,180.3,180.5,180.7,180.8,181.0,181.2,181.4,181.5,181.6,181.8,181.9,182.1,182.2,182.4,182.5,182.6,182.7,182.8,183.0,183.1,183.2,183.3,183.4,183.5,183.6,183.7,183.8,183.9,184.0,184.1,184.2,184.3,184.4,184.5,184.6,184.7,184.8,184.9,185.0,185.1,185.2,185.4,185.5,185.6,185.7,185.8,185.9,186.0,186.1,],
"[+3SD]":[55.6,60.6,64.4,67.6,70.1,72.2,74.0,75.7,77.2,78.7,80.1,81.5,82.9,84.2,85.5,86.7,88.0,89.2,90.4,91.5,92.6,93.8,94.9,95.9,96.3,97.3,98.3,99.3,100.3,101.2,102.1,103.0,103.9,104.8,105.6,106.4,109.8,110.4,110.9,111.4,112.0,112.5,113.1,113.6,114.2,114.7,115.3,115.8,116.4,116.9,117.5,118.0,118.6,119.1,119.7,120.3,120.8,121.4,122.0,122.5,123.1,123.7,124.4,125.0,125.6,126.2,126.8,127.5,128.1,128.8,129.4,130.0,130.7,131.3,131.9,132.5,133.2,133.8,134.4,134.9,135.5,136.1,136.7,137.3,137.8,138.4,138.9,139.5,140.1,140.6,141.1,141.7,142.2,142.7,143.2,143.8,144.3,144.8,145.2,145.7,146.2,146.7,147.2,147.8,148.3,148.8,149.3,149.9,150.4,151.0,151.6,152.2,152.7,153.3,153.9,154.5,155.1,155.7,156.3,156.9,157.5,158.1,158.7,159.3,159.9,160.5,161.1,161.7,162.4,163.0,163.6,164.2,164.9,165.6,166.2,166.9,167.5,168.2,168.8,169.5,170.1,170.8,171.4,172.1,172.7,173.3,173.9,174.5,175.1,175.7,176.2,176.8,177.3,177.8,178.3,178.9,179.3,179.7,180.1,180.5,180.9,181.3,181.6,181.9,182.1,182.4,182.7,183.0,183.2,183.4,183.6,183.8,184.0,184.2,184.4,184.6,184.8,185.0,185.2,185.3,185.5,185.7,185.9,186.0,186.2,186.4,186.6,186.7,186.9,187.1,187.2,187.4,187.6,187.7,187.9,188.0,188.2,188.3,188.5,188.6,188.7,188.9,189.0,189.2,189.3,189.4,189.5,189.7,189.8,189.9,190.1,190.2,190.3,190.4,190.5,190.7,190.8,190.9,191.0,191.2,191.3,191.4,191.6,191.7,191.8,192.0,192.1,192.2,],
}
pd.DataFrame(df).to_csv('cache/GC_Ht_M.csv')

In [ ]:
# GC_Ht_F (여아 신장)
df = {"Month":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,],
"L":[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5472,0.4997,0.4525,0.4056,0.359,0.3126,0.2825,0.2526,0.2229,0.1934,0.16399999999999998,0.1348,0.1129,0.0912,0.0695,0.048,0.0267,0.0054,-0.0216,-0.0484,-0.0751,-0.1016,-0.128,-0.1543,-0.1404,-0.1266,-0.113,-0.0994,-0.086,-0.0726,-0.0272,0.0175,0.0614,0.1047,0.1472,0.1891,0.2115,0.2335,0.255,0.2762,0.297,0.3174,0.2769,0.2373,0.1986,0.1606,0.1235,0.0872,0.0163,-0.0528,-0.1201,-0.1857,-0.2496,-0.3119,-0.3571,-0.4012,-0.4442,-0.4862,-0.5271,-0.5670000000000001,-0.5993,-0.6308,-0.6616,-0.6918,-0.7213,-0.7502,-0.779,-0.807,-0.8342,-0.8608,-0.8868,-0.9121,-0.8812,-0.851,-0.8214,-0.7924,-0.764,-0.7362,-0.6545,-0.574,-0.4946,-0.4162,-0.3389,-0.2627,-0.1573,-0.0528,0.0509,0.1538,0.2558,0.3569,0.4653,0.5738,0.6821,0.7904,0.8986,1.0066,1.1242,1.2433,1.3638,1.4857,1.609,1.7336,1.8239,1.9161,2.0103,2.1065,2.2047,2.3049,2.3447,2.3855,2.4272,2.4699,2.5136,2.5584,2.5648,2.5712,2.5775,2.5837,2.5899,2.5959,2.5607,2.5239,2.4856,2.4458,2.4042,2.3609,2.3149,2.2677,2.2195,2.1701,2.1196,2.0679,2.0549,2.0417,2.0284,2.015,2.0014,1.9877,1.9669999999999999,1.9462,1.9254,1.9045,1.8835,1.8625,1.79,1.7178,1.6457,1.5739,1.5022,1.4308,1.3515,1.2721,1.1928,1.1134,1.034,0.9547,0.8678,0.7803,0.6921,0.6032,0.5137,0.4236,0.332,0.2396,0.1464,0.0522,-0.0428,-0.1387,-0.16399999999999998,-0.1896,-0.2154,-0.2414,-0.2675,-0.294,-0.311,-0.3282,-0.3456,-0.3631,-0.3808,-0.3985,-0.4107,-0.4229,-0.4352,-0.4476,-0.4601,-0.4727,-0.4804,-0.4881,-0.4959,-0.5037,-0.5116,-0.5195,],
"M":[49.1477,53.6872,57.0673,59.8029,62.0899,64.0301,65.7311,67.2873,68.7498,70.1435,71.4818,72.771,74.015,75.2176,76.3817,77.5099,78.6055,79.671,80.7079,81.7182,82.7036,83.6654,84.604,85.5202,85.7153,86.5904,87.4462,88.28299999999999,89.1004,89.8991,90.6797,91.443,92.1906,92.9239,93.6444,94.3533,95.4078,95.9472,96.4867,97.0262,97.5658,98.1054,98.6465,99.1877,99.7288,100.27,100.8113,101.3525,101.8943,102.4361,102.9779,103.5197,104.0616,104.6034,105.1425,105.6816,106.2208,106.76,107.2992,107.8384,108.3714,108.9045,109.4375,109.9706,110.5036,111.0366,111.5656,112.0946,112.6235,113.1523,113.6811,114.2098,114.7289,115.2479,115.76700000000001,116.286,116.805,117.324,117.8257,118.3274,118.8292,119.331,119.8329,120.3348,120.8229,121.311,121.7993,122.2876,122.7761,123.2646,123.7505,124.2364,124.7224,125.2084,125.6944,126.1805,126.6703,127.1601,127.65,128.1399,128.6297,129.1197,129.6197,130.1199,130.62,131.1202,131.6204,132.1207,132.6442,133.1677,133.6912,134.2147,134.7382,135.2617,135.8116,136.3617,136.9119,137.4622,138.0126,138.5631,139.1218,139.6805,140.2395,140.7986,141.3578,141.9172,142.4689,143.0207,143.5726,144.1246,144.6765,145.2285,145.7568,146.2852,146.8136,147.3421,147.8706,148.399,148.8746,149.3501,149.8256,150.3011,150.7766,151.2521,151.6571,152.0621,152.4672,152.8722,153.2772,153.6821,154.0138,154.3455,154.6772,155.0089,155.3405,155.6722,155.9198,156.1672,156.4146,156.6619,156.9091,157.1562,157.3292,157.5021,157.675,157.8478,158.0205,158.1932,158.3159,158.4387,158.5614,158.6842,158.8069,158.9297,159.0139,159.0981,159.1823,159.2666,159.3508,159.4351,159.4917,159.5483,159.6051,159.6619,159.7189,159.776,159.8149,159.8539,159.893,159.9321,159.9714,160.0107,160.0286,160.0465,160.0644,160.0823,160.1002,160.1182,160.1342,160.1502,160.1661,160.1821,160.19799999999998,160.214,160.2483,160.2827,160.317,160.3514,160.3857,160.42,160.4524,160.4847,160.5171,160.5494,160.5817,160.614,160.6484,160.6827,160.7171,160.7514,160.7857,160.82,160.8591,160.8981,160.9372,160.9762,161.0153,161.0543,],
"S":[0.0379,0.0364,0.0357,0.0352,0.0349,0.0346,0.0345,0.0344,0.0344,0.0344,0.0345,0.0346,0.0348,0.035,0.0351,0.0353,0.0356,0.0358,0.036000000000000004,0.0362,0.0364,0.0367,0.0369,0.0371,0.0376,0.0379,0.0381,0.0383,0.0385,0.0387,0.0389,0.0391,0.0393,0.0395,0.0397,0.0399,0.0413,0.0412,0.0411,0.040999999999999995,0.0409,0.0408,0.0407,0.0406,0.0405,0.0404,0.0403,0.0402,0.0401,0.04,0.0399,0.0398,0.0397,0.0396,0.0395,0.0395,0.0394,0.0393,0.0392,0.0391,0.039,0.039,0.0389,0.0389,0.0388,0.0388,0.0388,0.0388,0.0388,0.0388,0.0388,0.0388,0.0388,0.0388,0.0389,0.0389,0.039,0.039,0.0391,0.0391,0.0392,0.0393,0.0394,0.0394,0.0396,0.0397,0.0398,0.0399,0.04,0.0401,0.0402,0.0404,0.0405,0.0406,0.0407,0.0408,0.040999999999999995,0.0411,0.0412,0.0413,0.0414,0.0415,0.0416,0.0417,0.0418,0.042,0.0421,0.0422,0.0423,0.0425,0.0426,0.0428,0.0429,0.043,0.0432,0.0433,0.0434,0.0435,0.0436,0.0438,0.0438,0.0439,0.0439,0.044000000000000004,0.044000000000000004,0.0441,0.044000000000000004,0.044000000000000004,0.0439,0.0438,0.0438,0.0437,0.0435,0.0433,0.0431,0.0429,0.0427,0.0425,0.0421,0.0418,0.0415,0.0412,0.0409,0.0406,0.0402,0.0399,0.0395,0.0392,0.0388,0.0385,0.0382,0.0378,0.0375,0.0372,0.0368,0.0365,0.0362,0.036000000000000004,0.0357,0.0355,0.0352,0.035,0.0348,0.0346,0.0345,0.0343,0.0342,0.034,0.0339,0.0338,0.0337,0.0336,0.0335,0.0335,0.0334,0.0333,0.0332,0.0332,0.0331,0.033,0.033,0.033,0.0329,0.0329,0.0328,0.0328,0.0328,0.0327,0.0327,0.0326,0.0326,0.0325,0.0325,0.0324,0.0323,0.0322,0.0322,0.0321,0.032,0.032,0.0319,0.0318,0.0317,0.0317,0.0316,0.0316,0.0315,0.0315,0.0314,0.0314,0.0313,0.0313,0.0313,0.0312,0.0312,0.0311,0.0311,0.031,0.031,0.031,0.0309,0.0309,0.0309,0.0308,0.0308,0.0307,0.0307,0.0307,],
"1p":[44.8,49.1,52.3,54.9,57.1,58.9,60.5,61.9,63.2,64.5,65.7,66.9,68.0,69.1,70.1,71.1,72.1,73.0,74.0,74.8,75.7,76.5,77.3,78.1,78.2,79.0,79.7,80.4,81.1,81.8,82.5,83.1,83.8,84.4,85.0,85.6,86.4,87.0,87.5,88.0,88.6,89.1,89.6,90.1,90.7,91.2,91.7,92.2,92.8,93.3,93.8,94.3,94.9,95.4,95.9,96.4,97.0,97.5,98.0,98.5,99.0,99.5,100.0,100.5,101.0,101.5,102.0,102.4,102.9,103.4,103.8,104.3,104.7,105.2,105.6,106.1,106.6,107.0,107.5,107.9,108.4,108.8,109.3,109.7,110.2,110.6,111.1,111.5,112.0,112.4,112.9,113.3,113.7,114.2,114.6,115.0,115.5,115.9,116.3,116.8,117.2,117.6,118.1,118.5,119.0,119.4,119.8,120.3,120.7,121.1,121.5,122.0,122.4,122.8,123.2,123.6,124.1,124.5,124.9,125.3,125.7,126.2,126.6,127.0,127.4,127.8,128.3,128.7,129.1,129.6,130.0,130.5,130.9,131.4,131.8,132.3,132.7,133.2,133.6,134.1,134.6,135.0,135.5,136.0,136.5,136.9,137.4,137.9,138.3,138.8,139.3,139.7,140.1,140.6,141.0,141.4,141.8,142.1,142.5,142.9,143.2,143.6,143.8,144.1,144.3,144.6,144.8,145.1,145.3,145.4,145.6,145.7,145.9,146.0,146.2,146.3,146.4,146.5,146.6,146.7,146.8,147.0,147.1,147.2,147.3,147.4,147.5,147.6,147.6,147.7,147.8,147.9,148.0,148.1,148.2,148.3,148.3,148.4,148.5,148.6,148.7,148.7,148.8,148.9,149.0,149.0,149.1,149.1,149.2,149.2,149.3,149.3,149.4,149.4,149.5,149.5,149.6,149.6,149.7,149.7,149.8,149.8,149.9,149.9,150.0,150.1,150.1,150.2,],
"3p":[45.6,50.0,53.2,55.8,58.0,59.9,61.5,62.9,64.3,65.6,66.8,68.0,69.2,70.3,71.3,72.4,73.3,74.3,75.2,76.2,77.0,77.9,78.7,79.6,79.6,80.4,81.2,81.9,82.6,83.4,84.0,84.7,85.4,86.0,86.7,87.3,88.1,88.7,89.2,89.7,90.2,90.8,91.3,91.8,92.4,92.9,93.4,93.9,94.5,95.0,95.5,96.0,96.6,97.1,97.6,98.1,98.7,99.2,99.7,100.2,100.7,101.2,101.7,102.2,102.7,103.3,103.7,104.2,104.7,105.2,105.6,106.1,106.6,107.1,107.5,108.0,108.5,108.9,109.4,109.9,110.3,110.8,111.2,111.7,112.2,112.6,113.1,113.5,114.0,114.4,114.8,115.3,115.7,116.2,116.6,117.0,117.5,117.9,118.4,118.8,119.3,119.7,120.1,120.6,121.0,121.5,121.9,122.4,122.8,123.3,123.7,124.1,124.6,125.0,125.5,125.9,126.4,126.8,127.3,127.7,128.2,128.6,129.1,129.5,130.0,130.5,130.9,131.4,131.9,132.3,132.8,133.3,133.8,134.2,134.7,135.2,135.7,136.2,136.7,137.1,137.6,138.1,138.6,139.1,139.5,140.0,140.5,140.9,141.4,141.9,142.3,142.7,143.1,143.5,143.9,144.3,144.7,145.0,145.3,145.7,146.0,146.3,146.5,146.8,147.0,147.2,147.5,147.7,147.9,148.0,148.2,148.3,148.4,148.6,148.7,148.8,148.9,149.0,149.1,149.2,149.3,149.4,149.5,149.6,149.7,149.8,149.9,149.9,150.0,150.1,150.2,150.2,150.3,150.4,150.4,150.5,150.6,150.6,150.7,150.7,150.8,150.9,150.9,151.0,151.0,151.1,151.1,151.2,151.3,151.3,151.4,151.4,151.4,151.5,151.5,151.6,151.6,151.7,151.7,151.8,151.8,151.9,151.9,152.0,152.0,152.1,152.1,152.2,],
"5p":[46.1,50.5,53.7,56.3,58.5,60.4,62.0,63.5,64.9,66.2,67.4,68.6,69.8,70.9,72.0,73.0,74.0,75.0,75.9,76.9,77.7,78.6,79.5,80.3,80.4,81.2,82.0,82.7,83.5,84.2,84.9,85.6,86.2,86.9,87.5,88.2,89.0,89.6,90.1,90.6,91.1,91.7,92.2,92.7,93.3,93.8,94.3,94.8,95.4,95.9,96.4,96.9,97.5,98.0,98.5,99.1,99.6,100.1,100.6,101.2,101.7,102.2,102.7,103.2,103.7,104.2,104.7,105.2,105.7,106.1,106.6,107.1,107.6,108.1,108.5,109.0,109.5,110.0,110.4,110.9,111.4,111.8,112.3,112.8,113.2,113.7,114.1,114.6,115.0,115.5,115.9,116.4,116.8,117.2,117.7,118.1,118.6,119.0,119.5,119.9,120.4,120.8,121.3,121.7,122.2,122.6,123.1,123.5,124.0,124.4,124.9,125.3,125.8,126.2,126.7,127.2,127.6,128.1,128.6,129.0,129.5,130.0,130.4,130.9,131.4,131.9,132.3,132.8,133.3,133.8,134.3,134.8,135.3,135.8,136.3,136.8,137.3,137.8,138.2,138.7,139.2,139.7,140.2,140.7,141.1,141.6,142.1,142.5,143.0,143.4,143.8,144.2,144.6,145.0,145.4,145.8,146.2,146.5,146.8,147.1,147.4,147.7,148.0,148.2,148.4,148.6,148.8,149.1,149.2,149.4,149.5,149.6,149.8,149.9,150.0,150.1,150.2,150.3,150.5,150.6,150.6,150.7,150.8,150.9,151.0,151.0,151.1,151.2,151.3,151.3,151.4,151.5,151.5,151.6,151.6,151.7,151.7,151.8,151.8,151.9,152.0,152.0,152.1,152.1,152.2,152.2,152.3,152.3,152.4,152.4,152.5,152.5,152.5,152.6,152.6,152.7,152.7,152.8,152.8,152.9,152.9,152.9,153.0,153.0,153.1,153.1,153.2,153.2,],
"10p":[46.8,51.2,54.5,57.1,59.3,61.2,62.8,64.3,65.7,67.0,68.3,69.5,70.7,71.8,72.9,74.0,75.0,76.0,77.0,77.9,78.8,79.7,80.6,81.5,81.6,82.4,83.2,83.9,84.7,85.4,86.2,86.9,87.5,88.2,88.9,89.5,90.4,90.9,91.5,92.0,92.5,93.1,93.6,94.1,94.7,95.2,95.7,96.2,96.8,97.3,97.8,98.4,98.9,99.4,99.9,100.5,101.0,101.5,102.1,102.6,103.1,103.6,104.1,104.6,105.2,105.7,106.2,106.7,107.2,107.7,108.2,108.6,109.1,109.6,110.1,110.6,111.1,111.6,112.0,112.5,113.0,113.4,113.9,114.4,114.8,115.3,115.8,116.2,116.7,117.1,117.6,118.0,118.5,118.9,119.4,119.8,120.3,120.7,121.2,121.7,122.1,122.6,123.0,123.5,123.9,124.4,124.9,125.3,125.8,126.3,126.7,127.2,127.7,128.1,128.6,129.1,129.6,130.1,130.6,131.1,131.6,132.1,132.6,133.0,133.5,134.0,134.6,135.1,135.6,136.1,136.6,137.1,137.6,138.1,138.6,139.1,139.6,140.2,140.6,141.1,141.6,142.1,142.6,143.1,143.5,144.0,144.4,144.9,145.3,145.8,146.2,146.6,146.9,147.3,147.7,148.1,148.4,148.7,149.0,149.3,149.6,149.9,150.1,150.3,150.5,150.7,150.9,151.1,151.3,151.4,151.5,151.7,151.8,152.0,152.1,152.2,152.3,152.4,152.5,152.6,152.6,152.7,152.8,152.8,152.9,153.0,153.1,153.1,153.2,153.2,153.3,153.3,153.4,153.4,153.5,153.5,153.6,153.6,153.7,153.7,153.7,153.8,153.8,153.9,153.9,154.0,154.0,154.0,154.1,154.1,154.2,154.2,154.3,154.3,154.3,154.4,154.4,154.5,154.5,154.5,154.6,154.6,154.7,154.7,154.8,154.8,154.9,154.9,],
"15p":[47.2,51.7,55.0,57.6,59.8,61.7,63.4,64.9,66.3,67.6,68.9,70.2,71.3,72.5,73.6,74.7,75.7,76.7,77.7,78.7,79.6,80.5,81.4,82.2,82.4,83.2,84.0,84.8,85.5,86.3,87.0,87.7,88.4,89.1,89.8,90.5,91.4,91.9,92.4,93.0,93.5,94.0,94.5,95.1,95.6,96.1,96.7,97.2,97.7,98.3,98.8,99.3,99.9,100.4,100.9,101.5,102.0,102.5,103.0,103.6,104.1,104.6,105.1,105.6,106.2,106.7,107.2,107.7,108.2,108.7,109.2,109.7,110.2,110.7,111.2,111.7,112.2,112.6,113.1,113.6,114.1,114.6,115.0,115.5,116.0,116.4,116.9,117.4,117.8,118.3,118.7,119.2,119.6,120.1,120.6,121.0,121.5,121.9,122.4,122.9,123.3,123.8,124.2,124.7,125.2,125.6,126.1,126.6,127.1,127.5,128.0,128.5,129.0,129.5,130.0,130.5,131.0,131.5,132.0,132.5,133.0,133.5,134.0,134.5,135.0,135.5,136.0,136.6,137.1,137.6,138.1,138.6,139.2,139.7,140.2,140.7,141.2,141.8,142.3,142.7,143.2,143.7,144.2,144.7,145.1,145.6,146.0,146.5,146.9,147.3,147.7,148.1,148.5,148.8,149.2,149.6,149.9,150.2,150.5,150.7,151.0,151.3,151.5,151.7,151.9,152.1,152.3,152.5,152.6,152.8,152.9,153.1,153.2,153.3,153.4,153.5,153.6,153.7,153.8,153.9,154.0,154.0,154.1,154.2,154.2,154.3,154.4,154.4,154.5,154.5,154.6,154.6,154.7,154.7,154.7,154.8,154.8,154.8,154.9,154.9,154.9,155.0,155.0,155.1,155.1,155.1,155.2,155.2,155.3,155.3,155.4,155.4,155.4,155.5,155.5,155.5,155.6,155.6,155.7,155.7,155.7,155.8,155.8,155.9,155.9,156.0,156.0,156.1,],
"25p":[47.9,52.4,55.7,58.4,60.6,62.5,64.2,65.7,67.2,68.5,69.8,71.1,72.3,73.4,74.6,75.7,76.7,77.7,78.7,79.7,80.7,81.6,82.5,83.4,83.5,84.4,85.2,86.0,86.8,87.6,88.3,89.0,89.7,90.4,91.1,91.8,92.8,93.3,93.8,94.4,94.9,95.4,96.0,96.5,97.0,97.6,98.1,98.6,99.2,99.7,100.2,100.8,101.3,101.8,102.4,102.9,103.4,104.0,104.5,105.0,105.6,106.1,106.6,107.1,107.7,108.2,108.7,109.2,109.7,110.2,110.7,111.3,111.8,112.3,112.8,113.3,113.8,114.3,114.8,115.2,115.7,116.2,116.7,117.2,117.6,118.1,118.6,119.0,119.5,120.0,120.5,120.9,121.4,121.8,122.3,122.8,123.2,123.7,124.2,124.7,125.1,125.6,126.1,126.5,127.0,127.5,128.0,128.5,129.0,129.5,129.9,130.4,130.9,131.4,132.0,132.5,133.0,133.5,134.0,134.5,135.1,135.6,136.1,136.7,137.2,137.7,138.3,138.8,139.3,139.9,140.4,140.9,141.5,142.0,142.5,143.1,143.6,144.1,144.6,145.1,145.6,146.1,146.5,147.0,147.5,147.9,148.3,148.8,149.2,149.6,150.0,150.3,150.7,151.0,151.4,151.8,152.0,152.3,152.6,152.8,153.1,153.4,153.6,153.8,154.0,154.1,154.3,154.5,154.6,154.8,154.9,155.0,155.2,155.3,155.4,155.5,155.6,155.7,155.8,155.8,155.9,156.0,156.0,156.1,156.2,156.2,156.3,156.3,156.4,156.4,156.5,156.5,156.5,156.6,156.6,156.6,156.6,156.7,156.7,156.7,156.8,156.8,156.8,156.8,156.9,156.9,157.0,157.0,157.0,157.1,157.1,157.1,157.2,157.2,157.3,157.3,157.3,157.4,157.4,157.4,157.5,157.5,157.6,157.6,157.6,157.7,157.7,157.8,],
"50p":[49.1,53.7,57.1,59.8,62.1,64.0,65.7,67.3,68.7,70.1,71.5,72.8,74.0,75.2,76.4,77.5,78.6,79.7,80.7,81.7,82.7,83.7,84.6,85.5,85.7,86.6,87.4,88.3,89.1,89.9,90.7,91.4,92.2,92.9,93.6,94.4,95.4,95.9,96.5,97.0,97.6,98.1,98.6,99.2,99.7,100.3,100.8,101.4,101.9,102.4,103.0,103.5,104.1,104.6,105.1,105.7,106.2,106.8,107.3,107.8,108.4,108.9,109.4,110.0,110.5,111.0,111.6,112.1,112.6,113.2,113.7,114.2,114.7,115.2,115.8,116.3,116.8,117.3,117.8,118.3,118.8,119.3,119.8,120.3,120.8,121.3,121.8,122.3,122.8,123.3,123.8,124.2,124.7,125.2,125.7,126.2,126.7,127.2,127.6,128.1,128.6,129.1,129.6,130.1,130.6,131.1,131.6,132.1,132.6,133.2,133.7,134.2,134.7,135.3,135.8,136.4,136.9,137.5,138.0,138.6,139.1,139.7,140.2,140.8,141.4,141.9,142.5,143.0,143.6,144.1,144.7,145.2,145.8,146.3,146.8,147.3,147.9,148.4,148.9,149.4,149.8,150.3,150.8,151.3,151.7,152.1,152.5,152.9,153.3,153.7,154.0,154.3,154.7,155.0,155.3,155.7,155.9,156.2,156.4,156.7,156.9,157.2,157.3,157.5,157.7,157.8,158.0,158.2,158.3,158.4,158.6,158.7,158.8,158.9,159.0,159.1,159.2,159.3,159.4,159.4,159.5,159.5,159.6,159.7,159.7,159.8,159.8,159.9,159.9,159.9,160.0,160.0,160.0,160.0,160.1,160.1,160.1,160.1,160.1,160.2,160.2,160.2,160.2,160.2,160.2,160.3,160.3,160.4,160.4,160.4,160.5,160.5,160.5,160.5,160.6,160.6,160.6,160.7,160.7,160.8,160.8,160.8,160.9,160.9,160.9,161.0,161.0,161.1,],
"75p":[50.4,55.0,58.4,61.2,63.5,65.5,67.3,68.8,70.3,71.8,73.1,74.5,75.8,77.0,78.2,79.4,80.5,81.6,82.7,83.7,84.7,85.7,86.7,87.7,87.9,88.8,89.7,90.6,91.4,92.2,93.1,93.9,94.6,95.4,96.2,96.9,98.1,98.6,99.2,99.7,100.3,100.8,101.4,101.9,102.5,103.0,103.6,104.1,104.7,105.2,105.8,106.3,106.9,107.4,108.0,108.5,109.1,109.6,110.2,110.7,111.3,111.8,112.4,112.9,113.4,114.0,114.5,115.1,115.6,116.1,116.7,117.2,117.8,118.3,118.8,119.4,119.9,120.4,121.0,121.5,122.0,122.5,123.1,123.6,124.1,124.6,125.1,125.6,126.1,126.7,127.2,127.7,128.2,128.7,129.2,129.7,130.2,130.8,131.3,131.8,132.3,132.8,133.3,133.9,134.4,134.9,135.5,136.0,136.5,137.1,137.6,138.2,138.7,139.3,139.9,140.4,141.0,141.6,142.2,142.7,143.3,143.9,144.5,145.0,145.6,146.2,146.7,147.3,147.8,148.4,149.0,149.5,150.0,150.5,151.1,151.6,152.1,152.6,153.1,153.5,154.0,154.4,154.9,155.3,155.7,156.1,156.5,156.8,157.2,157.6,157.9,158.2,158.5,158.8,159.1,159.4,159.7,159.9,160.1,160.3,160.6,160.8,161.0,161.1,161.3,161.5,161.6,161.8,161.9,162.0,162.1,162.2,162.4,162.5,162.6,162.6,162.7,162.8,162.9,163.0,163.0,163.1,163.1,163.2,163.2,163.3,163.3,163.4,163.4,163.4,163.5,163.5,163.5,163.6,163.6,163.6,163.6,163.6,163.6,163.6,163.6,163.7,163.7,163.7,163.7,163.7,163.8,163.8,163.8,163.9,163.9,163.9,163.9,164.0,164.0,164.0,164.1,164.1,164.1,164.2,164.2,164.2,164.3,164.3,164.3,164.4,164.4,164.4,],
"85p":[51.1,55.7,59.2,62.0,64.3,66.3,68.1,69.7,71.2,72.6,74.0,75.4,76.7,77.9,79.2,80.3,81.5,82.6,83.7,84.8,85.8,86.8,87.8,88.8,89.1,90.0,90.9,91.8,92.7,93.5,94.3,95.2,95.9,96.7,97.5,98.3,99.5,100.1,100.6,101.2,101.8,102.3,102.9,103.4,104.0,104.5,105.1,105.7,106.2,106.8,107.3,107.9,108.4,109.0,109.5,110.1,110.7,111.2,111.8,112.3,112.9,113.4,114.0,114.5,115.0,115.6,116.1,116.7,117.2,117.8,118.3,118.9,119.4,120.0,120.5,121.0,121.6,122.1,122.7,123.2,123.7,124.3,124.8,125.3,125.9,126.4,126.9,127.5,128.0,128.5,129.1,129.6,130.1,130.6,131.2,131.7,132.2,132.8,133.3,133.8,134.4,134.9,135.4,136.0,136.5,137.1,137.6,138.2,138.7,139.3,139.8,140.4,141.0,141.5,142.1,142.7,143.3,143.9,144.4,145.0,145.6,146.2,146.8,147.3,147.9,148.5,149.1,149.6,150.2,150.7,151.3,151.8,152.3,152.8,153.3,153.8,154.3,154.8,155.3,155.7,156.1,156.6,157.0,157.4,157.8,158.2,158.5,158.9,159.3,159.6,159.9,160.2,160.5,160.8,161.1,161.4,161.6,161.8,162.1,162.3,162.5,162.7,162.9,163.0,163.2,163.4,163.5,163.7,163.8,163.9,164.0,164.1,164.2,164.3,164.4,164.5,164.6,164.7,164.7,164.8,164.9,164.9,165.0,165.1,165.1,165.2,165.2,165.2,165.3,165.3,165.4,165.4,165.4,165.4,165.5,165.5,165.5,165.5,165.5,165.5,165.5,165.5,165.6,165.6,165.6,165.6,165.7,165.7,165.7,165.7,165.8,165.8,165.8,165.9,165.9,165.9,165.9,166.0,166.0,166.0,166.1,166.1,166.1,166.2,166.2,166.2,166.3,166.3,],
"90p":[51.5,56.2,59.7,62.5,64.9,66.9,68.6,70.3,71.8,73.2,74.6,76.0,77.3,78.6,79.8,81.0,82.2,83.3,84.4,85.5,86.6,87.6,88.6,89.6,89.9,90.8,91.7,92.6,93.5,94.4,95.2,96.0,96.8,97.6,98.4,99.2,100.5,101.1,101.6,102.2,102.8,103.3,103.9,104.5,105.0,105.6,106.1,106.7,107.3,107.8,108.4,108.9,109.5,110.1,110.6,111.2,111.7,112.3,112.8,113.4,114.0,114.5,115.1,115.6,116.2,116.7,117.3,117.8,118.4,118.9,119.4,120.0,120.5,121.1,121.6,122.2,122.7,123.3,123.8,124.4,124.9,125.5,126.0,126.6,127.1,127.6,128.2,128.7,129.3,129.8,130.4,130.9,131.4,132.0,132.5,133.1,133.6,134.1,134.7,135.2,135.8,136.3,136.9,137.4,138.0,138.5,139.1,139.7,140.2,140.8,141.4,141.9,142.5,143.1,143.7,144.3,144.9,145.4,146.0,146.6,147.2,147.8,148.4,148.9,149.5,150.1,150.6,151.2,151.7,152.3,152.8,153.4,153.9,154.3,154.8,155.3,155.8,156.3,156.7,157.2,157.6,158.0,158.4,158.9,159.2,159.6,159.9,160.3,160.6,161.0,161.3,161.6,161.8,162.1,162.4,162.7,162.9,163.1,163.3,163.6,163.8,164.0,164.2,164.3,164.5,164.6,164.8,164.9,165.0,165.2,165.3,165.4,165.5,165.6,165.7,165.8,165.8,165.9,166.0,166.1,166.1,166.2,166.3,166.3,166.4,166.4,166.5,166.5,166.6,166.6,166.6,166.7,166.7,166.7,166.7,166.8,166.8,166.8,166.8,166.8,166.8,166.8,166.9,166.9,166.9,166.9,167.0,167.0,167.0,167.0,167.1,167.1,167.1,167.1,167.2,167.2,167.2,167.3,167.3,167.3,167.3,167.4,167.4,167.4,167.5,167.5,167.5,167.6,],
"95p":[52.2,56.9,60.4,63.3,65.7,67.7,69.5,71.1,72.6,74.1,75.5,76.9,78.3,79.5,80.8,82.0,83.2,84.4,85.5,86.6,87.7,88.7,89.7,90.7,91.0,92.0,92.9,93.8,94.7,95.6,96.5,97.3,98.2,99.0,99.8,100.5,102.0,102.6,103.1,103.7,104.3,104.8,105.4,106.0,106.5,107.1,107.7,108.3,108.8,109.4,110.0,110.5,111.1,111.6,112.2,112.8,113.3,113.9,114.5,115.0,115.6,116.1,116.7,117.3,117.8,118.4,118.9,119.5,120.0,120.6,121.1,121.7,122.2,122.8,123.3,123.9,124.5,125.0,125.6,126.1,126.7,127.3,127.8,128.4,128.9,129.5,130.1,130.6,131.2,131.8,132.3,132.9,133.4,134.0,134.6,135.1,135.7,136.2,136.8,137.4,137.9,138.5,139.1,139.6,140.2,140.8,141.4,141.9,142.5,143.1,143.7,144.3,144.9,145.5,146.1,146.6,147.2,147.8,148.4,149.0,149.6,150.2,150.7,151.3,151.9,152.5,153.0,153.5,154.1,154.6,155.1,155.7,156.1,156.6,157.1,157.6,158.0,158.5,158.9,159.3,159.7,160.1,160.5,160.9,161.3,161.6,162.0,162.3,162.6,163.0,163.2,163.5,163.8,164.1,164.3,164.6,164.8,165.0,165.2,165.4,165.7,165.9,166.0,166.2,166.3,166.5,166.6,166.8,166.9,167.0,167.1,167.2,167.3,167.4,167.5,167.6,167.7,167.8,167.8,167.9,168.0,168.0,168.1,168.2,168.2,168.3,168.3,168.4,168.4,168.5,168.5,168.6,168.6,168.6,168.6,168.7,168.7,168.7,168.7,168.7,168.8,168.8,168.8,168.8,168.8,168.9,168.9,168.9,169.0,169.0,169.0,169.0,169.1,169.1,169.1,169.1,169.2,169.2,169.2,169.3,169.3,169.3,169.3,169.4,169.4,169.4,169.5,169.5,],
"97p":[52.7,57.4,60.9,63.8,66.2,68.2,70.0,71.6,73.2,74.7,76.1,77.5,78.9,80.2,81.4,82.7,83.9,85.0,86.2,87.3,88.4,89.4,90.5,91.5,91.8,92.8,93.7,94.6,95.6,96.4,97.3,98.2,99.0,99.8,100.6,101.4,103.0,103.5,104.1,104.7,105.3,105.8,106.4,107.0,107.6,108.1,108.7,109.3,109.8,110.4,111.0,111.6,112.1,112.7,113.3,113.8,114.4,115.0,115.5,116.1,116.7,117.2,117.8,118.3,118.9,119.5,120.0,120.6,121.1,121.7,122.2,122.8,123.3,123.9,124.5,125.0,125.6,126.1,126.7,127.3,127.9,128.4,129.0,129.6,130.2,130.7,131.3,131.9,132.5,133.0,133.6,134.2,134.8,135.3,135.9,136.5,137.1,137.6,138.2,138.8,139.4,139.9,140.5,141.1,141.7,142.3,142.9,143.5,144.1,144.7,145.2,145.8,146.4,147.0,147.6,148.2,148.8,149.4,150.0,150.6,151.2,151.7,152.3,152.9,153.4,154.0,154.5,155.1,155.6,156.1,156.6,157.2,157.6,158.1,158.5,159.0,159.5,159.9,160.3,160.7,161.1,161.5,161.9,162.3,162.6,162.9,163.3,163.6,163.9,164.2,164.5,164.8,165.0,165.3,165.6,165.8,166.0,166.2,166.4,166.7,166.9,167.1,167.2,167.4,167.5,167.7,167.8,168.0,168.1,168.2,168.3,168.4,168.5,168.6,168.7,168.8,168.9,168.9,169.0,169.1,169.2,169.2,169.3,169.4,169.4,169.5,169.6,169.6,169.7,169.7,169.8,169.8,169.8,169.9,169.9,169.9,169.9,170.0,170.0,170.0,170.0,170.0,170.1,170.1,170.1,170.1,170.2,170.2,170.2,170.3,170.3,170.3,170.3,170.4,170.4,170.4,170.4,170.5,170.5,170.5,170.6,170.6,170.6,170.6,170.7,170.7,170.7,170.8,],
"99p":[53.5,58.2,61.8,64.7,67.1,69.2,71.0,72.7,74.3,75.8,77.2,78.6,80.0,81.3,82.6,83.9,85.1,86.3,87.5,88.6,89.7,90.8,91.9,92.9,93.2,94.2,95.2,96.1,97.1,98.0,98.9,99.8,100.6,101.5,102.3,103.1,104.8,105.4,106.0,106.5,107.1,107.7,108.3,108.9,109.5,110.1,110.6,111.2,111.8,112.4,113.0,113.5,114.1,114.7,115.3,115.9,116.4,117.0,117.6,118.2,118.7,119.3,119.9,120.4,121.0,121.5,122.1,122.7,123.2,123.8,124.3,124.9,125.5,126.0,126.6,127.2,127.7,128.3,128.9,129.5,130.1,130.7,131.3,131.9,132.5,133.1,133.7,134.3,134.9,135.5,136.1,136.7,137.3,137.9,138.5,139.1,139.7,140.3,140.9,141.5,142.1,142.7,143.3,144.0,144.6,145.2,145.8,146.4,147.0,147.6,148.3,148.9,149.5,150.1,150.7,151.3,151.9,152.4,153.0,153.6,154.2,154.7,155.3,155.8,156.4,157.0,157.5,158.0,158.5,159.0,159.5,160.0,160.4,160.8,161.3,161.7,162.1,162.6,162.9,163.3,163.7,164.0,164.4,164.7,165.0,165.4,165.7,166.0,166.3,166.6,166.8,167.1,167.3,167.6,167.8,168.1,168.3,168.5,168.7,168.9,169.1,169.3,169.5,169.6,169.8,169.9,170.1,170.2,170.3,170.4,170.5,170.6,170.8,170.9,170.9,171.0,171.1,171.2,171.2,171.3,171.4,171.5,171.5,171.6,171.7,171.8,171.8,171.9,172.0,172.0,172.1,172.1,172.2,172.2,172.2,172.3,172.3,172.3,172.4,172.4,172.4,172.5,172.5,172.5,172.6,172.6,172.6,172.6,172.7,172.7,172.7,172.8,172.8,172.8,172.8,172.9,172.9,172.9,172.9,173.0,173.0,173.0,173.0,173.1,173.1,173.1,173.2,173.2,],
"[-3SD]":[43.6,47.8,51.0,53.5,55.6,57.4,58.9,60.3,61.7,62.9,64.1,65.2,66.3,67.3,68.3,69.3,70.2,71.1,72.0,72.8,73.7,74.5,75.2,76.0,76.0,76.8,77.5,78.1,78.8,79.5,80.1,80.7,81.3,81.9,82.5,83.1,83.9,84.4,85.0,85.5,86.1,86.6,87.1,87.6,88.2,88.7,89.2,89.7,90.3,90.8,91.3,91.8,92.4,92.9,93.4,93.9,94.4,95.0,95.5,96.0,96.5,97.0,97.5,97.9,98.4,98.9,99.3,99.8,100.2,100.7,101.1,101.5,102.0,102.4,102.8,103.3,103.7,104.1,104.6,105.0,105.5,105.9,106.4,106.8,107.3,107.7,108.2,108.6,109.1,109.5,110.0,110.4,110.8,111.2,111.7,112.1,112.5,112.9,113.4,113.8,114.2,114.6,115.1,115.5,115.9,116.4,116.8,117.2,117.6,118.0,118.4,118.8,119.2,119.6,119.9,120.3,120.7,121.1,121.4,121.8,122.1,122.5,122.9,123.2,123.6,123.9,124.3,124.7,125.1,125.4,125.8,126.2,126.6,127.0,127.4,127.7,128.1,128.5,128.9,129.4,129.8,130.3,130.7,131.1,131.6,132.1,132.6,133.1,133.5,134.0,134.5,135.0,135.4,135.9,136.4,136.9,137.3,137.7,138.1,138.5,138.9,139.3,139.6,139.9,140.2,140.5,140.8,141.1,141.2,141.4,141.6,141.8,141.9,142.1,142.2,142.4,142.5,142.6,142.8,142.9,143.0,143.2,143.3,143.4,143.6,143.7,143.8,143.9,144.1,144.2,144.3,144.4,144.5,144.7,144.8,144.9,145.0,145.1,145.2,145.3,145.5,145.6,145.7,145.8,145.9,145.9,146.0,146.1,146.1,146.2,146.3,146.3,146.4,146.4,146.5,146.5,146.6,146.7,146.7,146.8,146.8,146.9,146.9,147.0,147.0,147.1,147.2,147.2,],
"[-2SD]":[45.4,49.8,53.0,55.6,57.8,59.6,61.2,62.7,64.0,65.3,66.5,67.7,68.9,70.0,71.0,72.0,73.0,74.0,74.9,75.8,76.7,77.5,78.4,79.2,79.3,80.0,80.8,81.5,82.2,82.9,83.6,84.3,84.9,85.6,86.2,86.8,87.7,88.2,88.7,89.3,89.8,90.3,90.8,91.4,91.9,92.4,93.0,93.5,94.0,94.5,95.1,95.6,96.1,96.6,97.2,97.7,98.2,98.7,99.3,99.8,100.3,100.8,101.3,101.8,102.3,102.8,103.3,103.7,104.2,104.7,105.2,105.6,106.1,106.6,107.0,107.5,108.0,108.4,108.9,109.3,109.8,110.3,110.7,111.2,111.6,112.1,112.5,113.0,113.4,113.9,114.3,114.7,115.2,115.6,116.1,116.5,116.9,117.4,117.8,118.3,118.7,119.1,119.6,120.0,120.5,120.9,121.4,121.8,122.2,122.7,123.1,123.6,124.0,124.4,124.9,125.3,125.8,126.2,126.6,127.1,127.5,128.0,128.4,128.9,129.3,129.8,130.2,130.7,131.1,131.6,132.1,132.5,133.0,133.5,134.0,134.4,134.9,135.4,135.9,136.3,136.8,137.3,137.8,138.3,138.7,139.2,139.7,140.1,140.6,141.1,141.5,141.9,142.3,142.7,143.1,143.6,143.9,144.2,144.6,144.9,145.3,145.6,145.8,146.1,146.3,146.5,146.8,147.0,147.2,147.3,147.5,147.6,147.8,147.9,148.0,148.1,148.2,148.4,148.5,148.6,148.7,148.8,148.9,149.0,149.0,149.1,149.2,149.3,149.4,149.5,149.5,149.6,149.7,149.8,149.8,149.9,150.0,150.0,150.1,150.2,150.2,150.3,150.4,150.4,150.5,150.5,150.6,150.6,150.7,150.7,150.8,150.8,150.9,150.9,151.0,151.0,151.1,151.1,151.2,151.2,151.3,151.3,151.4,151.4,151.5,151.5,151.6,151.6,],
"[-1SD]":[47.3,51.7,55.0,57.7,59.9,61.8,63.5,65.0,66.4,67.7,69.0,70.3,71.4,72.6,73.7,74.8,75.8,76.8,77.8,78.8,79.7,80.6,81.5,82.3,82.5,83.3,84.1,84.9,85.7,86.4,87.1,87.9,88.6,89.3,89.9,90.6,91.5,92.0,92.6,93.1,93.6,94.2,94.7,95.2,95.8,96.3,96.8,97.3,97.9,98.4,98.9,99.5,100.0,100.5,101.1,101.6,102.1,102.7,103.2,103.7,104.2,104.8,105.3,105.8,106.3,106.8,107.3,107.8,108.3,108.8,109.3,109.9,110.3,110.8,111.3,111.8,112.3,112.8,113.3,113.8,114.2,114.7,115.2,115.7,116.1,116.6,117.1,117.5,118.0,118.4,118.9,119.4,119.8,120.3,120.7,121.2,121.6,122.1,122.6,123.0,123.5,124.0,124.4,124.9,125.4,125.8,126.3,126.8,127.2,127.7,128.2,128.7,129.2,129.7,130.2,130.7,131.2,131.7,132.2,132.7,133.2,133.7,134.2,134.7,135.2,135.7,136.3,136.8,137.3,137.8,138.4,138.9,139.4,139.9,140.4,141.0,141.5,142.0,142.5,143.0,143.5,144.0,144.5,144.9,145.4,145.8,146.3,146.7,147.1,147.6,148.0,148.3,148.7,149.1,149.4,149.8,150.1,150.4,150.7,151.0,151.2,151.5,151.7,151.9,152.1,152.3,152.5,152.7,152.8,153.0,153.1,153.3,153.4,153.5,153.6,153.7,153.8,153.9,154.0,154.1,154.2,154.2,154.3,154.4,154.4,154.5,154.5,154.6,154.7,154.7,154.8,154.8,154.8,154.9,154.9,155.0,155.0,155.0,155.1,155.1,155.1,155.2,155.2,155.2,155.3,155.3,155.4,155.4,155.4,155.5,155.5,155.6,155.6,155.6,155.7,155.7,155.8,155.8,155.8,155.9,155.9,156.0,156.0,156.1,156.1,156.1,156.2,156.2,],
"Mean":[49.1,53.7,57.1,59.8,62.1,64.0,65.7,67.3,68.7,70.1,71.5,72.8,74.0,75.2,76.4,77.5,78.6,79.7,80.7,81.7,82.7,83.7,84.6,85.5,85.7,86.6,87.4,88.3,89.1,89.9,90.7,91.4,92.2,92.9,93.6,94.4,95.4,95.9,96.5,97.0,97.6,98.1,98.6,99.2,99.7,100.3,100.8,101.4,101.9,102.4,103.0,103.5,104.1,104.6,105.1,105.7,106.2,106.8,107.3,107.8,108.4,108.9,109.4,110.0,110.5,111.0,111.6,112.1,112.6,113.2,113.7,114.2,114.7,115.2,115.8,116.3,116.8,117.3,117.8,118.3,118.8,119.3,119.8,120.3,120.8,121.3,121.8,122.3,122.8,123.3,123.8,124.2,124.7,125.2,125.7,126.2,126.7,127.2,127.6,128.1,128.6,129.1,129.6,130.1,130.6,131.1,131.6,132.1,132.6,133.2,133.7,134.2,134.7,135.3,135.8,136.4,136.9,137.5,138.0,138.6,139.1,139.7,140.2,140.8,141.4,141.9,142.5,143.0,143.6,144.1,144.7,145.2,145.8,146.3,146.8,147.3,147.9,148.4,148.9,149.4,149.8,150.3,150.8,151.3,151.7,152.1,152.5,152.9,153.3,153.7,154.0,154.3,154.7,155.0,155.3,155.7,155.9,156.2,156.4,156.7,156.9,157.2,157.3,157.5,157.7,157.8,158.0,158.2,158.3,158.4,158.6,158.7,158.8,158.9,159.0,159.1,159.2,159.3,159.4,159.4,159.5,159.5,159.6,159.7,159.7,159.8,159.8,159.9,159.9,159.9,160.0,160.0,160.0,160.0,160.1,160.1,160.1,160.1,160.1,160.2,160.2,160.2,160.2,160.2,160.2,160.3,160.3,160.4,160.4,160.4,160.5,160.5,160.5,160.5,160.6,160.6,160.6,160.7,160.7,160.8,160.8,160.8,160.9,160.9,160.9,161.0,161.0,161.1,],
"[+1SD]":[51.0,55.6,59.1,61.9,64.3,66.2,68.0,69.6,71.1,72.6,73.9,75.3,76.6,77.8,79.1,80.2,81.4,82.5,83.6,84.7,85.7,86.7,87.7,88.7,88.9,89.9,90.8,91.7,92.5,93.4,94.2,95.0,95.8,96.6,97.4,98.1,99.4,99.9,100.5,101.1,101.6,102.2,102.7,103.3,103.8,104.4,104.9,105.5,106.1,106.6,107.2,107.7,108.3,108.8,109.4,109.9,110.5,111.0,111.6,112.2,112.7,113.2,113.8,114.3,114.9,115.4,116.0,116.5,117.1,117.6,118.2,118.7,119.2,119.8,120.3,120.9,121.4,122.0,122.5,123.0,123.6,124.1,124.6,125.2,125.7,126.2,126.8,127.3,127.8,128.3,128.9,129.4,129.9,130.5,131.0,131.5,132.0,132.6,133.1,133.6,134.1,134.7,135.2,135.8,136.3,136.8,137.4,137.9,138.5,139.1,139.6,140.2,140.7,141.3,141.9,142.5,143.1,143.6,144.2,144.8,145.4,146.0,146.5,147.1,147.7,148.3,148.8,149.4,149.9,150.5,151.0,151.6,152.1,152.6,153.1,153.6,154.1,154.6,155.0,155.5,155.9,156.4,156.8,157.2,157.6,158.0,158.3,158.7,159.1,159.4,159.7,160.0,160.3,160.6,160.9,161.2,161.4,161.6,161.9,162.1,162.3,162.5,162.7,162.8,163.0,163.2,163.3,163.5,163.6,163.7,163.8,163.9,164.0,164.2,164.2,164.3,164.4,164.5,164.6,164.6,164.7,164.7,164.8,164.9,164.9,165.0,165.0,165.1,165.1,165.1,165.2,165.2,165.2,165.2,165.3,165.3,165.3,165.3,165.3,165.3,165.3,165.4,165.4,165.4,165.4,165.4,165.5,165.5,165.5,165.6,165.6,165.6,165.6,165.7,165.7,165.7,165.8,165.8,165.8,165.8,165.9,165.9,165.9,166.0,166.0,166.0,166.1,166.1,],
"[+2SD]":[52.9,57.6,61.1,64.0,66.4,68.5,70.3,71.9,73.5,75.0,76.4,77.8,79.2,80.5,81.7,83.0,84.2,85.4,86.5,87.6,88.7,89.8,90.8,91.9,92.2,93.1,94.1,95.0,96.0,96.9,97.7,98.6,99.4,100.3,101.1,101.9,103.4,104.0,104.6,105.2,105.8,106.3,106.9,107.5,108.1,108.6,109.2,109.8,110.4,110.9,111.5,112.1,112.7,113.2,113.8,114.4,114.9,115.5,116.1,116.7,117.2,117.8,118.3,118.9,119.5,120.0,120.6,121.1,121.7,122.2,122.8,123.3,123.9,124.5,125.0,125.6,126.2,126.7,127.3,127.9,128.4,129.0,129.6,130.2,130.8,131.4,131.9,132.5,133.1,133.7,134.3,134.9,135.4,136.0,136.6,137.2,137.8,138.3,138.9,139.5,140.1,140.7,141.3,141.9,142.5,143.1,143.6,144.2,144.8,145.4,146.0,146.6,147.2,147.8,148.4,149.0,149.6,150.2,150.8,151.4,152.0,152.5,153.1,153.7,154.2,154.8,155.3,155.8,156.4,156.9,157.4,157.9,158.4,158.8,159.3,159.7,160.2,160.6,161.0,161.4,161.8,162.2,162.5,162.9,163.3,163.6,163.9,164.2,164.5,164.9,165.1,165.4,165.7,165.9,166.2,166.4,166.6,166.8,167.1,167.3,167.5,167.7,167.8,168.0,168.1,168.3,168.4,168.6,168.7,168.8,168.9,169.0,169.1,169.2,169.3,169.4,169.5,169.5,169.6,169.7,169.8,169.8,169.9,170.0,170.0,170.1,170.2,170.2,170.3,170.3,170.4,170.4,170.5,170.5,170.5,170.5,170.6,170.6,170.6,170.6,170.7,170.7,170.7,170.7,170.8,170.8,170.8,170.9,170.9,170.9,170.9,171.0,171.0,171.0,171.0,171.1,171.1,171.1,171.1,171.2,171.2,171.2,171.3,171.3,171.3,171.4,171.4,171.4,],
"[+3SD]":[54.7,59.5,63.2,66.1,68.6,70.7,72.5,74.2,75.8,77.4,78.9,80.3,81.7,83.1,84.4,85.7,87.0,88.2,89.4,90.6,91.7,92.9,94.0,95.0,95.4,96.4,97.4,98.4,99.4,100.3,101.3,102.2,103.1,103.9,104.8,105.6,107.6,108.2,108.8,109.4,110.0,110.6,111.2,111.8,112.4,113.0,113.6,114.2,114.8,115.4,116.0,116.6,117.2,117.8,118.4,119.0,119.6,120.2,120.8,121.4,122.0,122.5,123.1,123.7,124.2,124.8,125.3,125.9,126.5,127.0,127.6,128.1,128.7,129.3,129.9,130.4,131.0,131.6,132.2,132.9,133.5,134.1,134.7,135.4,136.0,136.7,137.4,138.0,138.7,139.4,140.0,140.7,141.3,142.0,142.6,143.3,143.9,144.6,145.2,145.8,146.5,147.1,147.8,148.5,149.1,149.8,150.5,151.1,151.8,152.4,153.0,153.7,154.3,154.9,155.5,156.1,156.6,157.2,157.8,158.4,158.9,159.4,159.9,160.4,161.0,161.5,162.0,162.4,162.9,163.3,163.8,164.3,164.6,165.0,165.4,165.7,166.1,166.5,166.8,167.1,167.4,167.8,168.1,168.4,168.7,168.9,169.2,169.5,169.8,170.0,170.3,170.5,170.7,171.0,171.2,171.4,171.6,171.8,172.0,172.2,172.4,172.6,172.8,172.9,173.1,173.2,173.4,173.5,173.6,173.7,173.8,174.0,174.1,174.2,174.2,174.3,174.4,174.5,174.5,174.6,174.7,174.8,174.9,175.0,175.1,175.2,175.3,175.3,175.4,175.5,175.6,175.7,175.7,175.8,175.8,175.9,175.9,176.0,176.0,176.1,176.1,176.2,176.2,176.3,176.3,176.4,176.4,176.4,176.5,176.5,176.5,176.5,176.6,176.6,176.6,176.6,176.7,176.7,176.7,176.8,176.8,176.8,176.8,176.9,176.9,176.9,176.9,177.0,],
}
pd.DataFrame(df).to_csv('cache/GC_Ht_F.csv')

In [ ]:
# GC_Wt_M (남아 체중)
df = {"Month":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,],
"L":[0.3487,0.2297,0.19699999999999998,0.1738,0.1553,0.1395,0.1257,0.1134,0.1021,0.0917,0.08199999999999999,0.073,0.0644,0.0563,0.0487,0.0413,0.0343,0.0275,0.0211,0.0148,0.0087,0.0029,-0.0028,-0.0083,-0.0137,-0.0189,-0.024,-0.0289,-0.0337,-0.0385,-0.0431,-0.0476,-0.052000000000000005,-0.0564,-0.0606,-0.0648,0.0567,-0.0147,-0.0804,-0.1409,-0.1966,-0.248,-0.299,-0.3461,-0.3895,-0.4297,-0.4669,-0.5013,-0.5361,-0.5684,-0.5983,-0.6261,-0.6518,-0.6758,-0.6998,-0.7222,-0.743,-0.7623,-0.7803,-0.7971,-0.8162,-0.8339,-0.8503,-0.8655,-0.8797,-0.893,-0.9106,-0.9264,-0.9407,-0.9536,-0.9653,-0.9757,-0.9753,-0.9743,-0.973,-0.9714,-0.9694,-0.9672,-0.9576,-0.9481,-0.9388,-0.9298,-0.9209999999999999,-0.9124,-0.8993,-0.8867,-0.8745,-0.8628,-0.8514,-0.8404,-0.8208,-0.802,-0.7841,-0.7669,-0.7503,-0.7344,-0.7076,-0.682,-0.6574,-0.6339,-0.6113,-0.5896,-0.5621,-0.5357,-0.5103,-0.486,-0.4627,-0.4402,-0.41600000000000004,-0.3928,-0.3705,-0.3491,-0.3284,-0.3085,-0.2869,-0.2661,-0.2461,-0.2267,-0.2081,-0.1902,-0.17,-0.1505,-0.1317,-0.1134,-0.0958,-0.0788,-0.0612,-0.0441,-0.0275,-0.0113,0.0043,0.0195,0.0333,0.0467,0.0599,0.0726,0.0851,0.0973,0.1071,0.1168,0.1263,0.1356,0.1448,0.1538,0.1618,0.1698,0.1777,0.1855,0.1932,0.2009,0.2081,0.2154,0.2226,0.2299,0.2372,0.2445,0.2503,0.2562,0.2622,0.2683,0.2745,0.2808,0.2841,0.2876,0.2911,0.2948,0.2986,0.3026,0.3023,0.302,0.3019,0.3019,0.3019,0.3021,0.2993,0.2965,0.2938,0.2911,0.2885,0.2859,0.2818,0.2777,0.2736,0.2695,0.2654,0.2613,0.2514,0.2412,0.231,0.2207,0.2102,0.1996,0.1829,0.166,0.1489,0.1316,0.1141,0.0964,0.0862,0.0759,0.0655,0.055,0.0444,0.0337,0.0282,0.0226,0.0169,0.0113,0.0055,-0.0003,-0.0068,-0.0134,-0.02,-0.0267,-0.0335,-0.0403,-0.0469,-0.0535,-0.0602,-0.0669,-0.0738,-0.0807,-0.0859,-0.0911,-0.0965,-0.1018,-0.1072,-0.1127,],
"M":[3.3464,4.4709,5.5675,6.3762,7.0023,7.5105,7.934,8.297,8.6151,8.9014,9.1649,9.4122,9.6479,9.8749,10.0953,10.3108,10.5228,10.7319,10.9385,11.142999999999999,11.3462,11.5486,11.7504,11.9514,12.1515,12.3502,12.5466,12.7401,12.9303,13.1169,13.3,13.4798,13.6567,13.8309,14.0031,14.1736,14.7381,14.911,15.0839,15.2569,15.4299,15.603,15.7775,15.9521,16.1268,16.3015,16.4763,16.6512,16.8276,17.0041,17.1807,17.3573,17.534000000000002,17.7108,17.8888,18.067,18.2452,18.4235,18.6018,18.7802,18.9625,19.1449,19.3274,19.51,19.6926,19.8753,20.0814,20.2877,20.4941,20.7007,20.9073,21.1141,21.3417,21.5693,21.796999999999997,22.0247,22.2524,22.4802,22.7267,22.9732,23.2196,23.4661,23.7126,23.9591,24.2214,24.4837,24.7461,25.0084,25.2707,25.5329,25.8155,26.0979,26.3804,26.6629,26.9453,27.2277,27.5321,27.8365,28.1409,28.4453,28.7497,29.054000000000002,29.3761,29.6982,30.0203,30.3424,30.6645,30.9867,31.3251,31.6635,32.0019,32.3404,32.6788,33.0173,33.3742,33.7312,34.0882,34.4452,34.8022,35.1592,35.5349,35.9105,36.2862,36.6619,37.0376,37.4134,37.8103,38.2074,38.6045,39.0017,39.3989,39.7961,40.2146,40.6331,41.0518,41.4706,41.8894,42.3083,42.7515,43.1947,43.638000000000005,44.0813,44.5248,44.9684,45.4253,45.8824,46.3396,46.7968,47.2542,47.7116,48.1693,48.6271,49.085,49.5431,50.0012,50.4595,50.9021,51.3448,51.7875,52.2304,52.6733,53.1164,53.5329,53.9495,54.36600000000001,54.7826,55.1993,55.6159,55.9927,56.3694,56.7462,57.1229,57.4995,57.8762,58.2029,58.5296,58.8563,59.1829,59.5096,59.8362,60.1106,60.385,60.6594,60.9338,61.2081,61.4825,61.7129,61.9434,62.1739,62.4045,62.6351,62.8658,63.055,63.2442,63.4335,63.6229,63.8123,64.0018,64.1526,64.3033,64.4539,64.6045,64.7551,64.9057,65.0469,65.188,65.3291,65.4702,65.6113,65.7523,65.8907,66.0291,66.1674,66.3057,66.4439,66.5821,66.7186,66.8551,66.9915,67.1278,67.2641,67.4004,67.5362,67.6719,67.8077,67.9434,68.079,68.2146,],
"S":[0.146,0.134,0.1239,0.1173,0.1132,0.1108,0.1096,0.109,0.1088,0.1088,0.1089,0.1091,0.1093,0.1095,0.1098,0.1101,0.1104,0.1108,0.1112,0.1116,0.1121,0.1126,0.1131,0.1137,0.1143,0.1149,0.1154,0.11599999999999999,0.1166,0.1172,0.1178,0.1184,0.11900000000000001,0.1195,0.1201,0.1206,0.0969,0.0981,0.0993,0.1005,0.1016,0.1027,0.1038,0.1049,0.106,0.107,0.10800000000000001,0.1089,0.1099,0.1108,0.1118,0.1127,0.1136,0.1144,0.1153,0.1161,0.11699999999999999,0.1178,0.1186,0.1194,0.1202,0.1211,0.1219,0.1227,0.1235,0.1242,0.1254,0.1265,0.1275,0.1286,0.1296,0.1306,0.1318,0.133,0.1342,0.1353,0.1364,0.1375,0.1388,0.14,0.1412,0.1424,0.1435,0.1447,0.146,0.1474,0.1486,0.1499,0.1511,0.1524,0.1537,0.155,0.1563,0.1576,0.1588,0.16,0.1613,0.1625,0.1638,0.165,0.1661,0.1673,0.1685,0.1698,0.171,0.1721,0.1733,0.1744,0.1756,0.1768,0.1779,0.179,0.1801,0.1812,0.1823,0.1833,0.1844,0.1854,0.1864,0.1873,0.1882,0.18899999999999997,0.1899,0.1907,0.1914,0.1922,0.1929,0.1936,0.1942,0.1949,0.1955,0.1961,0.1966,0.1971,0.1976,0.1981,0.1986,0.1991,0.1992,0.1994,0.1996,0.1997,0.1999,0.2,0.1998,0.1995,0.1993,0.1991,0.1988,0.1986,0.198,0.1974,0.1969,0.1963,0.1957,0.1952,0.1942,0.1932,0.1923,0.1914,0.1904,0.1895,0.1882,0.1869,0.1856,0.1843,0.183,0.1818,0.1802,0.1787,0.1772,0.1757,0.1742,0.1727,0.1711,0.1696,0.168,0.1665,0.165,0.1635,0.162,0.1606,0.1591,0.1577,0.1563,0.1549,0.1537,0.1525,0.1513,0.1501,0.1489,0.1477,0.1468,0.1459,0.145,0.1441,0.1432,0.1424,0.1418,0.1413,0.1407,0.1402,0.1397,0.1391,0.1387,0.1382,0.1377,0.1373,0.1368,0.1364,0.1359,0.1355,0.1351,0.1346,0.1342,0.1338,0.1334,0.133,0.1326,0.1322,0.1318,0.1314,0.1311,0.1307,0.1303,0.1299,0.1296,0.1292,],
"1p":[2.3,3.2,4.1,4.8,5.4,5.8,6.1,6.4,6.7,6.9,7.1,7.3,7.5,7.6,7.8,8.0,8.1,8.3,8.4,8.6,8.7,8.9,9.0,9.2,9.3,9.5,9.6,9.7,9.9,10.0,10.1,10.3,10.4,10.5,10.6,10.7,11.7,11.9,12.0,12.1,12.2,12.4,12.5,12.6,12.7,12.9,13.0,13.1,13.2,13.4,13.5,13.6,13.7,13.9,14.0,14.1,14.2,14.4,14.5,14.6,14.7,14.9,15.0,15.1,15.2,15.4,15.5,15.6,15.8,15.9,16.0,16.2,16.3,16.5,16.6,16.7,16.9,17.0,17.2,17.3,17.4,17.6,17.7,17.9,18.0,18.2,18.3,18.4,18.6,18.7,18.9,19.0,19.2,19.3,19.4,19.6,19.7,19.9,20.0,20.2,20.3,20.5,20.6,20.8,20.9,21.0,21.2,21.3,21.5,21.6,21.8,21.9,22.1,22.2,22.4,22.5,22.7,22.8,23.0,23.1,23.3,23.5,23.6,23.8,23.9,24.1,24.3,24.5,24.6,24.8,25.0,25.2,25.4,25.6,25.8,26.0,26.1,26.3,26.6,26.8,27.0,27.3,27.5,27.7,28.0,28.3,28.6,28.8,29.1,29.4,29.7,30.0,30.3,30.6,30.9,31.2,31.5,31.8,32.2,32.5,32.8,33.2,33.5,33.9,34.3,34.6,35.0,35.4,35.8,36.2,36.5,36.9,37.3,37.7,38.1,38.5,38.9,39.2,39.6,40.0,40.4,40.7,41.1,41.4,41.8,42.1,42.4,42.8,43.1,43.4,43.7,44.0,44.3,44.6,44.9,45.2,45.4,45.7,45.9,46.1,46.3,46.5,46.7,46.9,47.0,47.2,47.4,47.5,47.7,47.9,48.0,48.2,48.4,48.5,48.7,48.9,49.0,49.2,49.3,49.5,49.7,49.8,50.0,50.1,50.3,50.4,50.6,50.8,],
"3p":[2.5,3.4,4.4,5.1,5.6,6.1,6.4,6.7,7.0,7.2,7.5,7.7,7.8,8.0,8.2,8.4,8.5,8.7,8.9,9.0,9.2,9.3,9.5,9.7,9.8,10.0,10.1,10.2,10.4,10.5,10.7,10.8,10.9,11.1,11.2,11.3,12.3,12.4,12.5,12.7,12.8,12.9,13.0,13.2,13.3,13.4,13.6,13.7,13.8,14.0,14.1,14.2,14.4,14.5,14.6,14.7,14.9,15.0,15.1,15.3,15.4,15.5,15.7,15.8,15.9,16.1,16.2,16.4,16.5,16.7,16.8,16.9,17.1,17.2,17.4,17.5,17.7,17.8,18.0,18.2,18.3,18.5,18.6,18.8,18.9,19.1,19.3,19.4,19.6,19.8,19.9,20.1,20.2,20.4,20.6,20.7,20.9,21.1,21.3,21.4,21.6,21.8,21.9,22.1,22.3,22.5,22.6,22.8,23.0,23.2,23.3,23.5,23.7,23.9,24.1,24.3,24.4,24.6,24.8,25.0,25.2,25.4,25.6,25.8,26.0,26.2,26.4,26.6,26.8,27.1,27.3,27.5,27.7,28.0,28.2,28.4,28.7,28.9,29.2,29.4,29.7,30.0,30.2,30.5,30.8,31.1,31.4,31.7,32.1,32.4,32.7,33.0,33.4,33.7,34.0,34.3,34.7,35.1,35.4,35.8,36.1,36.5,36.9,37.2,37.6,38.0,38.4,38.8,39.2,39.6,39.9,40.3,40.7,41.1,41.5,41.9,42.2,42.6,43.0,43.4,43.7,44.0,44.4,44.7,45.1,45.4,45.7,46.0,46.3,46.6,46.9,47.2,47.5,47.8,48.0,48.3,48.5,48.8,49.0,49.2,49.3,49.5,49.7,49.9,50.1,50.2,50.4,50.6,50.7,50.9,51.0,51.2,51.4,51.5,51.7,51.8,52.0,52.1,52.3,52.5,52.6,52.8,52.9,53.1,53.2,53.4,53.5,53.7,],
"5p":[2.6,3.6,4.5,5.2,5.8,6.2,6.6,6.9,7.2,7.4,7.7,7.9,8.1,8.2,8.4,8.6,8.8,8.9,9.1,9.3,9.4,9.6,9.8,9.9,10.1,10.2,10.4,10.5,10.7,10.8,11.0,11.1,11.2,11.4,11.5,11.6,12.6,12.7,12.8,13.0,13.1,13.2,13.4,13.5,13.6,13.8,13.9,14.0,14.2,14.3,14.4,14.6,14.7,14.8,15.0,15.1,15.2,15.4,15.5,15.6,15.8,15.9,16.1,16.2,16.3,16.5,16.6,16.8,16.9,17.1,17.2,17.4,17.5,17.7,17.8,18.0,18.2,18.3,18.5,18.7,18.8,19.0,19.2,19.3,19.5,19.7,19.8,20.0,20.2,20.3,20.5,20.7,20.9,21.0,21.2,21.4,21.6,21.8,22.0,22.1,22.3,22.5,22.7,22.9,23.1,23.3,23.5,23.7,23.8,24.0,24.2,24.4,24.6,24.8,25.0,25.2,25.4,25.6,25.9,26.1,26.3,26.5,26.7,26.9,27.2,27.4,27.6,27.9,28.1,28.3,28.6,28.8,29.1,29.3,29.6,29.8,30.1,30.3,30.6,30.9,31.2,31.5,31.8,32.1,32.4,32.7,33.1,33.4,33.7,34.0,34.4,34.7,35.1,35.4,35.8,36.1,36.5,36.9,37.2,37.6,38.0,38.3,38.7,39.1,39.5,39.9,40.3,40.7,41.0,41.4,41.8,42.2,42.6,43.0,43.4,43.7,44.1,44.5,44.9,45.2,45.6,45.9,46.2,46.6,46.9,47.2,47.5,47.8,48.1,48.4,48.7,49.0,49.3,49.5,49.8,50.0,50.3,50.5,50.7,50.9,51.0,51.2,51.4,51.6,51.7,51.9,52.1,52.2,52.4,52.5,52.7,52.9,53.0,53.2,53.3,53.5,53.6,53.8,53.9,54.1,54.2,54.4,54.5,54.7,54.8,55.0,55.1,55.3,],
"10p":[2.8,3.8,4.7,5.5,6.0,6.5,6.9,7.2,7.5,7.7,8.0,8.2,8.4,8.6,8.8,9.0,9.1,9.3,9.5,9.7,9.8,10.0,10.2,10.3,10.5,10.7,10.8,11.0,11.1,11.3,11.4,11.6,11.7,11.9,12.0,12.2,13.0,13.2,13.3,13.4,13.6,13.7,13.8,14.0,14.1,14.3,14.4,14.5,14.7,14.8,15.0,15.1,15.3,15.4,15.5,15.7,15.8,16.0,16.1,16.3,16.4,16.5,16.7,16.8,17.0,17.1,17.3,17.4,17.6,17.8,17.9,18.1,18.3,18.4,18.6,18.8,18.9,19.1,19.3,19.5,19.6,19.8,20.0,20.2,20.4,20.6,20.7,20.9,21.1,21.3,21.5,21.7,21.9,22.1,22.3,22.5,22.7,22.9,23.1,23.3,23.5,23.7,24.0,24.2,24.4,24.6,24.8,25.0,25.3,25.5,25.7,25.9,26.2,26.4,26.6,26.9,27.1,27.3,27.6,27.8,28.1,28.3,28.6,28.8,29.1,29.3,29.6,29.9,30.1,30.4,30.7,30.9,31.2,31.5,31.8,32.1,32.4,32.7,33.0,33.3,33.6,34.0,34.3,34.6,35.0,35.3,35.7,36.0,36.4,36.7,37.1,37.5,37.9,38.2,38.6,39.0,39.4,39.8,40.1,40.5,40.9,41.3,41.7,42.1,42.5,42.9,43.3,43.7,44.1,44.5,44.9,45.2,45.6,46.0,46.4,46.8,47.1,47.5,47.8,48.2,48.5,48.9,49.2,49.5,49.8,50.1,50.4,50.7,51.0,51.3,51.6,51.8,52.1,52.3,52.5,52.8,53.0,53.2,53.4,53.6,53.8,53.9,54.1,54.3,54.4,54.6,54.7,54.9,55.1,55.2,55.4,55.5,55.7,55.8,56.0,56.1,56.3,56.4,56.6,56.7,56.9,57.0,57.2,57.3,57.5,57.6,57.7,57.9,],
"15p":[2.9,3.9,4.9,5.6,6.2,6.7,7.1,7.4,7.7,7.9,8.2,8.4,8.6,8.8,9.0,9.2,9.4,9.6,9.7,9.9,10.1,10.3,10.5,10.6,10.8,11.0,11.1,11.3,11.5,11.6,11.8,11.9,12.1,12.2,12.4,12.5,13.3,13.5,13.6,13.8,13.9,14.0,14.2,14.3,14.5,14.6,14.8,14.9,15.1,15.2,15.4,15.5,15.7,15.8,15.9,16.1,16.2,16.4,16.5,16.7,16.8,17.0,17.1,17.3,17.4,17.6,17.8,17.9,18.1,18.3,18.4,18.6,18.8,19.0,19.1,19.3,19.5,19.7,19.9,20.1,20.2,20.4,20.6,20.8,21.0,21.2,21.4,21.6,21.8,22.0,22.2,22.4,22.6,22.9,23.1,23.3,23.5,23.7,24.0,24.2,24.4,24.6,24.9,25.1,25.3,25.6,25.8,26.0,26.3,26.5,26.8,27.0,27.3,27.5,27.8,28.0,28.3,28.5,28.8,29.1,29.3,29.6,29.9,30.2,30.4,30.7,31.0,31.3,31.6,31.9,32.2,32.5,32.8,33.1,33.4,33.7,34.0,34.3,34.7,35.0,35.4,35.7,36.1,36.4,36.8,37.2,37.5,37.9,38.3,38.7,39.1,39.4,39.8,40.2,40.6,41.0,41.4,41.8,42.2,42.6,43.0,43.4,43.8,44.2,44.6,45.0,45.4,45.8,46.2,46.6,47.0,47.4,47.8,48.1,48.5,48.9,49.2,49.6,49.9,50.3,50.6,50.9,51.2,51.6,51.9,52.2,52.5,52.7,53.0,53.3,53.5,53.8,54.0,54.3,54.5,54.7,54.9,55.2,55.3,55.5,55.7,55.8,56.0,56.2,56.3,56.5,56.6,56.8,56.9,57.1,57.2,57.4,57.5,57.7,57.8,58.0,58.1,58.3,58.4,58.6,58.7,58.9,59.0,59.1,59.3,59.4,59.6,59.7,],
"25p":[3.0,4.1,5.1,5.9,6.5,7.0,7.4,7.7,8.0,8.3,8.5,8.7,9.0,9.2,9.4,9.6,9.8,10.0,10.1,10.3,10.5,10.7,10.9,11.1,11.3,11.4,11.6,11.8,12.0,12.1,12.3,12.4,12.6,12.8,12.9,13.1,13.8,14.0,14.1,14.3,14.4,14.6,14.7,14.9,15.0,15.2,15.3,15.5,15.6,15.8,16.0,16.1,16.3,16.4,16.6,16.7,16.9,17.1,17.2,17.4,17.5,17.7,17.9,18.0,18.2,18.3,18.5,18.7,18.9,19.0,19.2,19.4,19.6,19.8,20.0,20.2,20.4,20.6,20.8,21.0,21.2,21.4,21.6,21.8,22.0,22.3,22.5,22.7,22.9,23.1,23.4,23.6,23.8,24.1,24.3,24.5,24.8,25.0,25.3,25.5,25.8,26.0,26.3,26.6,26.8,27.1,27.4,27.6,27.9,28.2,28.5,28.7,29.0,29.3,29.6,29.9,30.2,30.4,30.7,31.0,31.3,31.6,32.0,32.3,32.6,32.9,33.2,33.5,33.9,34.2,34.5,34.9,35.2,35.6,35.9,36.3,36.6,37.0,37.3,37.7,38.1,38.5,38.9,39.2,39.6,40.0,40.4,40.8,41.3,41.7,42.1,42.5,42.9,43.3,43.7,44.1,44.6,45.0,45.4,45.8,46.2,46.6,47.0,47.4,47.9,48.3,48.7,49.1,49.5,49.9,50.2,50.6,51.0,51.4,51.8,52.1,52.4,52.8,53.1,53.5,53.8,54.1,54.4,54.7,55.0,55.3,55.6,55.8,56.1,56.3,56.6,56.8,57.1,57.3,57.5,57.7,57.9,58.1,58.3,58.4,58.6,58.8,58.9,59.1,59.2,59.4,59.5,59.7,59.8,60.0,60.1,60.3,60.4,60.6,60.7,60.8,61.0,61.1,61.3,61.4,61.6,61.7,61.8,62.0,62.1,62.3,62.4,62.5,],
"50p":[3.3,4.5,5.6,6.4,7.0,7.5,7.9,8.3,8.6,8.9,9.2,9.4,9.6,9.9,10.1,10.3,10.5,10.7,10.9,11.1,11.3,11.5,11.8,12.0,12.2,12.4,12.5,12.7,12.9,13.1,13.3,13.5,13.7,13.8,14.0,14.2,14.7,14.9,15.1,15.3,15.4,15.6,15.8,16.0,16.1,16.3,16.5,16.7,16.8,17.0,17.2,17.4,17.5,17.7,17.9,18.1,18.2,18.4,18.6,18.8,19.0,19.1,19.3,19.5,19.7,19.9,20.1,20.3,20.5,20.7,20.9,21.1,21.3,21.6,21.8,22.0,22.3,22.5,22.7,23.0,23.2,23.5,23.7,24.0,24.2,24.5,24.7,25.0,25.3,25.5,25.8,26.1,26.4,26.7,26.9,27.2,27.5,27.8,28.1,28.4,28.7,29.1,29.4,29.7,30.0,30.3,30.7,31.0,31.3,31.7,32.0,32.3,32.7,33.0,33.4,33.7,34.1,34.4,34.8,35.2,35.5,35.9,36.3,36.7,37.0,37.4,37.8,38.2,38.6,39.0,39.4,39.8,40.2,40.6,41.1,41.5,41.9,42.3,42.8,43.2,43.6,44.1,44.5,45.0,45.4,45.9,46.3,46.8,47.3,47.7,48.2,48.6,49.1,49.5,50.0,50.5,50.9,51.3,51.8,52.2,52.7,53.1,53.5,53.9,54.4,54.8,55.2,55.6,56.0,56.4,56.7,57.1,57.5,57.9,58.2,58.5,58.9,59.2,59.5,59.8,60.1,60.4,60.7,60.9,61.2,61.5,61.7,61.9,62.2,62.4,62.6,62.9,63.1,63.2,63.4,63.6,63.8,64.0,64.2,64.3,64.5,64.6,64.8,64.9,65.0,65.2,65.3,65.5,65.6,65.8,65.9,66.0,66.2,66.3,66.4,66.6,66.7,66.9,67.0,67.1,67.3,67.4,67.5,67.7,67.8,67.9,68.1,68.2,],
"75p":[3.7,4.9,6.0,6.9,7.6,8.1,8.5,8.9,9.3,9.6,9.9,10.1,10.4,10.6,10.9,11.1,11.3,11.6,11.8,12.0,12.2,12.5,12.7,12.9,13.1,13.3,13.6,13.8,14.0,14.2,14.4,14.6,14.8,15.0,15.2,15.4,15.7,15.9,16.1,16.3,16.5,16.7,16.9,17.1,17.3,17.5,17.7,17.9,18.1,18.4,18.6,18.8,19.0,19.2,19.4,19.6,19.8,20.0,20.2,20.4,20.6,20.8,21.0,21.3,21.5,21.7,21.9,22.2,22.4,22.7,22.9,23.2,23.4,23.7,24.0,24.2,24.5,24.8,25.1,25.4,25.7,25.9,26.2,26.5,26.9,27.2,27.5,27.8,28.1,28.4,28.8,29.1,29.4,29.8,30.1,30.5,30.8,31.2,31.6,31.9,32.3,32.7,33.0,33.4,33.8,34.2,34.6,35.0,35.4,35.8,36.2,36.6,37.0,37.4,37.8,38.3,38.7,39.1,39.5,40.0,40.4,40.8,41.3,41.7,42.2,42.6,43.1,43.6,44.0,44.5,44.9,45.4,45.9,46.4,46.9,47.4,47.9,48.3,48.9,49.4,49.9,50.4,50.9,51.4,51.9,52.4,52.9,53.4,53.9,54.5,55.0,55.4,55.9,56.4,56.9,57.4,57.9,58.4,58.8,59.3,59.8,60.2,60.6,61.1,61.5,61.9,62.3,62.7,63.1,63.5,63.8,64.2,64.5,64.9,65.2,65.5,65.8,66.1,66.4,66.7,66.9,67.2,67.4,67.7,67.9,68.2,68.4,68.6,68.8,69.0,69.2,69.4,69.6,69.7,69.9,70.1,70.2,70.4,70.6,70.7,70.9,71.0,71.1,71.3,71.4,71.5,71.7,71.8,72.0,72.1,72.2,72.4,72.5,72.6,72.7,72.9,73.0,73.1,73.3,73.4,73.5,73.7,73.8,73.9,74.1,74.2,74.3,74.5,],
"85p":[3.9,5.1,6.3,7.2,7.9,8.4,8.9,9.3,9.6,10.0,10.3,10.5,10.8,11.1,11.3,11.6,11.8,12.0,12.3,12.5,12.7,13.0,13.2,13.4,13.7,13.9,14.1,14.4,14.6,14.8,15.0,15.2,15.5,15.7,15.9,16.1,16.3,16.5,16.7,16.9,17.2,17.4,17.6,17.8,18.0,18.3,18.5,18.7,18.9,19.1,19.4,19.6,19.8,20.0,20.3,20.5,20.7,20.9,21.2,21.4,21.6,21.9,22.1,22.3,22.6,22.8,23.1,23.3,23.6,23.9,24.1,24.4,24.7,25.0,25.3,25.6,25.9,26.2,26.5,26.9,27.2,27.5,27.8,28.2,28.5,28.9,29.2,29.5,29.9,30.2,30.6,31.0,31.4,31.7,32.1,32.5,32.9,33.3,33.7,34.1,34.5,34.9,35.3,35.7,36.1,36.6,37.0,37.4,37.9,38.3,38.7,39.2,39.6,40.1,40.5,41.0,41.5,41.9,42.4,42.9,43.3,43.8,44.3,44.8,45.3,45.7,46.2,46.7,47.2,47.7,48.2,48.7,49.3,49.8,50.3,50.8,51.4,51.9,52.4,53.0,53.5,54.1,54.6,55.1,55.7,56.2,56.8,57.3,57.8,58.4,58.9,59.4,59.9,60.4,61.0,61.5,61.9,62.4,62.9,63.4,63.8,64.3,64.7,65.1,65.6,66.0,66.4,66.8,67.1,67.5,67.8,68.2,68.5,68.9,69.2,69.5,69.8,70.0,70.3,70.6,70.8,71.1,71.3,71.5,71.7,72.0,72.1,72.3,72.5,72.7,72.9,73.1,73.3,73.4,73.6,73.8,73.9,74.1,74.2,74.4,74.5,74.7,74.8,74.9,75.1,75.2,75.3,75.5,75.6,75.7,75.9,76.0,76.1,76.3,76.4,76.5,76.6,76.8,76.9,77.0,77.2,77.3,77.4,77.6,77.7,77.8,77.9,78.1,],
"90p":[4.0,5.3,6.5,7.4,8.1,8.6,9.1,9.5,9.9,10.2,10.5,10.8,11.1,11.4,11.6,11.9,12.1,12.4,12.6,12.9,13.1,13.3,13.6,13.8,14.1,14.3,14.6,14.8,15.0,15.2,15.5,15.7,15.9,16.1,16.3,16.6,16.7,16.9,17.1,17.4,17.6,17.8,18.1,18.3,18.5,18.8,19.0,19.2,19.5,19.7,20.0,20.2,20.4,20.7,20.9,21.1,21.4,21.6,21.9,22.1,22.4,22.6,22.9,23.1,23.4,23.6,23.9,24.2,24.5,24.8,25.1,25.3,25.7,26.0,26.3,26.6,27.0,27.3,27.6,28.0,28.3,28.7,29.0,29.4,29.7,30.1,30.5,30.9,31.2,31.6,32.0,32.4,32.8,33.2,33.6,34.0,34.4,34.8,35.3,35.7,36.1,36.5,37.0,37.4,37.9,38.3,38.8,39.2,39.7,40.1,40.6,41.1,41.5,42.0,42.5,43.0,43.5,44.0,44.5,45.0,45.5,46.0,46.5,47.0,47.5,48.0,48.5,49.0,49.6,50.1,50.6,51.1,51.7,52.2,52.8,53.3,53.9,54.4,55.0,55.6,56.1,56.7,57.3,57.8,58.4,58.9,59.5,60.0,60.6,61.2,61.7,62.2,62.7,63.3,63.8,64.3,64.8,65.3,65.8,66.2,66.7,67.2,67.6,68.0,68.4,68.8,69.2,69.7,70.0,70.3,70.7,71.0,71.4,71.7,72.0,72.3,72.5,72.8,73.1,73.3,73.6,73.8,74.0,74.2,74.4,74.6,74.8,75.0,75.2,75.3,75.5,75.7,75.9,76.0,76.2,76.4,76.5,76.7,76.8,77.0,77.1,77.3,77.4,77.5,77.7,77.8,77.9,78.0,78.2,78.3,78.4,78.6,78.7,78.8,79.0,79.1,79.2,79.3,79.5,79.6,79.7,79.9,80.0,80.1,80.2,80.4,80.5,80.6,],
"95p":[4.2,5.5,6.8,7.7,8.4,9.0,9.5,9.9,10.3,10.6,10.9,11.2,11.5,11.8,12.1,12.3,12.6,12.9,13.1,13.4,13.6,13.9,14.2,14.4,14.7,14.9,15.2,15.4,15.7,15.9,16.2,16.4,16.6,16.9,17.1,17.3,17.3,17.5,17.8,18.0,18.3,18.5,18.8,19.1,19.3,19.6,19.8,20.1,20.4,20.6,20.9,21.1,21.4,21.7,21.9,22.2,22.5,22.7,23.0,23.3,23.5,23.8,24.1,24.4,24.6,24.9,25.2,25.6,25.9,26.2,26.5,26.9,27.2,27.6,27.9,28.3,28.7,29.0,29.4,29.8,30.2,30.6,30.9,31.3,31.7,32.2,32.6,33.0,33.4,33.8,34.3,34.7,35.2,35.6,36.0,36.5,36.9,37.4,37.9,38.3,38.8,39.2,39.7,40.2,40.7,41.1,41.6,42.1,42.6,43.1,43.6,44.1,44.6,45.1,45.7,46.2,46.7,47.2,47.8,48.3,48.8,49.4,49.9,50.5,51.0,51.5,52.1,52.7,53.2,53.8,54.3,54.9,55.5,56.1,56.6,57.2,57.8,58.4,59.0,59.6,60.2,60.8,61.4,62.0,62.6,63.1,63.7,64.3,64.9,65.5,66.0,66.6,67.1,67.6,68.2,68.7,69.2,69.7,70.2,70.7,71.1,71.6,72.0,72.4,72.8,73.2,73.6,74.0,74.4,74.7,75.0,75.4,75.7,76.0,76.3,76.5,76.8,77.0,77.3,77.5,77.7,77.9,78.1,78.3,78.5,78.7,78.9,79.0,79.2,79.4,79.5,79.7,79.9,80.0,80.2,80.4,80.5,80.7,80.8,81.0,81.1,81.2,81.4,81.5,81.7,81.8,81.9,82.0,82.2,82.3,82.4,82.5,82.7,82.8,82.9,83.1,83.2,83.3,83.4,83.6,83.7,83.8,84.0,84.1,84.2,84.3,84.5,84.6,],
"97p":[4.3,5.7,7.0,7.9,8.6,9.2,9.7,10.2,10.5,10.9,11.2,11.5,11.8,12.1,12.4,12.7,12.9,13.2,13.5,13.7,14.0,14.3,14.5,14.8,15.1,15.3,15.6,15.9,16.1,16.4,16.6,16.9,17.1,17.3,17.6,17.8,17.7,17.9,18.2,18.5,18.7,19.0,19.3,19.6,19.8,20.1,20.4,20.7,20.9,21.2,21.5,21.8,22.1,22.3,22.6,22.9,23.2,23.5,23.8,24.1,24.3,24.6,24.9,25.2,25.5,25.8,26.2,26.5,26.9,27.3,27.6,28.0,28.3,28.7,29.1,29.5,29.9,30.3,30.7,31.1,31.5,31.9,32.3,32.8,33.2,33.7,34.1,34.5,35.0,35.4,35.9,36.4,36.8,37.3,37.8,38.3,38.7,39.2,39.7,40.2,40.7,41.1,41.6,42.2,42.7,43.2,43.7,44.2,44.7,45.2,45.7,46.3,46.8,47.3,47.9,48.4,49.0,49.5,50.1,50.6,51.2,51.8,52.3,52.9,53.4,54.0,54.6,55.1,55.7,56.3,56.9,57.5,58.1,58.7,59.3,59.9,60.5,61.1,61.7,62.4,63.0,63.6,64.2,64.8,65.4,66.0,66.6,67.2,67.8,68.4,69.0,69.5,70.1,70.6,71.2,71.7,72.2,72.7,73.2,73.6,74.1,74.6,75.0,75.4,75.8,76.2,76.6,77.0,77.3,77.6,77.9,78.3,78.6,78.9,79.1,79.4,79.6,79.9,80.1,80.3,80.5,80.7,80.9,81.1,81.2,81.4,81.6,81.7,81.9,82.1,82.2,82.4,82.5,82.7,82.9,83.0,83.2,83.4,83.5,83.7,83.8,83.9,84.1,84.2,84.3,84.5,84.6,84.7,84.8,85.0,85.1,85.2,85.4,85.5,85.6,85.7,85.9,86.0,86.1,86.3,86.4,86.5,86.6,86.8,86.9,87.0,87.1,87.3,],
"99p":[4.6,6.0,7.4,8.3,9.1,9.7,10.2,10.7,11.1,11.4,11.8,12.1,12.4,12.7,13.0,13.3,13.6,13.9,14.2,14.4,14.7,15.0,15.3,15.6,15.9,16.1,16.4,16.7,17.0,17.3,17.5,17.8,18.0,18.3,18.6,18.8,18.4,18.7,19.0,19.4,19.7,20.0,20.3,20.6,20.9,21.2,21.5,21.8,22.2,22.5,22.8,23.1,23.4,23.8,24.1,24.4,24.7,25.1,25.4,25.7,26.0,26.4,26.7,27.1,27.4,27.8,28.2,28.6,29.0,29.4,29.9,30.3,30.7,31.2,31.6,32.1,32.5,33.0,33.4,33.9,34.4,34.9,35.3,35.8,36.3,36.8,37.4,37.9,38.4,38.9,39.4,39.9,40.5,41.0,41.5,42.1,42.6,43.1,43.6,44.2,44.7,45.2,45.7,46.3,46.8,47.4,47.9,48.4,49.0,49.6,50.2,50.7,51.3,51.9,52.5,53.0,53.6,54.2,54.8,55.4,56.0,56.6,57.2,57.8,58.4,59.0,59.6,60.2,60.8,61.4,62.1,62.7,63.3,64.0,64.6,65.3,65.9,66.5,67.2,67.9,68.5,69.2,69.8,70.5,71.1,71.7,72.4,73.0,73.6,74.2,74.8,75.4,75.9,76.5,77.0,77.6,78.1,78.6,79.1,79.5,80.0,80.5,80.9,81.3,81.6,82.0,82.4,82.8,83.1,83.4,83.7,84.0,84.3,84.6,84.8,85.0,85.2,85.4,85.6,85.9,86.0,86.2,86.3,86.5,86.6,86.8,86.9,87.1,87.2,87.4,87.5,87.6,87.8,88.0,88.2,88.3,88.5,88.7,88.8,89.0,89.1,89.3,89.4,89.6,89.7,89.8,89.9,90.1,90.2,90.3,90.4,90.6,90.7,90.8,90.9,91.1,91.2,91.3,91.5,91.6,91.7,91.9,92.0,92.1,92.2,92.4,92.5,92.6,],
"[-3SD]":[2.1,2.9,3.8,4.4,4.9,5.3,5.7,5.9,6.2,6.4,6.6,6.8,6.9,7.1,7.2,7.4,7.5,7.7,7.8,8.0,8.1,8.2,8.4,8.5,8.6,8.8,8.9,9.0,9.1,9.2,9.4,9.5,9.6,9.7,9.8,9.9,11.0,11.1,11.2,11.4,11.5,11.6,11.7,11.8,12.0,12.1,12.2,12.3,12.4,12.5,12.7,12.8,12.9,13.0,13.1,13.2,13.4,13.5,13.6,13.7,13.8,13.9,14.1,14.2,14.3,14.4,14.5,14.7,14.8,14.9,15.0,15.2,15.3,15.4,15.5,15.6,15.8,15.9,16.0,16.1,16.3,16.4,16.5,16.6,16.7,16.9,17.0,17.1,17.2,17.3,17.5,17.6,17.7,17.8,17.9,18.0,18.2,18.3,18.4,18.5,18.6,18.7,18.8,18.9,19.0,19.1,19.2,19.4,19.5,19.6,19.7,19.8,19.9,20.0,20.1,20.2,20.3,20.4,20.5,20.6,20.7,20.8,21.0,21.1,21.2,21.3,21.4,21.5,21.7,21.8,21.9,22.0,22.2,22.3,22.4,22.6,22.7,22.9,23.1,23.2,23.4,23.6,23.8,24.0,24.2,24.4,24.6,24.8,25.1,25.3,25.5,25.8,26.1,26.3,26.6,26.8,27.1,27.4,27.7,28.0,28.3,28.6,28.9,29.3,29.6,30.0,30.3,30.6,31.0,31.4,31.8,32.2,32.6,32.9,33.3,33.7,34.1,34.5,34.9,35.3,35.6,36.0,36.4,36.7,37.1,37.5,37.8,38.1,38.5,38.8,39.2,39.5,39.8,40.1,40.5,40.8,41.1,41.4,41.6,41.8,42.0,42.2,42.4,42.6,42.8,43.0,43.2,43.3,43.5,43.7,43.9,44.0,44.2,44.4,44.5,44.7,44.9,45.0,45.2,45.4,45.6,45.7,45.9,46.0,46.2,46.4,46.5,46.7,],
"[-2SD]":[2.5,3.4,4.3,5.0,5.6,6.0,6.4,6.7,6.9,7.1,7.4,7.6,7.7,7.9,8.1,8.3,8.4,8.6,8.8,8.9,9.1,9.2,9.4,9.5,9.7,9.8,10.0,10.1,10.2,10.4,10.5,10.7,10.8,10.9,11.0,11.2,12.1,12.3,12.4,12.5,12.6,12.8,12.9,13.0,13.2,13.3,13.4,13.5,13.7,13.8,13.9,14.1,14.2,14.3,14.4,14.6,14.7,14.8,15.0,15.1,15.2,15.4,15.5,15.6,15.7,15.9,16.0,16.2,16.3,16.4,16.6,16.7,16.9,17.0,17.2,17.3,17.5,17.6,17.8,17.9,18.1,18.2,18.4,18.5,18.7,18.8,19.0,19.2,19.3,19.5,19.6,19.8,19.9,20.1,20.3,20.4,20.6,20.8,20.9,21.1,21.2,21.4,21.6,21.7,21.9,22.1,22.2,22.4,22.6,22.7,22.9,23.1,23.2,23.4,23.6,23.8,24.0,24.1,24.3,24.5,24.7,24.9,25.1,25.2,25.4,25.6,25.8,26.0,26.2,26.4,26.6,26.8,27.1,27.3,27.5,27.7,28.0,28.2,28.5,28.7,29.0,29.2,29.5,29.8,30.1,30.4,30.6,30.9,31.2,31.5,31.9,32.2,32.5,32.8,33.2,33.5,33.8,34.2,34.5,34.9,35.2,35.6,36.0,36.3,36.7,37.1,37.5,37.8,38.2,38.6,39.0,39.4,39.8,40.2,40.6,40.9,41.3,41.7,42.1,42.5,42.8,43.1,43.5,43.8,44.2,44.5,44.8,45.1,45.4,45.7,46.1,46.4,46.6,46.9,47.2,47.4,47.7,48.0,48.1,48.3,48.5,48.7,48.9,49.1,49.2,49.4,49.6,49.7,49.9,50.1,50.2,50.4,50.5,50.7,50.9,51.0,51.2,51.3,51.5,51.7,51.8,52.0,52.1,52.3,52.4,52.6,52.7,52.9,],
"[-1SD]":[2.9,3.9,4.9,5.7,6.2,6.7,7.1,7.4,7.7,8.0,8.2,8.4,8.6,8.8,9.0,9.2,9.4,9.6,9.8,10.0,10.1,10.3,10.5,10.7,10.8,11.0,11.2,11.3,11.5,11.7,11.8,12.0,12.1,12.3,12.4,12.6,13.4,13.5,13.7,13.8,14.0,14.1,14.2,14.4,14.5,14.7,14.8,15.0,15.1,15.3,15.4,15.6,15.7,15.9,16.0,16.2,16.3,16.5,16.6,16.8,16.9,17.1,17.2,17.4,17.5,17.7,17.8,18.0,18.2,18.3,18.5,18.7,18.9,19.0,19.2,19.4,19.6,19.8,20.0,20.1,20.3,20.5,20.7,20.9,21.1,21.3,21.5,21.7,21.9,22.1,22.3,22.6,22.8,23.0,23.2,23.4,23.6,23.9,24.1,24.3,24.5,24.8,25.0,25.2,25.5,25.7,26.0,26.2,26.4,26.7,26.9,27.2,27.4,27.7,27.9,28.2,28.5,28.7,29.0,29.2,29.5,29.8,30.1,30.4,30.6,30.9,31.2,31.5,31.8,32.1,32.4,32.7,33.0,33.3,33.7,34.0,34.3,34.6,35.0,35.3,35.7,36.0,36.4,36.7,37.1,37.5,37.8,38.2,38.6,39.0,39.4,39.7,40.1,40.5,40.9,41.3,41.7,42.1,42.5,42.9,43.3,43.7,44.1,44.5,44.9,45.3,45.7,46.1,46.5,46.9,47.3,47.7,48.1,48.5,48.8,49.2,49.5,49.9,50.3,50.6,50.9,51.2,51.6,51.9,52.2,52.5,52.8,53.0,53.3,53.6,53.8,54.1,54.3,54.6,54.8,55.0,55.2,55.5,55.6,55.8,56.0,56.1,56.3,56.5,56.6,56.8,56.9,57.1,57.2,57.4,57.5,57.7,57.8,58.0,58.1,58.3,58.4,58.6,58.7,58.8,59.0,59.1,59.3,59.4,59.6,59.7,59.9,60.0,],
"Mean":[3.3,4.5,5.6,6.4,7.0,7.5,7.9,8.3,8.6,8.9,9.2,9.4,9.6,9.9,10.1,10.3,10.5,10.7,10.9,11.1,11.3,11.5,11.8,12.0,12.2,12.4,12.5,12.7,12.9,13.1,13.3,13.5,13.7,13.8,14.0,14.2,14.7,14.9,15.1,15.3,15.4,15.6,15.8,16.0,16.1,16.3,16.5,16.7,16.8,17.0,17.2,17.4,17.5,17.7,17.9,18.1,18.2,18.4,18.6,18.8,19.0,19.1,19.3,19.5,19.7,19.9,20.1,20.3,20.5,20.7,20.9,21.1,21.3,21.6,21.8,22.0,22.3,22.5,22.7,23.0,23.2,23.5,23.7,24.0,24.2,24.5,24.7,25.0,25.3,25.5,25.8,26.1,26.4,26.7,26.9,27.2,27.5,27.8,28.1,28.4,28.7,29.1,29.4,29.7,30.0,30.3,30.7,31.0,31.3,31.7,32.0,32.3,32.7,33.0,33.4,33.7,34.1,34.4,34.8,35.2,35.5,35.9,36.3,36.7,37.0,37.4,37.8,38.2,38.6,39.0,39.4,39.8,40.2,40.6,41.1,41.5,41.9,42.3,42.8,43.2,43.6,44.1,44.5,45.0,45.4,45.9,46.3,46.8,47.3,47.7,48.2,48.6,49.1,49.5,50.0,50.5,50.9,51.3,51.8,52.2,52.7,53.1,53.5,53.9,54.4,54.8,55.2,55.6,56.0,56.4,56.7,57.1,57.5,57.9,58.2,58.5,58.9,59.2,59.5,59.8,60.1,60.4,60.7,60.9,61.2,61.5,61.7,61.9,62.2,62.4,62.6,62.9,63.1,63.2,63.4,63.6,63.8,64.0,64.2,64.3,64.5,64.6,64.8,64.9,65.0,65.2,65.3,65.5,65.6,65.8,65.9,66.0,66.2,66.3,66.4,66.6,66.7,66.9,67.0,67.1,67.3,67.4,67.5,67.7,67.8,67.9,68.1,68.2,],
"[+1SD]":[3.9,5.1,6.3,7.2,7.8,8.4,8.8,9.2,9.6,9.9,10.2,10.5,10.8,11.0,11.3,11.5,11.7,12.0,12.2,12.5,12.7,12.9,13.2,13.4,13.6,13.9,14.1,14.3,14.5,14.8,15.0,15.2,15.4,15.6,15.8,16.0,16.2,16.4,16.7,16.9,17.1,17.3,17.5,17.8,18.0,18.2,18.4,18.6,18.8,19.1,19.3,19.5,19.7,20.0,20.2,20.4,20.6,20.8,21.1,21.3,21.5,21.8,22.0,22.2,22.4,22.7,22.9,23.2,23.5,23.7,24.0,24.3,24.6,24.9,25.2,25.5,25.8,26.1,26.4,26.7,27.0,27.3,27.7,28.0,28.3,28.7,29.0,29.4,29.7,30.1,30.4,30.8,31.2,31.5,31.9,32.3,32.7,33.1,33.5,33.9,34.3,34.6,35.1,35.5,35.9,36.3,36.7,37.2,37.6,38.0,38.5,38.9,39.3,39.8,40.2,40.7,41.2,41.6,42.1,42.5,43.0,43.5,44.0,44.5,44.9,45.4,45.9,46.4,46.9,47.4,47.9,48.4,48.9,49.4,50.0,50.5,51.0,51.5,52.1,52.6,53.1,53.7,54.2,54.8,55.3,55.8,56.4,56.9,57.4,58.0,58.5,59.0,59.5,60.0,60.5,61.1,61.5,62.0,62.5,62.9,63.4,63.9,64.3,64.7,65.1,65.6,66.0,66.4,66.7,67.1,67.4,67.8,68.1,68.5,68.8,69.1,69.3,69.6,69.9,70.2,70.4,70.7,70.9,71.1,71.3,71.6,71.8,71.9,72.1,72.3,72.5,72.7,72.9,73.1,73.2,73.4,73.6,73.7,73.9,74.0,74.1,74.3,74.4,74.6,74.7,74.8,75.0,75.1,75.2,75.4,75.5,75.6,75.7,75.9,76.0,76.1,76.3,76.4,76.5,76.7,76.8,76.9,77.1,77.2,77.3,77.4,77.6,77.7,],
"[+2SD]":[4.4,5.8,7.1,8.0,8.7,9.3,9.8,10.3,10.7,11.0,11.4,11.7,12.0,12.3,12.6,12.8,13.1,13.4,13.7,13.9,14.2,14.5,14.7,15.0,15.3,15.5,15.8,16.1,16.3,16.6,16.9,17.1,17.4,17.6,17.8,18.1,17.9,18.1,18.4,18.7,19.0,19.3,19.6,19.8,20.1,20.4,20.7,21.0,21.3,21.6,21.8,22.1,22.4,22.7,23.0,23.3,23.6,23.9,24.2,24.5,24.8,25.1,25.4,25.7,26.0,26.3,26.7,27.1,27.4,27.8,28.2,28.5,28.9,29.3,29.7,30.1,30.5,30.9,31.4,31.8,32.2,32.7,33.1,33.5,34.0,34.5,34.9,35.4,35.8,36.3,36.8,37.3,37.8,38.2,38.7,39.2,39.7,40.2,40.7,41.2,41.7,42.2,42.7,43.2,43.7,44.2,44.7,45.3,45.8,46.3,46.9,47.4,48.0,48.5,49.1,49.6,50.2,50.7,51.3,51.9,52.4,53.0,53.6,54.1,54.7,55.3,55.9,56.5,57.0,57.6,58.2,58.8,59.4,60.1,60.7,61.3,61.9,62.5,63.2,63.8,64.4,65.0,65.7,66.3,66.9,67.5,68.1,68.7,69.3,69.9,70.5,71.0,71.6,72.1,72.7,73.3,73.7,74.2,74.7,75.2,75.7,76.2,76.5,76.9,77.3,77.7,78.1,78.5,78.8,79.1,79.5,79.8,80.1,80.4,80.6,80.9,81.1,81.3,81.6,81.8,82.0,82.1,82.3,82.5,82.7,82.8,83.0,83.1,83.3,83.4,83.6,83.8,83.9,84.1,84.3,84.4,84.6,84.8,84.9,85.0,85.2,85.3,85.5,85.6,85.7,85.9,86.0,86.1,86.2,86.4,86.5,86.6,86.8,86.9,87.0,87.1,87.3,87.4,87.5,87.7,87.8,87.9,88.0,88.2,88.3,88.4,88.5,88.7,],
"[+3SD]":[5.0,6.6,8.0,9.0,9.7,10.4,10.9,11.4,11.9,12.3,12.7,13.0,13.3,13.7,14.0,14.3,14.6,14.9,15.3,15.6,15.9,16.2,16.5,16.8,17.1,17.5,17.8,18.1,18.4,18.7,19.0,19.3,19.6,19.9,20.2,20.4,19.7,20.0,20.4,20.8,21.1,21.5,21.9,22.3,22.6,23.0,23.4,23.8,24.2,24.6,25.0,25.4,25.8,26.2,26.6,27.0,27.4,27.8,28.2,28.6,29.1,29.5,29.9,30.4,30.8,31.3,31.8,32.4,32.9,33.5,34.0,34.6,35.2,35.7,36.3,36.9,37.5,38.1,38.7,39.2,39.8,40.4,41.0,41.6,42.3,42.9,43.5,44.2,44.8,45.5,46.1,46.7,47.3,48.0,48.6,49.2,49.8,50.4,50.9,51.5,52.1,52.7,53.3,53.8,54.4,55.0,55.6,56.2,56.8,57.4,58.0,58.6,59.2,59.9,60.5,61.1,61.8,62.4,63.1,63.7,64.3,65.0,65.6,66.2,66.9,67.5,68.2,68.8,69.5,70.1,70.8,71.4,72.1,72.8,73.5,74.2,74.9,75.6,76.3,77.0,77.7,78.4,79.1,79.8,80.5,81.1,81.8,82.5,83.1,83.8,84.3,84.9,85.5,86.1,86.6,87.2,87.7,88.2,88.6,89.1,89.6,90.0,90.4,90.7,91.1,91.4,91.8,92.1,92.4,92.6,92.9,93.2,93.4,93.7,93.8,94.0,94.2,94.3,94.5,94.7,94.8,94.9,95.0,95.1,95.2,95.3,95.5,95.6,95.7,95.9,96.0,96.1,96.3,96.5,96.7,96.9,97.1,97.3,97.4,97.6,97.8,97.9,98.1,98.2,98.4,98.5,98.6,98.7,98.9,99.0,99.1,99.3,99.4,99.5,99.7,99.8,99.9,100.1,100.2,100.3,100.5,100.6,100.7,100.9,101.0,101.1,101.3,101.4,],
}
pd.DataFrame(df).to_csv('cache/GC_Wt_M.csv')

In [ ]:
# GC_Wt_F (여아 체중)
df = {"Month":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,],
"L":[0.3809,0.1714,0.0962,0.0402,-0.005,-0.043,-0.0756,-0.1039,-0.1288,-0.1507,-0.17,-0.1872,-0.2024,-0.2158,-0.2278,-0.2384,-0.2478,-0.2562,-0.2637,-0.2703,-0.2762,-0.2815,-0.2862,-0.2903,-0.2941,-0.2975,-0.3005,-0.3032,-0.3057,-0.308,-0.3101,-0.312,-0.3138,-0.3155,-0.3171,-0.3186,0.5656,0.4441,0.3317,0.2276,0.1312,0.0419,-0.0428,-0.1212,-0.1938,-0.2611,-0.3236,-0.3816,-0.4372,-0.4888,-0.5367,-0.5812,-0.6227,-0.6613,-0.6987,-0.7335,-0.7659999999999999,-0.7962,-0.8245,-0.8509,-0.8759999999999999,-0.8993,-0.9209999999999999,-0.9412,-0.96,-0.9776,-0.9840000000000001,-0.9894,-0.9941,-0.9981,-1.0014,-1.0041,-0.9954,-0.9868,-0.9784,-0.97,-0.9618,-0.9537,-0.9411,-0.9288,-0.9169,-0.9054,-0.8942,-0.8833,-0.8685,-0.8541,-0.8403,-0.8271,-0.8142,-0.8019,-0.7852,-0.7693,-0.7539,-0.7392,-0.725,-0.7114,-0.6971,-0.6834,-0.6702,-0.6574,-0.6451,-0.6333,-0.6214,-0.6098,-0.5987,-0.5879,-0.5775,-0.5675,-0.5521,-0.5373,-0.5229,-0.509,-0.4956,-0.4825,-0.4623,-0.4427,-0.4237,-0.4052,-0.3874,-0.37,-0.3496,-0.3297,-0.3103,-0.2914,-0.2729,-0.2548,-0.2377,-0.2209,-0.2043,-0.188,-0.1719,-0.156,-0.1419,-0.1278,-0.1138,-0.0998,-0.0859,-0.07200000000000001,-0.0641,-0.0562,-0.0481,-0.04,-0.0318,-0.0235,-0.0241,-0.0246,-0.0251,-0.0255,-0.0258,-0.026000000000000002,-0.0326,-0.0391,-0.0457,-0.0523,-0.0589,-0.0655,-0.0745,-0.0837,-0.0929,-0.1021,-0.1115,-0.1208,-0.129,-0.1372,-0.1455,-0.1538,-0.1621,-0.1705,-0.1734,-0.1763,-0.1791,-0.1819,-0.1846,-0.1873,-0.1869,-0.1865,-0.18600000000000003,-0.1854,-0.1847,-0.1839,-0.1846,-0.1852,-0.1857,-0.1862,-0.1867,-0.1871,-0.1901,-0.193,-0.196,-0.19899999999999998,-0.2019,-0.2049,-0.21,-0.2151,-0.2202,-0.2253,-0.2305,-0.2357,-0.2499,-0.2643,-0.2788,-0.2934,-0.3081,-0.32299999999999995,-0.3423,-0.3619,-0.3816,-0.4016,-0.4217,-0.4421,-0.4627,-0.4836,-0.5046,-0.5259,-0.5474,-0.5691,-0.5908,-0.6128,-0.6351,-0.6575,-0.6801,-0.703,-0.7262,-0.7497,-0.7734,-0.7973,-0.8215,-0.8458,],
"M":[3.2322,4.1873,5.1282,5.8458,6.4237,6.8985,7.297000000000001,7.6422,7.9487,8.2254,8.48,8.7192,8.9481,9.1699,9.387,9.6008,9.8124,10.0226,10.2315,10.4393,10.6464,10.8534,11.0608,11.2688,11.4775,11.6864,11.8947,12.1015,12.3059,12.5073,12.7055,12.9006,13.093,13.2837,13.4731,13.6618,14.1998,14.3701,14.5405,14.7108,14.8813,15.0518,15.2236,15.3956,15.5676,15.7399,15.9123,16.0848,16.2585,16.4324,16.6064,16.7806,16.9549,17.1294,17.3046,17.48,17.6555,17.8311,18.0069,18.1827,18.3616,18.5405,18.7196,18.8988,19.078,19.2574,19.4555,19.6537,19.8519,20.0502,20.2485,20.4468,20.6619,20.8769,21.092,21.3071,21.5221,21.7372,21.9702,22.2032,22.4363,22.6693,22.9023,23.1353,23.386999999999997,23.6386,23.8903,24.1419,24.3935,24.6451,24.9161,25.1871,25.4581,25.7291,26.0001,26.271,26.5602,26.8493,27.1385,27.4276,27.7167,28.0058,28.3163,28.6269,28.9374,29.248,29.5585,29.869,30.2033,30.5375,30.8718,31.2061,31.5404,31.8748,32.2323,32.59,32.9478,33.3056,33.6636,34.0216,34.4014,34.7815,35.1617,35.5421,35.9226,36.3033,36.7011,37.0992,37.4974,37.8958,38.2944,38.6931,39.0908,39.4887,39.8868,40.2851,40.6835,41.0821,41.4657,41.8495,42.2333,42.6173,43.0013,43.3855,43.74,44.0946,44.4491,44.8037,45.1584,45.513000000000005,45.8261,46.1391,46.4522,46.7652,47.0782,47.3912,47.6564,47.9216,48.1868,48.4519,48.7171,48.9822,49.2069,49.4315,49.6562,49.8809,50.1055,50.3302,50.5233,50.7164,50.9095,51.1026,51.2957,51.4888,51.6479,51.806999999999995,51.966,52.125,52.284,52.443000000000005,52.5631,52.6832,52.8033,52.9233,53.0434,53.1634,53.2482,53.3331,53.4179,53.5027,53.5875,53.6723,53.7253,53.7783,53.8313,53.8843,53.9373,53.9903,54.0055,54.0208,54.036,54.0512,54.0663,54.0815,54.0797,54.0778,54.0759,54.0739,54.0719,54.0698,54.0634,54.056999999999995,54.0505,54.0439,54.0373,54.0306,54.0217,54.0126,54.0036,53.9944,53.9852,53.976000000000006,53.9649,53.9539,53.9427,53.9315,53.9203,53.909,],
"S":[0.1417,0.1372,0.13,0.1262,0.124,0.1227,0.122,0.1218,0.1218,0.122,0.1222,0.1225,0.1227,0.1228,0.1229,0.12300000000000001,0.12300000000000001,0.1231,0.1231,0.1232,0.1232,0.1234,0.1235,0.1237,0.1239,0.1241,0.1244,0.1247,0.1251,0.1255,0.1259,0.1263,0.1268,0.1274,0.1279,0.1286,0.0991,0.1004,0.1017,0.10300000000000001,0.1042,0.1053,0.1064,0.1075,0.1085,0.1095,0.1105,0.1114,0.1124,0.1133,0.1142,0.115,0.1159,0.1167,0.1175,0.1183,0.11900000000000001,0.1198,0.1205,0.1212,0.122,0.1228,0.1236,0.1243,0.125,0.1257,0.127,0.1281,0.1293,0.1304,0.1315,0.1326,0.134,0.1353,0.1367,0.138,0.1392,0.1405,0.1418,0.1432,0.1445,0.1458,0.1471,0.1484,0.1497,0.1511,0.1524,0.1536,0.1549,0.1561,0.1574,0.1586,0.1598,0.1609,0.1621,0.1632,0.1643,0.1654,0.1665,0.1675,0.1685,0.1695,0.1705,0.1714,0.1723,0.1732,0.1741,0.1749,0.1758,0.1766,0.1774,0.1782,0.179,0.1798,0.1806,0.1815,0.1823,0.1831,0.1838,0.1846,0.1852,0.1858,0.1864,0.1869,0.1875,0.188,0.1882,0.1883,0.1884,0.1885,0.1886,0.1887,0.1884,0.1881,0.1878,0.1875,0.1872,0.1869,0.1862,0.1855,0.1848,0.1842,0.1835,0.1829,0.1818,0.1808,0.1798,0.1788,0.1778,0.1769,0.1756,0.1743,0.17300000000000001,0.1717,0.1705,0.1693,0.1679,0.1666,0.1653,0.16399999999999998,0.1627,0.1614,0.1603,0.1591,0.158,0.1568,0.1557,0.1546,0.1536,0.1525,0.1515,0.1505,0.1495,0.1485,0.1476,0.1467,0.1458,0.1449,0.14400000000000002,0.1431,0.1424,0.1417,0.1409,0.1402,0.1395,0.1388,0.1382,0.1377,0.1371,0.1366,0.1361,0.1355,0.1351,0.1346,0.1342,0.1338,0.1334,0.1329,0.1326,0.1323,0.1319,0.1316,0.1312,0.1309,0.1306,0.1303,0.13,0.1298,0.1295,0.1292,0.1289,0.1286,0.1284,0.1281,0.1278,0.1275,0.1273,0.127,0.1267,0.1264,0.1261,0.1258,0.1256,0.1253,0.125,0.1247,0.1244,0.1241,],
"1p":[2.3,3.0,3.8,4.4,4.8,5.2,5.5,5.8,6.0,6.2,6.4,6.6,6.8,6.9,7.1,7.3,7.4,7.6,7.8,7.9,8.1,8.2,8.4,8.5,8.7,8.9,9.0,9.2,9.3,9.5,9.6,9.7,9.9,10.0,10.1,10.3,11.1,11.2,11.4,11.5,11.6,11.8,11.9,12.0,12.2,12.3,12.4,12.6,12.7,12.8,13.0,13.1,13.2,13.3,13.5,13.6,13.7,13.9,14.0,14.1,14.2,14.4,14.5,14.6,14.8,14.9,15.0,15.1,15.3,15.4,15.5,15.6,15.7,15.9,16.0,16.1,16.2,16.4,16.5,16.6,16.7,16.9,17.0,17.1,17.3,17.4,17.5,17.7,17.8,17.9,18.1,18.2,18.3,18.5,18.6,18.8,18.9,19.1,19.2,19.4,19.5,19.7,19.9,20.0,20.2,20.4,20.5,20.7,20.9,21.1,21.2,21.4,21.6,21.8,22.0,22.1,22.3,22.5,22.7,22.8,23.0,23.2,23.4,23.6,23.8,24.0,24.2,24.4,24.6,24.9,25.1,25.3,25.6,25.8,26.0,26.3,26.5,26.8,27.0,27.3,27.6,27.9,28.1,28.4,28.7,29.0,29.3,29.6,29.9,30.2,30.5,30.9,31.2,31.5,31.8,32.1,32.4,32.7,33.0,33.3,33.6,33.9,34.2,34.5,34.7,35.0,35.2,35.5,35.7,35.9,36.2,36.4,36.6,36.8,37.0,37.2,37.4,37.6,37.8,38.0,38.1,38.3,38.4,38.6,38.7,38.9,39.0,39.1,39.2,39.3,39.4,39.5,39.6,39.7,39.8,39.9,40.0,40.1,40.1,40.2,40.3,40.3,40.4,40.4,40.5,40.6,40.6,40.7,40.7,40.8,40.8,40.9,40.9,41.0,41.0,41.1,41.1,41.2,41.2,41.3,41.3,41.4,41.4,41.5,41.5,41.5,41.6,41.6,],
"3p":[2.4,3.2,4.0,4.6,5.1,5.5,5.8,6.1,6.3,6.6,6.8,7.0,7.1,7.3,7.5,7.7,7.8,8.0,8.2,8.3,8.5,8.7,8.8,9.0,9.2,9.3,9.5,9.6,9.8,10.0,10.1,10.3,10.4,10.5,10.7,10.8,11.7,11.8,11.9,12.1,12.2,12.3,12.5,12.6,12.7,12.9,13.0,13.1,13.3,13.4,13.6,13.7,13.8,14.0,14.1,14.2,14.4,14.5,14.6,14.8,14.9,15.0,15.2,15.3,15.4,15.6,15.7,15.8,16.0,16.1,16.2,16.4,16.5,16.6,16.8,16.9,17.0,17.2,17.3,17.5,17.6,17.7,17.9,18.0,18.2,18.3,18.5,18.6,18.8,18.9,19.1,19.3,19.4,19.6,19.7,19.9,20.1,20.3,20.4,20.6,20.8,20.9,21.1,21.3,21.5,21.7,21.9,22.1,22.3,22.5,22.7,22.9,23.1,23.3,23.5,23.7,23.9,24.1,24.3,24.5,24.8,25.0,25.2,25.4,25.7,25.9,26.1,26.4,26.6,26.9,27.1,27.4,27.7,27.9,28.2,28.5,28.8,29.0,29.3,29.6,29.9,30.2,30.5,30.8,31.1,31.4,31.7,32.1,32.4,32.7,33.0,33.3,33.6,33.9,34.3,34.6,34.9,35.2,35.5,35.8,36.1,36.4,36.6,36.9,37.1,37.4,37.6,37.9,38.1,38.3,38.6,38.8,39.0,39.2,39.4,39.6,39.8,40.0,40.1,40.3,40.5,40.6,40.8,40.9,41.1,41.2,41.3,41.4,41.5,41.6,41.8,41.9,41.9,42.0,42.1,42.2,42.3,42.3,42.4,42.5,42.5,42.6,42.6,42.7,42.7,42.8,42.8,42.8,42.9,42.9,43.0,43.0,43.0,43.1,43.1,43.2,43.2,43.2,43.3,43.3,43.3,43.4,43.4,43.4,43.5,43.5,43.5,43.6,],
"5p":[2.5,3.3,4.1,4.7,5.2,5.6,6.0,6.3,6.5,6.8,7.0,7.2,7.3,7.5,7.7,7.9,8.1,8.2,8.4,8.6,8.7,8.9,9.1,9.2,9.4,9.6,9.8,9.9,10.1,10.2,10.4,10.5,10.7,10.8,11.0,11.1,12.0,12.1,12.2,12.4,12.5,12.7,12.8,12.9,13.1,13.2,13.3,13.5,13.6,13.7,13.9,14.0,14.2,14.3,14.4,14.6,14.7,14.8,15.0,15.1,15.3,15.4,15.5,15.7,15.8,16.0,16.1,16.2,16.4,16.5,16.6,16.8,16.9,17.1,17.2,17.4,17.5,17.6,17.8,17.9,18.1,18.2,18.4,18.5,18.7,18.9,19.0,19.2,19.4,19.5,19.7,19.9,20.0,20.2,20.4,20.5,20.7,20.9,21.1,21.3,21.5,21.7,21.9,22.1,22.3,22.5,22.7,22.9,23.1,23.3,23.5,23.7,24.0,24.2,24.4,24.6,24.8,25.1,25.3,25.5,25.8,26.0,26.2,26.5,26.7,27.0,27.2,27.5,27.8,28.0,28.3,28.6,28.9,29.2,29.4,29.7,30.0,30.3,30.6,30.9,31.2,31.5,31.8,32.1,32.5,32.8,33.1,33.4,33.7,34.1,34.4,34.7,35.0,35.3,35.6,36.0,36.3,36.5,36.8,37.1,37.4,37.7,38.0,38.2,38.5,38.7,39.0,39.2,39.5,39.7,39.9,40.1,40.3,40.6,40.7,40.9,41.1,41.3,41.5,41.6,41.8,41.9,42.1,42.2,42.4,42.5,42.6,42.7,42.8,42.9,43.1,43.2,43.2,43.3,43.4,43.5,43.5,43.6,43.7,43.7,43.8,43.8,43.9,43.9,44.0,44.0,44.0,44.1,44.1,44.1,44.2,44.2,44.2,44.3,44.3,44.3,44.3,44.4,44.4,44.4,44.5,44.5,44.5,44.5,44.6,44.6,44.6,44.7,],
"10p":[2.7,3.5,4.3,5.0,5.5,5.9,6.2,6.5,6.8,7.0,7.3,7.5,7.7,7.9,8.0,8.2,8.4,8.6,8.8,8.9,9.1,9.3,9.5,9.7,9.8,10.0,10.2,10.4,10.5,10.7,10.9,11.0,11.2,11.3,11.5,11.6,12.4,12.6,12.7,12.9,13.0,13.1,13.3,13.4,13.6,13.7,13.9,14.0,14.1,14.3,14.4,14.6,14.7,14.9,15.0,15.1,15.3,15.4,15.6,15.7,15.9,16.0,16.1,16.3,16.4,16.6,16.7,16.9,17.0,17.2,17.3,17.5,17.6,17.8,17.9,18.1,18.3,18.4,18.6,18.7,18.9,19.1,19.2,19.4,19.6,19.8,19.9,20.1,20.3,20.5,20.7,20.9,21.0,21.2,21.4,21.6,21.8,22.0,22.2,22.4,22.6,22.8,23.1,23.3,23.5,23.7,24.0,24.2,24.4,24.7,24.9,25.1,25.4,25.6,25.9,26.1,26.4,26.6,26.9,27.1,27.4,27.7,27.9,28.2,28.5,28.7,29.0,29.3,29.6,29.9,30.2,30.5,30.8,31.1,31.5,31.8,32.1,32.4,32.7,33.0,33.4,33.7,34.0,34.3,34.7,35.0,35.3,35.7,36.0,36.3,36.6,36.9,37.3,37.6,37.9,38.2,38.5,38.8,39.1,39.4,39.6,39.9,40.2,40.4,40.7,40.9,41.2,41.4,41.6,41.8,42.1,42.3,42.5,42.7,42.9,43.1,43.2,43.4,43.6,43.8,43.9,44.1,44.2,44.3,44.5,44.6,44.7,44.8,44.9,45.0,45.1,45.3,45.3,45.4,45.5,45.5,45.6,45.7,45.7,45.8,45.8,45.8,45.9,45.9,46.0,46.0,46.0,46.0,46.1,46.1,46.1,46.1,46.1,46.2,46.2,46.2,46.2,46.3,46.3,46.3,46.3,46.3,46.3,46.4,46.4,46.4,46.4,46.4,],
"15p":[2.8,3.6,4.5,5.1,5.6,6.1,6.4,6.7,7.0,7.3,7.5,7.7,7.9,8.1,8.3,8.5,8.7,8.8,9.0,9.2,9.4,9.6,9.8,9.9,10.1,10.3,10.5,10.7,10.8,11.0,11.2,11.3,11.5,11.7,11.8,12.0,12.8,12.9,13.1,13.2,13.3,13.5,13.6,13.8,13.9,14.1,14.2,14.4,14.5,14.7,14.8,15.0,15.1,15.2,15.4,15.5,15.7,15.8,16.0,16.1,16.3,16.4,16.6,16.7,16.9,17.0,17.2,17.3,17.5,17.7,17.8,18.0,18.1,18.3,18.5,18.6,18.8,19.0,19.1,19.3,19.5,19.7,19.9,20.0,20.2,20.4,20.6,20.8,21.0,21.2,21.4,21.6,21.8,22.0,22.2,22.4,22.6,22.8,23.1,23.3,23.5,23.7,23.9,24.2,24.4,24.7,24.9,25.1,25.4,25.6,25.9,26.2,26.4,26.7,26.9,27.2,27.5,27.7,28.0,28.3,28.6,28.9,29.1,29.4,29.7,30.0,30.3,30.6,31.0,31.3,31.6,31.9,32.2,32.6,32.9,33.2,33.6,33.9,34.2,34.6,34.9,35.2,35.6,35.9,36.2,36.6,36.9,37.2,37.6,37.9,38.2,38.5,38.9,39.2,39.5,39.8,40.1,40.4,40.7,40.9,41.2,41.5,41.7,42.0,42.2,42.5,42.7,43.0,43.2,43.4,43.6,43.8,44.0,44.2,44.4,44.6,44.8,44.9,45.1,45.3,45.4,45.6,45.7,45.9,46.0,46.1,46.2,46.3,46.4,46.5,46.6,46.7,46.8,46.9,46.9,47.0,47.1,47.1,47.2,47.2,47.3,47.3,47.3,47.4,47.4,47.4,47.4,47.4,47.5,47.5,47.5,47.5,47.5,47.5,47.6,47.6,47.6,47.6,47.6,47.6,47.6,47.6,47.7,47.7,47.7,47.7,47.7,47.7,],
"25p":[2.9,3.8,4.7,5.4,5.9,6.4,6.7,7.0,7.3,7.6,7.8,8.0,8.2,8.4,8.6,8.8,9.0,9.2,9.4,9.6,9.8,10.0,10.2,10.4,10.6,10.8,10.9,11.1,11.3,11.5,11.7,11.9,12.0,12.2,12.4,12.5,13.3,13.4,13.6,13.7,13.9,14.0,14.2,14.3,14.5,14.6,14.8,14.9,15.1,15.2,15.4,15.6,15.7,15.9,16.0,16.2,16.3,16.5,16.6,16.8,17.0,17.1,17.3,17.4,17.6,17.8,17.9,18.1,18.3,18.4,18.6,18.8,18.9,19.1,19.3,19.5,19.7,19.9,20.0,20.2,20.4,20.6,20.8,21.0,21.2,21.4,21.6,21.9,22.1,22.3,22.5,22.7,23.0,23.2,23.4,23.6,23.9,24.1,24.4,24.6,24.8,25.1,25.3,25.6,25.9,26.1,26.4,26.6,26.9,27.2,27.5,27.8,28.1,28.3,28.6,28.9,29.2,29.5,29.8,30.1,30.4,30.8,31.1,31.4,31.7,32.0,32.4,32.7,33.1,33.4,33.8,34.1,34.5,34.8,35.2,35.5,35.9,36.2,36.6,36.9,37.3,37.7,38.0,38.4,38.7,39.0,39.4,39.7,40.1,40.4,40.7,41.0,41.3,41.7,42.0,42.3,42.6,42.9,43.1,43.4,43.7,44.0,44.2,44.4,44.7,44.9,45.1,45.4,45.6,45.8,46.0,46.2,46.4,46.6,46.8,47.0,47.1,47.3,47.5,47.7,47.8,47.9,48.1,48.2,48.3,48.5,48.5,48.6,48.7,48.8,48.9,49.0,49.1,49.2,49.2,49.3,49.3,49.4,49.4,49.5,49.5,49.5,49.5,49.6,49.6,49.6,49.6,49.6,49.6,49.6,49.6,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,],
"50p":[3.2,4.2,5.1,5.8,6.4,6.9,7.3,7.6,7.9,8.2,8.5,8.7,8.9,9.2,9.4,9.6,9.8,10.0,10.2,10.4,10.6,10.9,11.1,11.3,11.5,11.7,11.9,12.1,12.3,12.5,12.7,12.9,13.1,13.3,13.5,13.7,14.2,14.4,14.5,14.7,14.9,15.1,15.2,15.4,15.6,15.7,15.9,16.1,16.3,16.4,16.6,16.8,17.0,17.1,17.3,17.5,17.7,17.8,18.0,18.2,18.4,18.5,18.7,18.9,19.1,19.3,19.5,19.7,19.9,20.1,20.2,20.4,20.7,20.9,21.1,21.3,21.5,21.7,22.0,22.2,22.4,22.7,22.9,23.1,23.4,23.6,23.9,24.1,24.4,24.6,24.9,25.2,25.5,25.7,26.0,26.3,26.6,26.8,27.1,27.4,27.7,28.0,28.3,28.6,28.9,29.2,29.6,29.9,30.2,30.5,30.9,31.2,31.5,31.9,32.2,32.6,32.9,33.3,33.7,34.0,34.4,34.8,35.2,35.5,35.9,36.3,36.7,37.1,37.5,37.9,38.3,38.7,39.1,39.5,39.9,40.3,40.7,41.1,41.5,41.8,42.2,42.6,43.0,43.4,43.7,44.1,44.4,44.8,45.2,45.5,45.8,46.1,46.5,46.8,47.1,47.4,47.7,47.9,48.2,48.5,48.7,49.0,49.2,49.4,49.7,49.9,50.1,50.3,50.5,50.7,50.9,51.1,51.3,51.5,51.6,51.8,52.0,52.1,52.3,52.4,52.6,52.7,52.8,52.9,53.0,53.2,53.2,53.3,53.4,53.5,53.6,53.7,53.7,53.8,53.8,53.9,53.9,54.0,54.0,54.0,54.0,54.1,54.1,54.1,54.1,54.1,54.1,54.1,54.1,54.1,54.1,54.1,54.1,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,53.9,53.9,53.9,53.9,],
"75p":[3.6,4.6,5.6,6.4,7.0,7.5,7.9,8.3,8.6,8.9,9.2,9.5,9.7,10.0,10.2,10.4,10.7,10.9,11.1,11.4,11.6,11.8,12.0,12.3,12.5,12.7,12.9,13.2,13.4,13.6,13.8,14.1,14.3,14.5,14.7,14.9,15.2,15.4,15.6,15.8,16.0,16.2,16.4,16.6,16.8,17.0,17.2,17.4,17.6,17.8,18.0,18.2,18.4,18.6,18.8,19.0,19.2,19.4,19.6,19.8,20.0,20.2,20.4,20.6,20.8,21.0,21.3,21.5,21.7,22.0,22.2,22.5,22.7,23.0,23.2,23.5,23.7,24.0,24.3,24.6,24.8,25.1,25.4,25.7,26.0,26.3,26.6,26.9,27.2,27.5,27.8,28.2,28.5,28.8,29.1,29.5,29.8,30.2,30.5,30.8,31.2,31.5,31.9,32.3,32.6,33.0,33.4,33.7,34.1,34.5,34.9,35.3,35.7,36.1,36.5,37.0,37.4,37.8,38.2,38.6,39.1,39.5,40.0,40.4,40.9,41.3,41.7,42.2,42.6,43.1,43.6,44.0,44.4,44.9,45.3,45.8,46.2,46.6,47.0,47.4,47.9,48.3,48.7,49.1,49.5,49.8,50.2,50.6,50.9,51.3,51.6,51.9,52.2,52.5,52.8,53.1,53.4,53.6,53.9,54.2,54.4,54.7,54.9,55.1,55.3,55.5,55.7,55.9,56.1,56.3,56.4,56.6,56.8,57.0,57.1,57.2,57.4,57.5,57.7,57.8,57.9,58.0,58.1,58.2,58.3,58.4,58.5,58.6,58.6,58.7,58.8,58.9,58.9,58.9,59.0,59.0,59.1,59.1,59.1,59.1,59.1,59.1,59.1,59.1,59.1,59.1,59.1,59.1,59.1,59.1,59.1,59.1,59.1,59.0,59.0,59.0,59.0,59.0,59.0,58.9,58.9,58.9,58.9,58.9,58.9,58.8,58.8,58.8,],
"85p":[3.7,4.8,5.9,6.7,7.3,7.8,8.3,8.7,9.0,9.3,9.6,9.9,10.2,10.4,10.7,10.9,11.2,11.4,11.6,11.9,12.1,12.4,12.6,12.8,13.1,13.3,13.6,13.8,14.0,14.3,14.5,14.7,15.0,15.2,15.4,15.7,15.7,15.9,16.1,16.3,16.6,16.8,17.0,17.2,17.4,17.7,17.9,18.1,18.3,18.5,18.8,19.0,19.2,19.4,19.7,19.9,20.1,20.3,20.5,20.8,21.0,21.2,21.5,21.7,21.9,22.1,22.4,22.7,22.9,23.2,23.4,23.7,24.0,24.3,24.6,24.9,25.1,25.4,25.7,26.0,26.4,26.7,27.0,27.3,27.6,28.0,28.3,28.6,29.0,29.3,29.7,30.0,30.4,30.7,31.1,31.5,31.8,32.2,32.6,33.0,33.4,33.7,34.1,34.5,35.0,35.4,35.8,36.2,36.6,37.0,37.5,37.9,38.3,38.7,39.2,39.7,40.1,40.6,41.0,41.5,42.0,42.4,42.9,43.4,43.9,44.3,44.8,45.3,45.8,46.2,46.7,47.2,47.7,48.1,48.6,49.0,49.5,49.9,50.4,50.8,51.2,51.6,52.0,52.5,52.8,53.2,53.6,53.9,54.3,54.7,55.0,55.3,55.6,55.9,56.2,56.5,56.8,57.0,57.3,57.5,57.8,58.0,58.2,58.4,58.6,58.8,59.0,59.2,59.4,59.5,59.7,59.9,60.0,60.2,60.3,60.4,60.6,60.7,60.8,61.0,61.0,61.1,61.2,61.3,61.4,61.5,61.6,61.6,61.7,61.8,61.8,61.9,61.9,62.0,62.0,62.0,62.1,62.1,62.1,62.1,62.1,62.1,62.1,62.1,62.1,62.1,62.1,62.1,62.1,62.1,62.1,62.0,62.0,62.0,62.0,62.0,62.0,62.0,61.9,61.9,61.9,61.9,61.9,61.9,61.8,61.8,61.8,61.8,],
"90p":[3.9,5.0,6.0,6.9,7.5,8.1,8.5,8.9,9.3,9.6,9.9,10.2,10.5,10.8,11.0,11.3,11.5,11.8,12.0,12.3,12.5,12.8,13.0,13.3,13.5,13.8,14.0,14.3,14.5,14.7,15.0,15.2,15.5,15.7,15.9,16.2,16.1,16.3,16.5,16.8,17.0,17.2,17.5,17.7,17.9,18.2,18.4,18.6,18.9,19.1,19.3,19.6,19.8,20.0,20.3,20.5,20.8,21.0,21.2,21.5,21.7,22.0,22.2,22.5,22.7,22.9,23.2,23.5,23.8,24.1,24.4,24.6,24.9,25.3,25.6,25.9,26.2,26.5,26.8,27.2,27.5,27.8,28.2,28.5,28.8,29.2,29.6,29.9,30.3,30.7,31.0,31.4,31.8,32.2,32.6,32.9,33.4,33.8,34.2,34.6,35.0,35.4,35.8,36.2,36.7,37.1,37.5,38.0,38.4,38.9,39.3,39.8,40.2,40.7,41.2,41.7,42.1,42.6,43.1,43.6,44.1,44.6,45.1,45.6,46.1,46.6,47.0,47.5,48.0,48.5,49.0,49.5,50.0,50.4,50.9,51.4,51.8,52.3,52.7,53.2,53.6,54.0,54.5,54.9,55.3,55.6,56.0,56.4,56.8,57.1,57.4,57.7,58.0,58.4,58.7,59.0,59.2,59.4,59.7,59.9,60.2,60.4,60.6,60.8,61.0,61.2,61.4,61.6,61.7,61.9,62.0,62.2,62.3,62.5,62.6,62.7,62.8,63.0,63.1,63.2,63.3,63.4,63.5,63.5,63.6,63.7,63.8,63.8,63.9,63.9,64.0,64.1,64.1,64.1,64.1,64.2,64.2,64.2,64.2,64.2,64.3,64.3,64.3,64.3,64.3,64.2,64.2,64.2,64.2,64.2,64.2,64.2,64.2,64.2,64.2,64.1,64.1,64.1,64.1,64.1,64.1,64.1,64.0,64.0,64.0,64.0,64.0,64.0,],
"95p":[4.0,5.2,6.3,7.2,7.9,8.4,8.9,9.4,9.7,10.1,10.4,10.7,11.0,11.3,11.5,11.8,12.1,12.3,12.6,12.9,13.1,13.4,13.6,13.9,14.2,14.4,14.7,15.0,15.2,15.5,15.7,16.0,16.2,16.5,16.8,17.0,16.6,16.9,17.1,17.4,17.6,17.9,18.1,18.4,18.7,18.9,19.2,19.5,19.7,20.0,20.2,20.5,20.8,21.0,21.3,21.6,21.8,22.1,22.4,22.6,22.9,23.2,23.4,23.7,24.0,24.3,24.6,24.9,25.2,25.5,25.8,26.2,26.5,26.8,27.2,27.5,27.9,28.2,28.6,29.0,29.3,29.7,30.1,30.5,30.9,31.3,31.7,32.1,32.5,32.9,33.3,33.7,34.1,34.5,35.0,35.4,35.8,36.3,36.7,37.2,37.6,38.1,38.5,39.0,39.5,39.9,40.4,40.9,41.4,41.9,42.4,42.9,43.4,43.9,44.4,44.9,45.4,45.9,46.4,46.9,47.5,48.0,48.5,49.1,49.6,50.1,50.6,51.1,51.6,52.2,52.7,53.2,53.7,54.1,54.6,55.1,55.6,56.1,56.5,56.9,57.4,57.8,58.2,58.7,59.1,59.4,59.8,60.2,60.6,60.9,61.3,61.6,61.9,62.2,62.5,62.8,63.0,63.2,63.5,63.7,63.9,64.2,64.3,64.5,64.7,64.9,65.1,65.3,65.4,65.6,65.7,65.8,66.0,66.1,66.2,66.3,66.4,66.5,66.6,66.7,66.8,66.9,66.9,67.0,67.1,67.1,67.2,67.2,67.3,67.3,67.4,67.4,67.5,67.5,67.5,67.5,67.6,67.6,67.6,67.6,67.6,67.6,67.6,67.6,67.6,67.6,67.6,67.6,67.6,67.6,67.6,67.6,67.6,67.6,67.6,67.6,67.5,67.5,67.5,67.5,67.5,67.5,67.5,67.5,67.5,67.5,67.5,67.5,],
"97p":[4.2,5.4,6.5,7.4,8.1,8.7,9.2,9.6,10.0,10.4,10.7,11.0,11.3,11.6,11.9,12.2,12.5,12.7,13.0,13.3,13.5,13.8,14.1,14.3,14.6,14.9,15.2,15.4,15.7,16.0,16.2,16.5,16.8,17.0,17.3,17.6,17.0,17.2,17.5,17.8,18.1,18.3,18.6,18.9,19.2,19.5,19.7,20.0,20.3,20.6,20.9,21.1,21.4,21.7,22.0,22.3,22.6,22.9,23.1,23.4,23.7,24.0,24.3,24.6,24.9,25.2,25.5,25.9,26.2,26.6,26.9,27.2,27.6,28.0,28.4,28.7,29.1,29.5,29.9,30.3,30.7,31.1,31.5,31.9,32.3,32.7,33.2,33.6,34.0,34.4,34.9,35.3,35.8,36.2,36.7,37.1,37.6,38.1,38.6,39.0,39.5,40.0,40.5,41.0,41.5,42.0,42.5,43.0,43.5,44.0,44.5,45.1,45.6,46.1,46.6,47.2,47.7,48.3,48.8,49.3,49.9,50.4,51.0,51.5,52.0,52.6,53.1,53.6,54.2,54.7,55.2,55.7,56.2,56.7,57.2,57.7,58.2,58.7,59.1,59.5,60.0,60.4,60.8,61.3,61.7,62.0,62.4,62.8,63.2,63.6,63.9,64.2,64.5,64.8,65.1,65.4,65.6,65.8,66.1,66.3,66.5,66.7,66.9,67.1,67.3,67.5,67.6,67.8,67.9,68.1,68.2,68.3,68.5,68.6,68.7,68.8,68.9,68.9,69.0,69.1,69.2,69.2,69.3,69.4,69.4,69.5,69.5,69.6,69.6,69.6,69.7,69.7,69.8,69.8,69.8,69.8,69.8,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,69.9,],
"99p":[4.4,5.7,6.9,7.8,8.6,9.2,9.7,10.2,10.6,11.0,11.3,11.7,12.0,12.3,12.6,12.9,13.2,13.5,13.8,14.1,14.4,14.6,14.9,15.2,15.5,15.8,16.1,16.4,16.7,17.0,17.3,17.6,17.8,18.1,18.4,18.7,17.6,17.9,18.3,18.6,18.9,19.2,19.5,19.8,20.2,20.5,20.8,21.1,21.5,21.8,22.1,22.4,22.8,23.1,23.4,23.8,24.1,24.4,24.8,25.1,25.4,25.8,26.1,26.5,26.8,27.2,27.6,28.0,28.4,28.8,29.2,29.6,30.0,30.4,30.9,31.3,31.7,32.2,32.6,33.1,33.5,34.0,34.5,34.9,35.4,35.9,36.4,36.9,37.4,37.9,38.4,38.9,39.4,39.9,40.4,40.9,41.4,42.0,42.5,43.0,43.6,44.1,44.6,45.2,45.8,46.3,46.9,47.4,48.0,48.6,49.1,49.7,50.3,50.9,51.4,52.0,52.6,53.2,53.7,54.3,54.9,55.4,56.0,56.6,57.2,57.7,58.3,58.8,59.3,59.9,60.4,61.0,61.5,62.0,62.4,62.9,63.4,63.9,64.3,64.8,65.2,65.7,66.1,66.5,66.9,67.3,67.7,68.1,68.4,68.8,69.1,69.4,69.7,70.0,70.3,70.6,70.8,71.1,71.3,71.5,71.7,71.9,72.1,72.3,72.4,72.6,72.8,73.0,73.1,73.2,73.3,73.4,73.5,73.6,73.6,73.7,73.8,73.8,73.9,73.9,74.0,74.0,74.1,74.1,74.1,74.2,74.2,74.2,74.3,74.3,74.3,74.3,74.4,74.4,74.4,74.4,74.4,74.4,74.4,74.5,74.5,74.5,74.5,74.5,74.5,74.6,74.6,74.6,74.6,74.6,74.7,74.7,74.7,74.7,74.8,74.8,74.8,74.8,74.8,74.9,74.9,74.9,74.9,75.0,75.0,75.0,75.0,75.1,],
"[-3SD]":[2.0,2.7,3.4,4.0,4.4,4.8,5.1,5.3,5.6,5.8,5.9,6.1,6.3,6.4,6.6,6.7,6.9,7.0,7.2,7.3,7.5,7.6,7.8,7.9,8.1,8.2,8.4,8.5,8.6,8.8,8.9,9.0,9.1,9.3,9.4,9.5,10.3,10.4,10.5,10.7,10.8,11.0,11.1,11.2,11.4,11.5,11.6,11.7,11.9,12.0,12.1,12.3,12.4,12.5,12.6,12.8,12.9,13.0,13.1,13.2,13.4,13.5,13.6,13.7,13.8,14.0,14.1,14.2,14.3,14.4,14.5,14.6,14.7,14.8,14.9,15.0,15.1,15.3,15.4,15.5,15.6,15.7,15.8,15.9,16.0,16.1,16.2,16.3,16.4,16.6,16.7,16.8,16.9,17.0,17.1,17.3,17.4,17.5,17.6,17.8,17.9,18.0,18.2,18.3,18.4,18.6,18.7,18.9,19.0,19.2,19.3,19.4,19.6,19.7,19.9,20.0,20.1,20.3,20.4,20.6,20.7,20.9,21.0,21.1,21.3,21.4,21.6,21.8,22.0,22.1,22.3,22.5,22.7,22.9,23.1,23.3,23.5,23.7,24.0,24.2,24.4,24.7,24.9,25.2,25.4,25.7,26.0,26.3,26.6,26.9,27.2,27.5,27.8,28.1,28.4,28.8,29.1,29.4,29.7,30.0,30.3,30.6,30.9,31.1,31.4,31.7,31.9,32.2,32.4,32.7,32.9,33.1,33.3,33.6,33.8,33.9,34.1,34.3,34.5,34.7,34.8,35.0,35.1,35.3,35.5,35.6,35.7,35.8,36.0,36.1,36.2,36.3,36.4,36.5,36.6,36.7,36.8,36.9,37.0,37.0,37.1,37.2,37.3,37.4,37.4,37.5,37.6,37.7,37.7,37.8,37.9,37.9,38.0,38.1,38.2,38.2,38.3,38.4,38.4,38.5,38.6,38.6,38.7,38.7,38.8,38.9,38.9,39.0,],
"[-2SD]":[2.4,3.2,3.9,4.5,5.0,5.4,5.7,6.0,6.3,6.5,6.7,6.9,7.0,7.2,7.4,7.6,7.7,7.9,8.1,8.2,8.4,8.6,8.7,8.9,9.0,9.2,9.4,9.5,9.7,9.8,10.0,10.1,10.3,10.4,10.5,10.7,11.5,11.6,11.8,11.9,12.0,12.2,12.3,12.5,12.6,12.7,12.9,13.0,13.1,13.3,13.4,13.5,13.7,13.8,13.9,14.1,14.2,14.3,14.5,14.6,14.7,14.9,15.0,15.1,15.2,15.4,15.5,15.6,15.8,15.9,16.0,16.2,16.3,16.4,16.6,16.7,16.8,16.9,17.1,17.2,17.4,17.5,17.6,17.8,17.9,18.1,18.2,18.4,18.5,18.7,18.8,19.0,19.1,19.3,19.4,19.6,19.8,19.9,20.1,20.3,20.4,20.6,20.8,21.0,21.2,21.3,21.5,21.7,21.9,22.1,22.3,22.5,22.7,22.9,23.1,23.3,23.5,23.7,23.9,24.1,24.3,24.5,24.7,24.9,25.1,25.4,25.6,25.8,26.1,26.3,26.6,26.8,27.1,27.3,27.6,27.9,28.1,28.4,28.7,29.0,29.3,29.6,29.9,30.1,30.5,30.8,31.1,31.4,31.7,32.0,32.3,32.6,33.0,33.3,33.6,33.9,34.2,34.5,34.8,35.1,35.4,35.7,35.9,36.2,36.5,36.7,37.0,37.2,37.5,37.7,37.9,38.1,38.3,38.6,38.8,38.9,39.1,39.3,39.5,39.7,39.8,40.0,40.1,40.3,40.4,40.6,40.7,40.8,40.9,41.0,41.1,41.2,41.3,41.4,41.5,41.6,41.6,41.7,41.8,41.8,41.9,41.9,42.0,42.1,42.1,42.2,42.2,42.2,42.3,42.3,42.4,42.4,42.5,42.5,42.5,42.6,42.6,42.7,42.7,42.7,42.8,42.8,42.8,42.9,42.9,43.0,43.0,43.0,],
"[-1SD]":[2.8,3.6,4.5,5.2,5.7,6.1,6.5,6.8,7.0,7.3,7.5,7.7,7.9,8.1,8.3,8.5,8.7,8.9,9.1,9.2,9.4,9.6,9.8,10.0,10.2,10.3,10.5,10.7,10.9,11.1,11.2,11.4,11.6,11.7,11.9,12.0,12.8,13.0,13.1,13.3,13.4,13.5,13.7,13.8,14.0,14.1,14.3,14.4,14.6,14.7,14.9,15.0,15.2,15.3,15.5,15.6,15.8,15.9,16.1,16.2,16.4,16.5,16.7,16.8,17.0,17.1,17.3,17.4,17.6,17.7,17.9,18.1,18.2,18.4,18.6,18.7,18.9,19.1,19.2,19.4,19.6,19.8,19.9,20.1,20.3,20.5,20.7,20.9,21.1,21.3,21.5,21.7,21.9,22.1,22.3,22.5,22.7,23.0,23.2,23.4,23.6,23.8,24.1,24.3,24.6,24.8,25.0,25.3,25.5,25.8,26.1,26.3,26.6,26.8,27.1,27.4,27.6,27.9,28.2,28.5,28.8,29.0,29.3,29.6,29.9,30.2,30.5,30.8,31.2,31.5,31.8,32.1,32.5,32.8,33.1,33.5,33.8,34.1,34.5,34.8,35.1,35.5,35.8,36.1,36.5,36.8,37.1,37.5,37.8,38.2,38.5,38.8,39.1,39.4,39.7,40.0,40.3,40.6,40.9,41.2,41.5,41.7,42.0,42.2,42.5,42.7,43.0,43.2,43.4,43.6,43.8,44.1,44.3,44.5,44.7,44.8,45.0,45.2,45.4,45.5,45.7,45.8,45.9,46.1,46.2,46.4,46.5,46.6,46.7,46.8,46.9,47.0,47.0,47.1,47.2,47.2,47.3,47.4,47.4,47.4,47.5,47.5,47.5,47.6,47.6,47.6,47.6,47.6,47.7,47.7,47.7,47.7,47.7,47.7,47.8,47.8,47.8,47.8,47.8,47.8,47.8,47.8,47.9,47.9,47.9,47.9,47.9,47.9,],
"Mean":[3.2,4.2,5.1,5.8,6.4,6.9,7.3,7.6,7.9,8.2,8.5,8.7,8.9,9.2,9.4,9.6,9.8,10.0,10.2,10.4,10.6,10.9,11.1,11.3,11.5,11.7,11.9,12.1,12.3,12.5,12.7,12.9,13.1,13.3,13.5,13.7,14.2,14.4,14.5,14.7,14.9,15.1,15.2,15.4,15.6,15.7,15.9,16.1,16.3,16.4,16.6,16.8,17.0,17.1,17.3,17.5,17.7,17.8,18.0,18.2,18.4,18.5,18.7,18.9,19.1,19.3,19.5,19.7,19.9,20.1,20.2,20.4,20.7,20.9,21.1,21.3,21.5,21.7,22.0,22.2,22.4,22.7,22.9,23.1,23.4,23.6,23.9,24.1,24.4,24.6,24.9,25.2,25.5,25.7,26.0,26.3,26.6,26.8,27.1,27.4,27.7,28.0,28.3,28.6,28.9,29.2,29.6,29.9,30.2,30.5,30.9,31.2,31.5,31.9,32.2,32.6,32.9,33.3,33.7,34.0,34.4,34.8,35.2,35.5,35.9,36.3,36.7,37.1,37.5,37.9,38.3,38.7,39.1,39.5,39.9,40.3,40.7,41.1,41.5,41.8,42.2,42.6,43.0,43.4,43.7,44.1,44.4,44.8,45.2,45.5,45.8,46.1,46.5,46.8,47.1,47.4,47.7,47.9,48.2,48.5,48.7,49.0,49.2,49.4,49.7,49.9,50.1,50.3,50.5,50.7,50.9,51.1,51.3,51.5,51.6,51.8,52.0,52.1,52.3,52.4,52.6,52.7,52.8,52.9,53.0,53.2,53.2,53.3,53.4,53.5,53.6,53.7,53.7,53.8,53.8,53.9,53.9,54.0,54.0,54.0,54.0,54.1,54.1,54.1,54.1,54.1,54.1,54.1,54.1,54.1,54.1,54.1,54.1,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,53.9,53.9,53.9,53.9,],
"[+1SD]":[3.7,4.8,5.8,6.6,7.3,7.8,8.2,8.6,9.0,9.3,9.6,9.9,10.1,10.4,10.6,10.9,11.1,11.4,11.6,11.8,12.1,12.3,12.5,12.8,13.0,13.3,13.5,13.7,14.0,14.2,14.4,14.7,14.9,15.1,15.4,15.6,15.6,15.9,16.1,16.3,16.5,16.7,16.9,17.2,17.4,17.6,17.8,18.0,18.2,18.5,18.7,18.9,19.1,19.3,19.6,19.8,20.0,20.2,20.4,20.7,20.9,21.1,21.3,21.6,21.8,22.0,22.3,22.5,22.8,23.1,23.3,23.6,23.9,24.1,24.4,24.7,25.0,25.3,25.6,25.9,26.2,26.5,26.8,27.1,27.5,27.8,28.1,28.5,28.8,29.1,29.5,29.8,30.2,30.5,30.9,31.2,31.6,32.0,32.4,32.8,33.1,33.5,33.9,34.3,34.7,35.1,35.5,35.9,36.3,36.8,37.2,37.6,38.0,38.5,38.9,39.4,39.8,40.3,40.7,41.2,41.7,42.1,42.6,43.1,43.5,44.0,44.5,45.0,45.4,45.9,46.4,46.9,47.3,47.8,48.2,48.7,49.1,49.6,50.0,50.4,50.8,51.3,51.7,52.1,52.5,52.9,53.2,53.6,54.0,54.3,54.6,55.0,55.3,55.6,55.9,56.2,56.4,56.7,56.9,57.2,57.4,57.7,57.9,58.1,58.3,58.5,58.7,58.9,59.0,59.2,59.4,59.5,59.7,59.9,60.0,60.1,60.2,60.4,60.5,60.6,60.7,60.8,60.9,61.0,61.1,61.2,61.3,61.3,61.4,61.4,61.5,61.6,61.6,61.7,61.7,61.7,61.8,61.8,61.8,61.8,61.8,61.8,61.8,61.8,61.8,61.8,61.8,61.8,61.8,61.8,61.7,61.7,61.7,61.7,61.7,61.7,61.7,61.6,61.6,61.6,61.6,61.6,61.6,61.5,61.5,61.5,61.5,61.5,],
"[+2SD]":[4.2,5.5,6.6,7.5,8.2,8.8,9.3,9.8,10.2,10.5,10.9,11.2,11.5,11.8,12.1,12.4,12.6,12.9,13.2,13.5,13.7,14.0,14.3,14.6,14.8,15.1,15.4,15.7,16.0,16.2,16.5,16.8,17.1,17.3,17.6,17.9,17.1,17.4,17.7,18.0,18.3,18.6,18.9,19.1,19.4,19.7,20.0,20.3,20.6,20.9,21.2,21.5,21.8,22.1,22.4,22.7,23.0,23.3,23.6,23.9,24.2,24.5,24.8,25.1,25.4,25.7,26.1,26.4,26.8,27.1,27.5,27.8,28.2,28.6,29.0,29.4,29.8,30.1,30.6,31.0,31.4,31.8,32.2,32.6,33.1,33.5,34.0,34.4,34.9,35.3,35.8,36.2,36.7,37.2,37.6,38.1,38.6,39.1,39.6,40.0,40.5,41.0,41.5,42.1,42.6,43.1,43.6,44.1,44.6,45.2,45.7,46.2,46.8,47.3,47.9,48.4,49.0,49.5,50.1,50.6,51.1,51.7,52.3,52.8,53.4,53.9,54.4,55.0,55.5,56.0,56.6,57.1,57.6,58.1,58.6,59.0,59.5,60.0,60.4,60.9,61.3,61.8,62.2,62.6,63.0,63.4,63.8,64.2,64.6,64.9,65.2,65.5,65.8,66.1,66.4,66.7,67.0,67.2,67.4,67.6,67.9,68.1,68.3,68.4,68.6,68.8,69.0,69.2,69.3,69.4,69.5,69.6,69.8,69.9,70.0,70.0,70.1,70.2,70.3,70.4,70.4,70.5,70.5,70.6,70.6,70.7,70.7,70.8,70.8,70.9,70.9,70.9,71.0,71.0,71.0,71.0,71.0,71.0,71.1,71.1,71.1,71.1,71.1,71.1,71.1,71.1,71.1,71.1,71.1,71.1,71.1,71.2,71.2,71.2,71.2,71.2,71.2,71.2,71.2,71.2,71.2,71.2,71.2,71.2,71.2,71.2,71.2,71.2,],
"[+3SD]":[4.8,6.2,7.5,8.5,9.3,10.0,10.6,11.1,11.6,12.0,12.4,12.8,13.1,13.5,13.8,14.1,14.5,14.8,15.1,15.4,15.7,16.0,16.4,16.7,17.0,17.3,17.7,18.0,18.3,18.7,19.0,19.3,19.6,20.0,20.3,20.6,18.7,19.1,19.4,19.8,20.2,20.6,21.0,21.4,21.8,22.2,22.6,23.0,23.4,23.8,24.2,24.7,25.1,25.5,25.9,26.4,26.8,27.2,27.7,28.1,28.5,29.0,29.5,29.9,30.4,30.8,31.4,31.9,32.4,32.9,33.5,34.0,34.5,35.1,35.6,36.2,36.7,37.3,37.8,38.4,39.0,39.6,40.2,40.7,41.3,41.9,42.5,43.1,43.7,44.3,44.9,45.5,46.1,46.8,47.4,48.0,48.6,49.2,49.8,50.5,51.1,51.7,52.4,53.1,53.7,54.4,55.0,55.7,56.3,57.0,57.6,58.3,58.9,59.5,60.1,60.8,61.4,62.0,62.6,63.2,63.8,64.4,65.0,65.6,66.2,66.7,67.3,67.8,68.4,68.9,69.5,70.0,70.5,70.9,71.4,71.9,72.3,72.8,73.2,73.7,74.1,74.5,74.9,75.4,75.7,76.1,76.5,76.9,77.3,77.7,78.0,78.2,78.5,78.8,79.1,79.4,79.6,79.8,80.1,80.3,80.5,80.7,80.8,81.0,81.1,81.3,81.4,81.6,81.7,81.7,81.8,81.9,81.9,82.0,82.0,82.0,82.0,82.0,82.0,82.0,82.0,82.0,82.0,82.0,82.0,82.0,82.0,82.0,82.0,82.0,82.0,82.1,82.1,82.1,82.1,82.1,82.1,82.1,82.1,82.1,82.2,82.2,82.3,82.3,82.4,82.4,82.5,82.6,82.6,82.7,82.8,82.9,82.9,83.0,83.1,83.2,83.3,83.3,83.4,83.5,83.6,83.7,83.8,83.9,84.0,84.1,84.2,84.3,],
}
pd.DataFrame(df).to_csv('cache/GC_Wt_F.csv')